<h1 align='center'> 영상처리 프로그래밍 중간고사</h1>

<h6 align='right'> 2025. 4. 24. </h6>

<div class="alert alert-block alert-info">
    
- 파일 이름에서 00000000을 자신의 학번으로, name을 자신의 이름으로 수정하세요.

- 다음 줄에 자신의 이름, 학번, 학과(전공)을 적으세요.

* 이름:  이선재 &nbsp;&nbsp;          학번: 20227123   &nbsp;&nbsp;         학과(전공): 빅데이터
    
</div>

## 아래에 있는 두 문제 중에 하나를 풀어라. 단, 2번을 푸는 학생에게는 보너스 점수 30/100 추가함.

## 문제 1. 다음 프로그램애 여러 도형을 추가하라.

도형은 'd', 'e', 's', 'r' 등 다양하게 정의한 것을 여기에 표시하라.


In [1]:
import cv2
import numpy as np
import math
import image_tools

class ImageDrawer:
    def __init__(self, img):

    def draw_rectangle(self, event, x, y, flags, param):


    def get_star_points(self, center_x, center_y, width, height):


    def apply_bitwise_not(self):


    def update_shape(self):

IMAGE_FILE = 'bird.jpg'
WINDOW_MAIN = 'Image'

try:
    img = image_tools.load_image(IMAGE_FILE, 'BGR')


# 문제 2. 여러 메뉴를 추가하라

In [14]:
%gui qt  # Jupyter Notebook에서만 실행 (일반 스크립트에서는 주석 처리)

import sys
import numpy as np
import cv2
import matplotlib
matplotlib.use('Agg')  # 비대화형 백엔드
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QLabel, QAction, QFileDialog, QHBoxLayout, QWidget,
    QVBoxLayout, QDockWidget, QSlider, QGroupBox, QInputDialog
)
from PyQt5.QtCore import Qt, QPoint
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen
import image_tools
import random

def log_transformation(image, f_max=255):
    f_max = float(f_max)
    if f_max <= 0 or np.isnan(f_max) or np.isinf(f_max):
        f_max = 255
    image = image.astype(np.float32)
    image = np.clip(image, 0, 255)
    C = 255 / np.log(1 + f_max + 1e-10)
    return (C * np.log(1. + image)).round().clip(0, 255).astype(np.uint8)

def gamma_transformation(image, gamma=1.0):
    image = image.astype(np.float32)
    image = np.clip(image, 0, 255)
    normalized = image / 255.0
    transformed = np.power(normalized, gamma) * 255.0
    return transformed.round().clip(0, 255).astype(np.uint8)

def grayscale_transformation(image, weight=1.0):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray_rgb = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)
    return (image * (1 - weight) + gray_rgb * weight).round().clip(0, 255).astype(np.uint8)

def sepia_transformation(image):
    sepia_matrix = np.array([
        [0.393, 0.769, 0.189],
        [0.349, 0.686, 0.168],
        [0.272, 0.534, 0.131]
    ])
    sepia = np.dot(image.astype(np.float32), sepia_matrix.T)
    return sepia.round().clip(0, 255).astype(np.uint8)

def cartoon_effect(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray = cv2.medianBlur(gray, 5)
    edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 9)
    color = cv2.bilateralFilter(image, 9, 300, 300)
    color = cv2.bitwise_and(color, color, mask=edges)
    return color

def glitch_effect(image):
    h, w = image.shape[:2]
    noise = np.random.normal(0, 25, (h, w, 3)).astype(np.float32)
    glitched = image.astype(np.float32) + noise
    glitched = np.clip(glitched, 0, 255).astype(np.uint8)
    for _ in range(10):
        y = random.randint(0, h-10)
        shift = random.randint(-20, 20)
        glitched[y:y+10] = np.roll(glitched[y:y+10], shift, axis=1)
    return glitched

def vintage_effect(image):
    image = image.astype(np.float32)
    image = image * np.array([0.8, 0.9, 1.0])
    noise = np.random.normal(0, 10, image.shape).astype(np.float32)
    image = np.clip(image + noise, 0, 255).astype(np.uint8)
    return sepia_transformation(image)

def sketch_effect(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    inv = 255 - gray
    blur = cv2.GaussianBlur(inv, (21, 21), 0)
    sketch = cv2.divide(gray, 255 - blur, scale=256)
    return cv2.cvtColor(sketch, cv2.COLOR_GRAY2RGB)

def mosaic_effect(image, rect, block_size=10):
    (x1, y1, x2, y2) = rect
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(image.shape[1], x2), min(image.shape[0], y2)
    if x2 <= x1 or y2 <= y1:
        return image
    roi = image[y1:y2, x1:x2].copy()
    h, w = roi.shape[:2]
    for i in range(0, h, block_size):
        for j in range(0, w, block_size):
            block = roi[i:i+block_size, j:j+block_size]
            if block.size > 0:
                mean_color = block.mean(axis=(0, 1)).astype(np.uint8)
                roi[i:i+block_size, j:j+block_size] = mean_color
    image[y1:y2, x1:x2] = roi
    return image

def adjust_brightness_contrast(image, brightness=0, contrast=1.0):
    image = image.astype(np.float32)
    image = image * contrast + brightness
    return image.round().clip(0, 255).astype(np.uint8)

def histogram_equalization(image):
    img_yuv = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
    return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)

class ImageLabel(QLabel):
    def __init__(self, viewer, parent=None):
        super().__init__(parent)
        self.viewer = viewer
        self.start_pos = None
        self.end_pos = None
        self.drawing = False
        self.setMouseTracking(True)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.start_pos = event.pos()
            if self.viewer.brush_mode:
                self.viewer.brush_points.append([self.viewer.map_to_image_coords(event.pos(), self.viewer.current_image.shape)])
            self.drawing = True

    def mouseMoveEvent(self, event):
        if self.drawing:
            self.end_pos = event.pos()
            if self.viewer.brush_mode:
                self.viewer.brush_points[-1].append(self.viewer.map_to_image_coords(event.pos(), self.viewer.current_image.shape))
                self.viewer.apply_brush(self.viewer.brush_points)
            self.update()

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.end_pos = event.pos()
            self.drawing = False
            if not self.viewer.brush_mode:
                self.viewer.apply_mosaic_or_crop(self.start_pos, self.end_pos)
            self.start_pos = None
            self.end_pos = None

class BasicViewer(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("영상처리 - 크리에이티브 편집기")
        self.setGeometry(100, 300, 1200, 800)

        # 메인 레이아웃
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)
        self.main_layout = QVBoxLayout(self.central_widget)
        self.main_layout.setContentsMargins(10, 10, 10, 10)

        # 이미지 표시 레이아웃
        self.image_widget = QWidget()
        self.hbox = QHBoxLayout(self.image_widget)
        self.hbox.setContentsMargins(0, 0, 0, 0)

        # 원본 이미지 라벨
        self.original_label = ImageLabel(self, self.image_widget)
        self.original_label.setAlignment(Qt.AlignCenter)
        self.original_label.setMinimumSize(400, 400)
        self.original_label.setText("원본 이미지를 열어보세요")
        self.hbox.addWidget(self.original_label)

        # 변환된 이미지 라벨
        self.transformed_label = QLabel(self)
        self.transformed_label.setAlignment(Qt.AlignCenter)
        self.transformed_label.setMinimumSize(400, 400)
        self.transformed_label.setText("변환된 이미지가 여기에 표시됩니다")
        self.hbox.addWidget(self.transformed_label)

        self.main_layout.addWidget(self.image_widget)

        self.current_image = None
        self.transformed_image = None
        self.brush_mode = False
        self.brush_points = []
        self.mode = None

        # 변환 조절 도킹 위젯
        self.transform_dock = QDockWidget("변환 조절", self)
        self.transform_dock.setAllowedAreas(Qt.RightDockWidgetArea)
        self.transform_dock.setFeatures(QDockWidget.DockWidgetMovable | QDockWidget.DockWidgetClosable)
        self.transform_widget = QWidget()
        self.transform_layout = QVBoxLayout(self.transform_widget)
        self.transform_layout.setContentsMargins(10, 10, 10, 10)

        # 슬라이더 그룹
        self.slider_group = QGroupBox("조절 패널")
        self.slider_layout = QVBoxLayout(self.slider_group)

        # 흑백 변환 슬라이더
        self.grayscale_label = QLabel("흑백 강도: 0%")
        self.grayscale_slider = QSlider(Qt.Horizontal)
        self.grayscale_slider.setRange(0, 100)
        self.grayscale_slider.setValue(0)
        self.grayscale_slider.valueChanged.connect(self.apply_grayscale_transform)
        self.slider_layout.addWidget(self.grayscale_label)
        self.slider_layout.addWidget(self.grayscale_slider)

        # 감마 변환 슬라이더
        self.gamma_label = QLabel("감마 값: 1.0")
        self.gamma_slider = QSlider(Qt.Horizontal)
        self.gamma_slider.setRange(10, 500)
        self.gamma_slider.setValue(100)
        self.gamma_slider.valueChanged.connect(self.apply_gamma_transform)
        self.slider_layout.addWidget(self.gamma_label)
        self.slider_layout.addWidget(self.gamma_slider)

        # 밝기 슬라이더
        self.brightness_label = QLabel("밝기: 0")
        self.brightness_slider = QSlider(Qt.Horizontal)
        self.brightness_slider.setRange(-100, 100)
        self.brightness_slider.setValue(0)
        self.brightness_slider.valueChanged.connect(self.apply_brightness_contrast)
        self.slider_layout.addWidget(self.brightness_label)
        self.slider_layout.addWidget(self.brightness_slider)

        # 대비 슬라이더
        self.contrast_label = QLabel("대비: 1.0")
        self.contrast_slider = QSlider(Qt.Horizontal)
        self.contrast_slider.setRange(10, 300)
        self.contrast_slider.setValue(100)
        self.contrast_slider.valueChanged.connect(self.apply_brightness_contrast)
        self.slider_layout.addWidget(self.contrast_label)
        self.slider_layout.addWidget(self.contrast_slider)

        self.transform_layout.addWidget(self.slider_group)
        self.transform_dock.setWidget(self.transform_widget)
        self.addDockWidget(Qt.RightDockWidgetArea, self.transform_dock)
        self.transform_dock.setVisible(False)

        # 초기 이미지 로드
        self.load_image('forest.jpg')

        self.create_menu()

    def create_menu(self):
        menubar = self.menuBar()

        # 파일 메뉴
        file_menu = menubar.addMenu("파일")
        open_action = QAction("열기", self)
        open_action.triggered.connect(self.open_image_dialog)
        file_menu.addAction(open_action)
        exit_action = QAction("종료", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        # 변환 메뉴
        transform_menu = menubar.addMenu("변환")
        log_action = QAction("로그 변환", self)
        log_action.triggered.connect(self.apply_log_transform)
        transform_menu.addAction(log_action)
        gamma_action = QAction("감마 변환", self)
        gamma_action.triggered.connect(self.apply_gamma_transform_menu)
        transform_menu.addAction(gamma_action)
        control_action = QAction("변환 조절", self)
        control_action.triggered.connect(self.toggle_transform_dock)
        transform_menu.addAction(control_action)

        # 보기 메뉴
        view_menu = menubar.addMenu("보기")
        hist_action = QAction("히스토그램 보기", self)
        hist_action.triggered.connect(self.show_histogram)
        view_menu.addAction(hist_action)
        hist_eq_action = QAction("히스토그램 이퀄라이제이션", self)
        hist_eq_action.triggered.connect(self.apply_histogram_equalization)
        view_menu.addAction(hist_eq_action)
        info_action = QAction("이미지 정보", self)
        info_action.triggered.connect(self.show_image_info)
        view_menu.addAction(info_action)

        # 효과 메뉴
        effect_menu = menubar.addMenu("효과")
        sepia_action = QAction("세피아 톤", self)
        sepia_action.triggered.connect(self.apply_sepia_transform)
        effect_menu.addAction(sepia_action)
        cartoon_action = QAction("만화 효과", self)
        cartoon_action.triggered.connect(self.apply_cartoon_transform)
        effect_menu.addAction(cartoon_action)
        glitch_action = QAction("글리치 효과", self)
        glitch_action.triggered.connect(self.apply_glitch_transform)
        effect_menu.addAction(glitch_action)
        vintage_action = QAction("빈티지 필름", self)
        vintage_action.triggered.connect(self.apply_vintage_transform)
        effect_menu.addAction(vintage_action)
        sketch_action = QAction("스케치 효과", self)
        sketch_action.triggered.connect(self.apply_sketch_transform)
        effect_menu.addAction(sketch_action)

        # 도구 메뉴
        tool_menu = menubar.addMenu("도구")
        mosaic_action = QAction("모자이크", self)
        mosaic_action.triggered.connect(self.enable_mosaic_mode)
        tool_menu.addAction(mosaic_action)
        crop_action = QAction("크롭", self)
        crop_action.triggered.connect(self.enable_crop_mode)
        tool_menu.addAction(crop_action)
        text_action = QAction("텍스트 삽입", self)
        text_action.triggered.connect(self.insert_text)
        tool_menu.addAction(text_action)
        brush_action = QAction("브러시", self)
        brush_action.triggered.connect(self.toggle_brush_mode)
        tool_menu.addAction(brush_action)

    def load_image(self, filename):
        try:
            img = image_tools.load_image(filename, color_channel="BGR")
            if not isinstance(img, np.ndarray):
                raise ValueError(f"Could not load {filename}")
            self.current_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            self.current_image = np.ascontiguousarray(self.current_image, dtype=np.uint8)
            self.transformed_image = None
            self.update_image()
            self.update_transformed_image()
            self.grayscale_slider.setValue(0)
            self.gamma_slider.setValue(100)
            self.brightness_slider.setValue(0)
            self.contrast_slider.setValue(100)
            self.grayscale_label.setText("흑백 강도: 0%")
            self.gamma_label.setText("감마 값: 1.0")
            self.brightness_label.setText("밝기: 0")
            self.contrast_label.setText("대비: 1.0")
            self.brush_mode = False
            self.brush_points = []
            self.mode = None
        except Exception as e:
            self.original_label.setText(f"이미지 로드 실패: {str(e)}")
            self.transformed_label.setText("변환된 이미지가 여기에 표시됩니다")

    def open_image_dialog(self):
        filename, _ = QFileDialog.getOpenFileName(
            self,
            "이미지 열기",
            "",
            "Image Files (*.png *.jpg *.jpeg *.bmp *.tif *.tiff)"
        )
        if filename:
            self.load_image(filename)

    def numpy_to_qpixmap(self, image):
        if image is None:
            return None
        h, w, ch = image.shape
        if ch != 3:
            return None
        bytes_per_line = ch * w
        image = np.ascontiguousarray(image, dtype=np.uint8)
        qimage = QImage(image.data, w, h, bytes_per_line, QImage.Format_RGB888)
        if qimage.isNull():
            return None
        return QPixmap.fromImage(qimage)

    def update_image(self):
        pixmap = self.numpy_to_qpixmap(self.current_image)
        if pixmap is not None:
            self.original_label.setPixmap(pixmap.scaled(
                self.original_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation
            ))
        else:
            self.original_label.setText("원본 이미지를 열어보세요")

    def update_transformed_image(self):
        pixmap = self.numpy_to_qpixmap(self.transformed_image)
        if pixmap is not None:
            self.transformed_label.setPixmap(pixmap.scaled(
                self.transformed_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation
            ))
        else:
            self.transformed_label.setText("변환된 이미지가 여기에 표시됩니다")

    def map_to_image_coords(self, pos, image_shape):
        """마우스 좌표를 이미지 좌표로 변환"""
        if self.current_image is None:
            return pos.x(), pos.y()
        pixmap = self.original_label.pixmap()
        if not pixmap or pixmap.isNull():
            return pos.x(), pos.y()

        label_size = self.original_label.size()
        pixmap_size = pixmap.size()

        # QLabel 내에서 픽스맵의 실제 표시 영역 계산
        aspect_ratio = pixmap_size.width() / pixmap_size.height()
        label_aspect = label_size.width() / label_size.height()

        if aspect_ratio > label_aspect:
            # 픽스맵이 너비 기준으로 스케일링
            scaled_width = label_size.width()
            scaled_height = int(scaled_width / aspect_ratio)
            offset_x = 0
            offset_y = (label_size.height() - scaled_height) // 2
        else:
            # 픽스맵이 높이 기준으로 스케일링
            scaled_height = label_size.height()
            scaled_width = int(scaled_height * aspect_ratio)
            offset_x = (label_size.width() - scaled_width) // 2
            offset_y = 0

        # 마우스 좌표를 픽스맵 좌표로 변환
        x = pos.x() - offset_x
        y = pos.y() - offset_y

        # 픽스맵 좌표를 원본 이미지 좌표로 변환
        scale_x = image_shape[1] / scaled_width
        scale_y = image_shape[0] / scaled_height

        img_x = int(x * scale_x)
        img_y = int(y * scale_y)

        # 이미지 범위 내로 제한
        img_x = max(0, min(img_x, image_shape[1] - 1))
        img_y = max(0, min(img_y, image_shape[0] - 1))

        return img_x, img_y

    def apply_log_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = log_transformation(self.current_image, f_max=self.current_image.max())
        self.update_transformed_image()

    def apply_grayscale_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        weight = self.grayscale_slider.value() / 100.0
        self.grayscale_label.setText(f"흑백 강도: {int(weight * 100)}%")
        self.transformed_image = grayscale_transformation(self.current_image, weight)
        self.update_transformed_image()

    def apply_gamma_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        gamma = self.gamma_slider.value() / 100.0
        self.gamma_label.setText(f"감마 값: {gamma:.2f}")
        self.transformed_image = gamma_transformation(self.current_image, gamma)
        self.update_transformed_image()

    def apply_gamma_transform_menu(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        gamma, ok = QInputDialog.getDouble(
            self, "감마 변환", "감마 값을 입력하세요:", 1.0, 0.1, 5.0, 2
        )
        if ok:
            self.transformed_image = gamma_transformation(self.current_image, gamma)
            self.update_transformed_image()

    def apply_sepia_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = sepia_transformation(self.current_image)
        self.update_transformed_image()

    def apply_cartoon_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = cartoon_effect(self.current_image)
        self.update_transformed_image()

    def apply_glitch_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = glitch_effect(self.current_image)
        self.update_transformed_image()

    def apply_vintage_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = vintage_effect(self.current_image)
        self.update_transformed_image()

    def apply_sketch_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = sketch_effect(self.current_image)
        self.update_transformed_image()

    def apply_brightness_contrast(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        brightness = self.brightness_slider.value()
        contrast = self.contrast_slider.value() / 100.0
        self.brightness_label.setText(f"밝기: {brightness}")
        self.contrast_label.setText(f"대비: {contrast:.2f}")
        self.transformed_image = adjust_brightness_contrast(self.current_image, brightness, contrast)
        self.update_transformed_image()

    def apply_histogram_equalization(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = histogram_equalization(self.current_image)
        self.update_transformed_image()

    def show_histogram(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        fig, ax = plt.subplots(figsize=(6, 4))
        colors = ('r', 'g', 'b')
        for i, color in enumerate(colors):
            hist = np.histogram(self.current_image[:, :, i], bins=256, range=(0, 256))[0]
            ax.plot(hist, color=color, label=f'{color.upper()} 채널')
        ax.set_title('RGB 히스토그램')
        ax.set_xlabel('픽셀 강도')
        ax.set_ylabel('빈도')
        ax.legend()
        ax.grid(True)

        # Matplotlib 플롯을 이미지로 변환
        canvas = FigureCanvasAgg(fig)
        canvas.draw()
        width, height = fig.get_size_inches() * fig.get_dpi()
        width, height = int(width), int(height)
        image = np.frombuffer(canvas.tostring_rgb(), dtype=np.uint8)
        image = image.reshape(height, width, 3)
        self.transformed_image = image
        self.update_transformed_image()
        plt.close(fig)

    def show_image_info(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        h, w, ch = self.current_image.shape
        info = f"크기: {w}x{h}\n채널: {ch}\n데이터 타입: {self.current_image.dtype}"
        self.transformed_label.setText(info)

    def enable_mosaic_mode(self):
        self.brush_mode = False
        self.brush_points = []
        self.mode = 'mosaic'
        self.transformed_label.setText("마우스로 원본 이미지에서 모자이크 영역을 드래그하세요")

    def enable_crop_mode(self):
        self.brush_mode = False
        self.brush_points = []
        self.mode = 'crop'
        self.transformed_label.setText("마우스로 원본 이미지에서 크롭 영역을 드래그하세요")

    def toggle_brush_mode(self):
        self.brush_mode = not self.brush_mode
        self.brush_points = []
        self.mode = 'brush' if self.brush_mode else None
        if self.brush_mode:
            self.transformed_label.setText("마우스로 원본 이미지에 그림을 그리세요")
        else:
            self.update_transformed_image()

    def apply_mosaic_or_crop(self, start_pos, end_pos):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        x1, y1 = self.map_to_image_coords(start_pos, self.current_image.shape)
        x2, y2 = self.map_to_image_coords(end_pos, self.current_image.shape)
        rect = (min(x1, x2), min(y1, y2), max(x1, x2), max(y1, y2))
        if self.mode == 'crop':
            x1, y1, x2, y2 = rect
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(self.current_image.shape[1], x2), min(self.current_image.shape[0], y2)
            if x2 > x1 and y2 > y1:
                self.current_image = self.current_image[y1:y2, x1:x2].copy()
                self.transformed_image = None
                self.update_image()
                self.update_transformed_image()
        elif self.mode == 'mosaic':
            self.transformed_image = mosaic_effect(self.current_image.copy(), rect, block_size=10)
            self.update_transformed_image()

    def insert_text(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        text, ok = QInputDialog.getText(self, "텍스트 삽입", "삽입할 텍스트를 입력하세요:")
        if ok and text:
            img = self.current_image.copy()
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(img, text, (50, 50), font, 1, (255, 0, 0), 2, cv2.LINE_AA)
            self.transformed_image = img
            self.update_transformed_image()

    def apply_brush(self, brush_points):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        img = self.current_image.copy()
        for points in brush_points:
            for i in range(1, len(points)):
                start = points[i-1]
                end = points[i]
                cv2.line(img, (start[0], start[1]), (end[0], end[1]), (255, 0, 0), 5)
        self.transformed_image = img
        self.update_transformed_image()

    def toggle_transform_dock(self):
        self.transform_dock.setVisible(not self.transform_dock.isVisible())

    def showEvent(self, event):
        super().showEvent(event)
        self.update_image()
        self.update_transformed_image()

    def resizeEvent(self, event):
        super().resizeEvent(event)
        self.update_image()
        self.update_transformed_image()

# QApplication 인스턴스 관리
app = QApplication.instance()
if app is None:
    app = QApplication(sys.argv)
else:
    print("기존 QApplication 인스턴스를 사용합니다.")

viewer = BasicViewer()
viewer.show()

# Jupyter와 일반 스크립트 구분
if 'IPython' in sys.modules:
    app.exec_()
else:
    sys.exit(app.exec_())

ERROR:root:Invalid GUI request 'qt # Jupyter Notebook에서만 실행 (일반 스크립트에서는 주석 처리)', valid ones are:dict_keys(['inline', 'nbagg', 'webagg', 'notebook', 'ipympl', 'widget', None, 'qt', 'qt5', 'qt6', 'wx', 'tk', 'gtk', 'gtk3', 'osx', 'asyncio'])


기존 QApplication 인스턴스를 사용합니다.


# GPT와의 대화

- Grok AI

## 질문

import sys
from PyQt5.QtWidgets import (
    QApplication, 
    QMainWindow, 
    QLabel,
    QMenuBar, QAction,
    QFileDialog, QHBoxLayout, QWidget
)
from PyQt5.QtCore import Qt 
from PyQt5.QtGui import QPixmap, QImage
import image_tools  # 사용자 정의 모듈 (예: OpenCV 기반)
import numpy as np

def log_transformation(image, f_max=255):
    C = 255 / np.log(1 + f_max)
    return (C * np.log(1. + image)).round().clip(0, 255).astype(np.uint8)

class BasicViewer(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle()
        self.setGeometry()

        # QLabel 두 개: 원본 / 변환
        self.original_label = QLabel("원본 이미지를 열어보세요")
        self.original_label.setAlignment(Qt.AlignCenter)

        self.transformed_label = QLabel("변환된 이미지가 여기에 표시됩니다")
        self.transformed_label.setAlignment(Qt.AlignCenter)

        # 수평 레이아웃
        self.central_widget = QWidget()
        self.hbox = QHBoxLayout(self.central_widget)
        self.hbox.addWidget(self.original_label)
        self.hbox.addWidget(self.transformed_label)

        self.setCentralWidget(self.central_widget)

        self.current_image = None  # 원본 이미지 저장용 (NumPy)

        self.create_menu()

    def create_menu(self):
        menubar = self.menuBar()

        # 파일 메뉴
        file_menu = menubar.addMenu("파일")
        # 열기, 종류 부메뉴 추가

        # 변환 메뉴
        transform_menu = menubar.addMenu("변환")
        # 다양한 변화 추가
    
    def open_image_dialog(self):

    def apply_log_transform(self):

    def numpy_to_qpixmap(self, image):
        h, w, ch = image.shape
        bytes_per_line = ch * w
        qimage = QImage(image.data, w, h, bytes_per_line, QImage.Format_RGB888)
        return QPixmap.fromImage(qimage)

app = QApplication.instance()
if app is None:
    app = QApplication([])

viewer = BasicViewer()
viewer.show()

try:
    sys.exit(app.exec_())
except SystemExit:
    pass

를 
import sys
import numpy as np
from PyQt5.QtWidgets import (
    QApplication,
    QMainWindow,
    QLabel,
    QAction,
    QFileDialog,
    QWidget,
    QHBoxLayout,
)
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QPixmap, QImage
import image_tools

class BasicViewer(QMainWindow):
    def __init__(self):
        super().__init__()
        
        self.setWindowTitle("영상처리")
        self.setGeometry(100, 300, 800, 600)
        
        # 중앙 위젯 설정: QHBoxLayout으로 두 이미지 나란히 표시
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)
        self.layout = QHBoxLayout(self.central_widget)
        self.layout.setContentsMargins(10, 10, 10, 10)
        
        # 원본 이미지 라벨
        self.image_label = QLabel("이미지", self)
        self.image_label.setAlignment(Qt.AlignCenter)
        self.image_label.setMinimumSize(200, 200)
        self.layout.addWidget(self.image_label)
        self.layout.setStretch(0, 1)
        
        # 변환된 이미지 라벨
        self.transformed_label = QLabel("변환된 이미지", self)
        self.transformed_label.setAlignment(Qt.AlignCenter)
        self.transformed_label.setMinimumSize(200, 200)
        self.layout.addWidget(self.transformed_label)
        self.layout.setStretch(1, 1)
        
        # 초기 이미지 로드
        self.img = image_tools.load_image('forest.jpg')
        if self.img is not None:
            self.img = np.ascontiguousarray(self.img[:, :, ::-1], dtype=np.uint8)  # BGR to RGB
        else:
            self.image_label.setText("이미지 로드 실패")
            self.transformed_label.setText("이미지 없음")
       
        self.create_menu()

    def create_menu(self):
        menubar = self.menuBar()
        file_menu = menubar.addMenu("파일")
        open_action = QAction("열기", self)
        open_action.triggered.connect(self.open_image_dialog)
        file_menu.addAction(open_action)
        exit_action = QAction("종료", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)
        
        transform_menu = menubar.addMenu("변환")
        log_action = QAction("로그 변환", self)
        log_action.triggered.connect(self.apply_log_transform)
        transform_menu.addAction(log_action)
        gamma_action = QAction("감마 변환", self)  # 감마 변환 추가
        gamma_action.triggered.connect(self.apply_gamma_transform)
        transform_menu.addAction(gamma_action)

    def update_image(self):
        if self.img is not None:
            h, w, ch = self.img.shape
            if ch != 3:
                self.image_label.setText("오류: RGB 이미지만 지원됩니다")
                return
            bytes_per_line = ch * w
            self.img = np.ascontiguousarray(self.img, dtype=np.uint8)
            qimg = QImage(self.img.data, w, h, bytes_per_line, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(qimg)
            label_size = self.image_label.size()
            self.image_label.setPixmap(pixmap.scaled(
                label_size, Qt.KeepAspectRatio, Qt.SmoothTransformation
            ))
        else:
            self.image_label.setText("이미지 로드 실패")

    def update_transformed_image(self):
        if hasattr(self, 'transformed_img') and self.transformed_img is not None:
            h, w, ch = self.transformed_img.shape
            if ch != 3:
                self.transformed_label.setText("오류: RGB 이미지만 지원됩니다")
                return
            bytes_per_line = ch * w
            self.transformed_img = np.ascontiguousarray(self.transformed_img, dtype=np.uint8)
            qimg = QImage(self.transformed_img.data, w, h, bytes_per_line, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(qimg)
            label_size = self.transformed_label.size()
            self.transformed_label.setPixmap(pixmap.scaled(
                label_size, Qt.KeepAspectRatio, Qt.SmoothTransformation
            ))
        else:
            self.transformed_label.setText("변환된 이미지 없음")

    def showEvent(self, event):
        super().showEvent(event)
        self.update_image()
        self.update_transformed_image()

    def resizeEvent(self, event):
        super().resizeEvent(event)
        self.update_image()
        self.update_transformed_image()

    def open_image_dialog(self):
        filename, _ = QFileDialog.getOpenFileName(
            self,
            "이미지 열기",
            "",
            "Image Files (*.png *.jpg *.jpeg *.bmp *.tif *.tiff)"
        )
        if filename:
            image = image_tools.load_image(filename)
            if image is not None:
                self.img = np.ascontiguousarray(image[:, :, ::-1], dtype=np.uint8)
                self.update_image()
                self.transformed_label.clear()
            else:
                self.image_label.setText("이미지 로드 실패")
                self.transformed_label.setText("이미지 없음")

    def apply_log_transform(self):
        def log_transformation(image, f_max=255):
            f_max = float(f_max)
            if f_max <= 0 or np.isnan(f_max) or np.isinf(f_max):
                f_max = 255
            image = image.astype(np.float32)
            image = np.clip(image, 0, 255)
            C = 255 / np.log(1 + f_max + 1e-10)
            return (C * np.log(1. + image)).round().clip(0, 255).astype(np.uint8)

        if self.img is None:
            self.transformed_label.setText("이미지 없음")
            return

        transformed = log_transformation(self.img, f_max=self.img.max())
        self.transformed_img = transformed
        self.update_transformed_image()

    def apply_gamma_transform(self):
        def gamma_transformation(image, gamma=0.5):
            image = image.astype(np.float32)
            image = np.clip(image, 0, 255)  # 값 범위 제한
            normalized = image / 255.0  # 0~1로 정규화
            transformed = np.power(normalized, gamma) * 255.0  # 감마 변환
            return transformed.round().clip(0, 255).astype(np.uint8)

        if self.img is None:
            self.transformed_label.setText("이미지 없음")
            return

        transformed = gamma_transformation(self.img)
        self.transformed_img = transformed
        self.update_transformed_image()

app = QApplication.instance()
if app is None:
    app = QApplication([])
viewer = BasicViewer()
viewer.show()
try:
    sys.exit(app.exec_())
except SystemExit:
    pass
요거처럼 만들어줘 다만 처음에 있던 내용을 위주로 작성해줘

## 답변

요청하신 대로 첫 번째 코드(`log_transformation` 함수와 두 개의 `QLabel`을 사용한 원본/변환 이미지 표시)를 기반으로 하되, 두 번째 코드의 기능(로그 변환, 감마 변환, 이미지 열기 다이얼로그, 창 크기 조정 등)을 통합하여 수정된 코드를 작성하겠습니다. `image_tools.load_image`는 OpenCV로 대체하고, Jupyter Notebook, VS Code, PyCharm 등 모든 환경에서 동작하도록 최적화하겠습니다. 또한, 첫 번째 코드의 구조(수평 레이아웃, 메뉴바, 원본/변환 이미지 표시)를 최대한 유지하겠습니다.

### 요구사항 정리
1. **첫 번째 코드의 구조 유지**:
   - 두 개의 `QLabel`(원본 이미지와 변환된 이미지)을 수평 레이아웃(`QHBoxLayout`)으로 나란히 표시.
   - `log_transformation` 함수 사용.
   - 메뉴바에 "파일" 메뉴(열기, 종료)와 "변환" 메뉴(로그 변환 등) 추가.
   - `numpy_to_qpixmap` 메서드 사용.
2. **두 번째 코드의 기능 통합**:
   - "파일" > "열기"로 이미지 선택 다이얼로그 표시.
   - "변환" 메뉴에 로그 변환과 감마 변환 추가.
   - 창 크기 변경 시 이미지 자동 스케일링(`resizeEvent`).
   - 이미지 로드 및 변환 에러 처리.
   - `showEvent`와 `resizeEvent`로 초기 및 동적 이미지 업데이트.
3. **환경 호환성**:
   - Jupyter Notebook에서 `%gui qt` 사용.
   - 일반 Python 스크립트(VS Code, PyCharm) 지원.
4. **`image_tools` 대체**:
   - `image_tools.load_image` 대신 OpenCV 사용.
5. **초기 이미지**:
   - 첫 번째 코드에서는 초기 이미지가 없지만, 두 번째 코드처럼 'forest.jpg'를 초기 이미지로 로드.

### 수정된 코드

```python
%gui qt  # Jupyter Notebook에서만 실행 (일반 스크립트에서는 주석 처리)

import sys
import numpy as np
import cv2  # image_tools 대체
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QLabel, QAction, QFileDialog, QHBoxLayout, QWidget
)
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QPixmap, QImage

def log_transformation(image, f_max=255):
    """로그 변환 함수"""
    f_max = float(f_max)
    if f_max <= 0 or np.isnan(f_max) or np.isinf(f_max):
        f_max = 255
    image = image.astype(np.float32)
    image = np.clip(image, 0, 255)
    C = 255 / np.log(1 + f_max + 1e-10)
    return (C * np.log(1. + image)).round().clip(0, 255).astype(np.uint8)

class BasicViewer(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("영상처리")
        self.setGeometry(100, 300, 800, 600)

        # QLabel 두 개: 원본 / 변환
        self.original_label = QLabel("원본 이미지를 열어보세요", self)
        self.original_label.setAlignment(Qt.AlignCenter)
        self.original_label.setMinimumSize(200, 200)

        self.transformed_label = QLabel("변환된 이미지가 여기에 표시됩니다", self)
        self.transformed_label.setAlignment(Qt.AlignCenter)
        self.transformed_label.setMinimumSize(200, 200)

        # 수평 레이아웃
        self.central_widget = QWidget(self)
        self.hbox = QHBoxLayout(self.central_widget)
        self.hbox.setContentsMargins(10, 10, 10, 10)
        self.hbox.addWidget(self.original_label)
        self.hbox.addWidget(self.transformed_label)
        self.hbox.setStretch(0, 1)
        self.hbox.setStretch(1, 1)
        self.setCentralWidget(self.central_widget)

        self.current_image = None  # 원본 이미지 저장용 (NumPy)

        # 초기 이미지 로드
        self.load_image('forest.jpg')

        self.create_menu()

    def create_menu(self):
        menubar = self.menuBar()

        # 파일 메뉴
        file_menu = menubar.addMenu("파일")
        open_action = QAction("열기", self)
        open_action.triggered.connect(self.open_image_dialog)
        file_menu.addAction(open_action)
        exit_action = QAction("종료", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        # 변환 메뉴
        transform_menu = menubar.addMenu("변환")
        log_action = QAction("로그 변환", self)
        log_action.triggered.connect(self.apply_log_transform)
        transform_menu.addAction(log_action)
        gamma_action = QAction("감마 변환", self)
        gamma_action.triggered.connect(self.apply_gamma_transform)
        transform_menu.addAction(gamma_action)

    def load_image(self, filename):
        """이미지 로드 및 원본 이미지 표시"""
        try:
            img = cv2.imread(filename)
            if img is None:
                raise FileNotFoundError(f"이미지 파일 '{filename}'을 찾을 수 없습니다.")
            # BGR에서 RGB로 변환
            self.current_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            self.current_image = np.ascontiguousarray(self.current_image, dtype=np.uint8)
            self.update_image()
            self.transformed_label.setText("변환된 이미지가 여기에 표시됩니다")
        except Exception as e:
            self.original_label.setText(f"이미지 로드 실패: {str(e)}")
            self.transformed_label.setText("변환된 이미지가 여기에 표시됩니다")

    def open_image_dialog(self):
        """파일 다이얼로그로 이미지 열기"""
        filename, _ = QFileDialog.getOpenFileName(
            self,
            "이미지 열기",
            "",
            "Image Files (*.png *.jpg *.jpeg *.bmp *.tif *.tiff)"
        )
        if filename:
            self.load_image(filename)

    def apply_log_transform(self):
        """로그 변환 적용"""
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = log_transformation(self.current_image, f_max=self.current_image.max())
        self.update_transformed_image()

    def apply_gamma_transform(self):
        """감마 변환 적용"""
        def gamma_transformation(image, gamma=0.5):
            image = image.astype(np.float32)
            image = np.clip(image, 0, 255)
            normalized = image / 255.0
            transformed = np.power(normalized, gamma) * 255.0
            return transformed.round().clip(0, 255).astype(np.uint8)

        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = gamma_transformation(self.current_image)
        self.update_transformed_image()

    def numpy_to_qpixmap(self, image):
        """NumPy 배열을 QPixmap으로 변환"""
        if image is None:
            return None
        h, w, ch = image.shape
        if ch != 3:
            return None
        bytes_per_line = ch * w
        image = np.ascontiguousarray(image, dtype=np.uint8)
        qimage = QImage(image.data, w, h, bytes_per_line, QImage.Format_RGB888)
        if qimage.isNull():
            return None
        return QPixmap.fromImage(qimage)

    def update_image(self):
        """원본 이미지 업데이트"""
        pixmap = self.numpy_to_qpixmap(self.current_image)
        if pixmap is not None:
            self.original_label.setPixmap(pixmap.scaled(
                self.original_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation
            ))
        else:
            self.original_label.setText("이미지 로드 실패")

    def update_transformed_image(self):
        """변환된 이미지 업데이트"""
        pixmap = self.numpy_to_qpixmap(self.transformed_image)
        if pixmap is not None:
            self.transformed_label.setPixmap(pixmap.scaled(
                self.transformed_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation
            ))
        else:
            self.transformed_label.setText("변환된 이미지가 없음")

    def showEvent(self, event):
        """창 표시 시 이미지 업데이트"""
        super().showEvent(event)
        self.update_image()
        self.update_transformed_image()

    def resizeEvent(self, event):
        """창 크기 변경 시 이미지 업데이트"""
        super().resizeEvent(event)
        self.update_image()
        self.update_transformed_image()

# QApplication 인스턴스 관리
app = QApplication.instance()
if app is None:
    app = QApplication(sys.argv)  # sys.argv 사용
else:
    print("기존 QApplication 인스턴스를 사용합니다.")

viewer = BasicViewer()
viewer.show()

# Jupyter와 일반 스크립트 구분
if 'IPython' in sys.modules:
    app.exec_()  # Jupyter 환경
else:
    sys.exit(app.exec_())  # 일반 Python 스크립트
```

### 주요 특징 및 수정 사항

1. **첫 번째 코드 구조 유지**:
   - 두 개의 `QLabel`(원본: `original_label`, 변환: `transformed_label`)을 `QHBoxLayout`으로 나란히 배치.
   - `log_transformation` 함수는 첫 번째 코드의 정의를 그대로 사용하되, 두 번째 코드의 안정성 개선(부동소수점 처리, 클리핑 등)을 반영.
   - `numpy_to_qpixmap` 메서드를 유지하여 NumPy 배열을 `QPixmap`으로 변환.
   - 메뉴바에 "파일" 메뉴(열기, 종료)와 "변환" 메뉴(로그 변환, 감마 변환) 추가.

2. **두 번째 코드의 기능 통합**:
   - `open_image_dialog`로 파일 선택 다이얼로그를 통해 이미지를 로드.
   - "변환" 메뉴에 로그 변환(`apply_log_transform`)과 감마 변환(`apply_gamma_transform`) 추가.
   - `showEvent`와 `resizeEvent`로 창 표시 및 크기 변경 시 이미지 자동 스케일링.
   - `update_image`와 `update_transformed_image`로 원본/변환 이미지 업데이트를 공통화.
   - 초기 이미지로 'forest.jpg'를 로드(파일이 없으면 에러 메시지 표시).

3. **`image_tools` 대체**:
   - `image_tools.load_image` 대신 OpenCV(`cv2.imread`) 사용.
   - BGR에서 RGB로 변환하고, 메모리 연속성 확인(`np.ascontiguousarray`)으로 안정성 보장.

4. **환경 호환성**:
   - Jupyter Notebook: `%gui qt` 추가, `IPython` 확인으로 `sys.exit` 생략.
   - VS Code/PyCharm: `sys.argv` 사용, `sys.exit(app.exec_())`로 정상 종료.

5. **에러 처리**:
   - 이미지 로드 실패, 잘못된 채널 수, QImage 생성 실패 시 `QLabel`에 에러 메시지 표시.
   - 변환 적용 시 원본 이미지가 없는 경우 메시지 표시.

### 실행 방법

#### Jupyter Notebook
1. 새 Jupyter Notebook을 엽니다.
2. 첫 번째 셀에 다음을 실행:
   ```python
   %gui qt
   ```
3. 두 번째 셀에 위 코드를 붙여넣고 실행합니다.
4. 'forest.jpg' 파일이 Jupyter Notebook의 작업 디렉토리에 있는지 확인하세요.
5. 실행 후:
   - 800x600 창이 열리고, 'forest.jpg'가 왼쪽(`original_label`)에 표시됩니다.
   - "파일" > "열기"로 새 이미지를 선택하면 원본 이미지로 업데이트.
   - "변환" > "로그 변환" 또는 "감마 변환"을 클릭하면 오른쪽(`transformed_label`)에 변환된 이미지가 표시됩니다.
   - "파일" > "종료"로 창 닫기.

#### VS Code 또는 PyCharm
1. 위 코드를 `.py` 파일(예: `image_viewer.py`)로 저장합니다.
2. 'forest.jpg' 파일이 스크립트와 동일한 디렉토리에 있는지 확인합니다.
3. 터미널 또는 IDE에서 실행:
   ```bash
   python image_viewer.py
   ```
4. `%gui qt` 줄은 주석 처리하거나 제거하세요.
5. 동일한 동작 확인.

### 예상 출력
- 800x600 창에 두 개의 `QLabel`이 수평으로 배치.
- 왼쪽: 'forest.jpg' 원본 이미지(또는 "이미지 로드 실패" 메시지).
- 오른쪽: 변환된 이미지(초기에는 "변환된 이미지가 여기에 표시됩니다").
- 메뉴바:
  - "파일" > "열기": 파일 선택 다이얼로그로 새 이미지 로드.
  - "파일" > "종료": 창 닫기.
  - "변환" > "로그 변환": 원본 이미지에 로그 변환 적용.
  - "변환" > "감마 변환": 원본 이미지에 감마 변환(기본값 gamma=0.5) 적용.
- 창 크기 변경 시 양쪽 이미지가 비율을 유지하며 스케일링.

### 추가 고려사항

1. **`image_tools` 모듈**:
   - `image_tools`가 특정 모듈이라면 정의를 알려주시면 OpenCV 대신 사용하도록 수정하겠습니다.
   - 예: PIL 사용 시:
     ```python
     from PIL import Image
     def load_image(self, filename):
         try:
             self.current_image = np.array(Image.open(filename).convert('RGB'))
             self.update_image()
             self.transformed_label.setText("변환된 이미지가 여기에 표시됩니다")
         except Exception as e:
             self.original_label.setText(f"이미지 로드 실패: {str(e)}")
     ```

2. **히스토그램 표시**:
   - 이전 질문에서 히스토그램 표시를 요청하셨으므로, "변환" 메뉴에 히스토그램 보기 기능을 추가할 수 있습니다:
     ```python
     import matplotlib.pyplot as plt

     def create_menu(self):
         menubar = self.menuBar()
         file_menu = menubar.addMenu("파일")
         open_action = QAction("열기", self)
         open_action.triggered.connect(self.open_image_dialog)
         file_menu.addAction(open_action)
         exit_action = QAction("종료", self)
         exit_action.triggered.connect(self.close)
         file_menu.addAction(exit_action)

         transform_menu = menubar.addMenu("변환")
         log_action = QAction("로그 변환", self)
         log_action.triggered.connect(self.apply_log_transform)
         transform_menu.addAction(log_action)
         gamma_action = QAction("감마 변환", self)
         gamma_action.triggered.connect(self.apply_gamma_transform)
         transform_menu.addAction(gamma_action)
         hist_action = QAction("히스토그램 보기", self)
         hist_action.triggered.connect(self.show_histogram)
         transform_menu.addAction(hist_action)

     def show_histogram(self):
         if self.current_image is None:
             self.transformed_label.setText("이미지 없음")
             return
         plt.figure(figsize=(8, 6))
         colors = ('r', 'g', 'b')
         for i, color in enumerate(colors):
             hist = np.histogram(self.current_image[:, :, i], bins=256, range=(0, 256))[0]
             plt.plot(hist, color=color, label=f'{color.upper()} 채널')
         plt.title('RGB 히스토그램')
         plt.xlabel('픽셀 강도')
         plt.ylabel('빈도')
         plt.legend()
         plt.grid(True)
         plt.show()
     ```

3. **감마 변환 사용자 입력**:
   - 현재 감마 변환은 고정값(`gamma=0.5`)을 사용합니다. 사용자 입력으로 감마 값을 지정하려면 다이얼로그를 추가할 수 있습니다:
     ```python
     from PyQt5.QtWidgets import QInputDialog

     def apply_gamma_transform(self):
         if self.current_image is None:
             self.transformed_label.setText("이미지 없음")
             return
         gamma, ok = QInputDialog.getDouble(
             self, "감마 변환", "감마 값을 입력하세요:", 0.5, 0.1, 10.0, 2
         )
         if ok:
             def gamma_transformation(image, gamma):
                 image = image.astype(np.float32)
                 image = np.clip(image, 0, 255)
                 normalized = image / 255.0
                 transformed = np.power(normalized, gamma) * 255.0
                 return transformed.round().clip(0, 255).astype(np.uint8)
             self.transformed_image = gamma_transformation(self.current_image, gamma)
             self.update_transformed_image()
     ```

4. **환경별 디버깅**:
   - Jupyter 로그(`~/.jupyter/jupyter_notebook.log`) 또는 IDE 디버거로 에러 추적.
   - PyQt5 설치 확인:
     ```bash
     pip install --force-reinstall PyQt5
     ```

5. **이미지 파일**:
   - 'forest.jpg'가 없으면 초기 이미지가 로드되지 않습니다. 다른 이미지를 사용하려면 파일명을 변경하세요.
   - 이미지를 업로드할 수 있다면 테스트 가능.

### 추가 요청
- `image_tools`의 구체적인 정의를 알려주시면 코드 조정 가능.
- 히스토그램 표시, 추가 변환(예: 밝기 조절, 대비 조정), 사용자 입력 다이얼로그 등 원하는 기능 요청.
- 특정 환경에서의 에러 메시지 공유 시 정확한 디버깅 가능.
- 첫 번째 코드에 있던 "종류 부메뉴"가 특정 기능(예: 변환 유형 선택)을 의미한다면 구체적으로 알려주세요.

위 코드로 요청하신 기능이 모두 구현되었기를 바랍니다! 추가 수정이나 질문 있으시면 말씀해주세요.

## 추가 질문 1

현 디렉토리에 image_tools.py 라는 모듈이 있어 그걸로 불러와줘
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

def load_image(filename: str, color_channel : str ="RGB") -> np.ndarray:
    """파일에서 이미지를 로드합니다.

    Args:
        filename (str): 이미지 파일 경로
    Returns:
        np.ndarray: 로드된 이미지 데이터
    Raises:
        FileNotFoundError: 파일이 존재하지 않을 경우
        ValueError: 이미지를 로드할 수 없을 경우
    """
    if not os.path.exists(filename):
        # raise는 함수안에서 사용을 한다고 하여도 main body에서 작용되는 글로벌로 작용된다?
        raise FileNotFoundError(f"File {filename} does not exist")
    
    # 삼항 연산자
    # imread = plt.imread if color_channel == "RGB" else cv2.imread
    
    if color_channel == "RGB":
        # 함수 자체를 넣을 수 있음
        imread = plt.imread
    elif color_channel == "BGR":
        imread = cv2.imread
    else:
        raise ValueError(f"undifined color_channel {color_channel}")
    img: np.ndarray = cv2.imread(filename)
    if img is None:
        return ValueError(f"Could Not load {filename}")
    return img

이고
흑백변환, 감마 변환은 bar를 사용해서 조절할 수 있게 해줘, 히스토그램 변환도 해주고 또 추가 해야 할 변환이나 추가해야 할 메뉴가 있으면 추가해줘 신박한것일 수록 좋으니까 만들어줘

## 추가 답변 1

요청하신 대로 `image_tools.py` 모듈을 사용하고, 흑백 변환과 감마 변환을 슬라이더 바(`QSlider`)로 조절할 수 있도록 구현하겠습니다. 또한, 히스토그램 표시 기능을 추가하고, 신박한 이미지 변환(예: 세피아 톤, 엣지 감지, 색상 반전 등)을 메뉴에 추가하겠습니다. 첫 번째 코드의 구조(수평 레이아웃, 원본/변환 이미지 표시)를 기반으로 하되, 두 번째 코드의 기능(로그 변환, 감마 변환 등)을 통합하며, 새로운 기능을 추가하겠습니다. Jupyter Notebook, VS Code, PyCharm 등 모든 환경에서 동작하도록 최적화하겠습니다.

### 요구사항 정리
1. **모듈 사용**:
   - 현 디렉토리의 `image_tools.py`를 사용. 제공된 `load_image` 함수는 OpenCV(`cv2.imread`)로 이미지를 로드하며, `color_channel` 옵션으로 RGB/BGR 선택 가능.
   - 기본적으로 BGR 형식으로 로드되므로, RGB로 표시하려면 변환 필요.
2. **변환 기능**:
   - **흑백 변환**: 슬라이더로 흑백 강도(그레이스케일 혼합 비율) 조절.
   - **감마 변환**: 슬라이더로 감마 값 조절(예: 0.1~5.0).
   - **히스토그램 표시**: Matplotlib로 원본 이미지의 RGB 히스토그램 표시.
   - **신박한 변환**:
     - 세피아 톤: 빈티지 효과.
     - 엣지 감지: Canny 엣지 검출.
     - 색상 반전: 네거티브 효과.
     - 밝기/대비 조절: 슬라이더로 조절.
3. **UI 개선**:
   - 슬라이더 바를 하단에 추가하여 흑백 변환과 감마 변환 조절.
   - 메뉴바에 "파일"(열기, 종료), "변환"(로그, 감마, 흑백, 세피아, 엣지, 색상 반전, 밝기/대비), "보기"(히스토그램) 메뉴 추가.
4. **환경 호환성**:
   - Jupyter: `%gui qt` 사용.
   - VS Code/PyCharm: `sys.argv`로 실행.
5. **초기 이미지**:
   - 'forest.jpg'를 초기 이미지로 로드(파일 없으면 에러 메시지).

### image_tools.py 분석
- `load_image(filename, color_channel="RGB")`:
  - `color_channel="RGB"` 시 `plt.imread`로 로드, `"BGR"` 시 `cv2.imread`로 로드.
  - 하지만 함수 내부에서 항상 `cv2.imread`를 호출하므로 `color_channel` 옵션이 무시됨(버그로 보임).
  - 반환값은 BGR 형식의 NumPy 배열(`cv2.imread` 출력).
- **수정 필요**:
  - RGB로 로드하려면 `cv2.cvtColor(img, cv2.COLOR_BGR2RGB)` 적용.
  - 버그 수정: `color_channel`에 따라 올바른 로드 함수 사용.

### 수정된 코드

```python
%gui qt  # Jupyter Notebook에서만 실행 (일반 스크립트에서는 주석 처리)

import sys
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QLabel, QAction, QFileDialog, QHBoxLayout, QWidget,
    QVBoxLayout, QSlider, QGroupBox
)
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QPixmap, QImage
import image_tools  # 현 디렉토리의 image_tools.py

def log_transformation(image, f_max=255):
    """로그 변환"""
    f_max = float(f_max)
    if f_max <= 0 or np.isnan(f_max) or np.isinf(f_max):
        f_max = 255
    image = image.astype(np.float32)
    image = np.clip(image, 0, 255)
    C = 255 / np.log(1 + f_max + 1e-10)
    return (C * np.log(1. + image)).round().clip(0, 255).astype(np.uint8)

def gamma_transformation(image, gamma=1.0):
    """감마 변환"""
    image = image.astype(np.float32)
    image = np.clip(image, 0, 255)
    normalized = image / 255.0
    transformed = np.power(normalized, gamma) * 255.0
    return transformed.round().clip(0, 255).astype(np.uint8)

def grayscale_transformation(image, weight=1.0):
    """흑백 변환 (0: 원본, 1: 완전 흑백)"""
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray_rgb = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)
    return (image * (1 - weight) + gray_rgb * weight).round().clip(0, 255).astype(np.uint8)

def sepia_transformation(image):
    """세피아 톤 변환"""
    sepia_matrix = np.array([
        [0.393, 0.769, 0.189],
        [0.349, 0.686, 0.168],
        [0.272, 0.534, 0.131]
    ])
    sepia = np.dot(image.astype(np.float32), sepia_matrix.T)
    return sepia.round().clip(0, 255).astype(np.uint8)

def edge_detection(image):
    """Canny 엣지 감지"""
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    return cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)

def invert_colors(image):
    """색상 반전"""
    return (255 - image).astype(np.uint8)

def adjust_brightness_contrast(image, brightness=0, contrast=1.0):
    """밝기/대비 조절"""
    image = image.astype(np.float32)
    image = image * contrast + brightness
    return image.round().clip(0, 255).astype(np.uint8)

class BasicViewer(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("영상처리")
        self.setGeometry(100, 300, 1000, 700)

        # 메인 레이아웃
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)
        self.main_layout = QVBoxLayout(self.central_widget)
        self.main_layout.setContentsMargins(10, 10, 10, 10)

        # 이미지 표시 레이아웃
        self.image_widget = QWidget()
        self.hbox = QHBoxLayout(self.image_widget)
        self.hbox.setContentsMargins(0, 0, 0, 0)

        # 원본 이미지 라벨
        self.original_label = QLabel("원본 이미지를 열어보세요", self)
        self.original_label.setAlignment(Qt.AlignCenter)
        self.original_label.setMinimumSize(300, 300)
        self.hbox.addWidget(self.original_label)

        # 변환된 이미지 라벨
        self.transformed_label = QLabel("변환된 이미지가 여기에 표시됩니다", self)
        self.transformed_label.setAlignment(Qt.AlignCenter)
        self.transformed_label.setMinimumSize(300, 300)
        self.hbox.addWidget(self.transformed_label)

        self.main_layout.addWidget(self.image_widget)

        # 슬라이더 그룹
        self.slider_group = QGroupBox("변환 조절")
        self.slider_layout = QVBoxLayout(self.slider_group)

        # 흑백 변환 슬라이더
        self.grayscale_slider = QSlider(Qt.Horizontal)
        self.grayscale_slider.setRange(0, 100)
        self.grayscale_slider.setValue(0)
        self.grayscale_slider.valueChanged.connect(self.apply_grayscale_transform)
        self.slider_layout.addWidget(QLabel("흑백 강도 (0: 원본, 100: 완전 흑백)"))
        self.slider_layout.addWidget(self.grayscale_slider)

        # 감마 변환 슬라이더
        self.gamma_slider = QSlider(Qt.Horizontal)
        self.gamma_slider.setRange(10, 500)
        self.gamma_slider.setValue(100)
        self.gamma_slider.valueChanged.connect(self.apply_gamma_transform)
        self.slider_layout.addWidget(QLabel("감마 값 (10: 0.1, 500: 5.0)"))
        self.slider_layout.addWidget(self.gamma_slider)

        # 밝기 슬라이더
        self.brightness_slider = QSlider(Qt.Horizontal)
        self.brightness_slider.setRange(-100, 100)
        self.brightness_slider.setValue(0)
        self.brightness_slider.valueChanged.connect(self.apply_brightness_contrast)
        self.slider_layout.addWidget(QLabel("밝기 (-100 ~ 100)"))
        self.slider_layout.addWidget(self.brightness_slider)

        # 대비 슬라이더
        self.contrast_slider = QSlider(Qt.Horizontal)
        self.contrast_slider.setRange(10, 300)
        self.contrast_slider.setValue(100)
        self.contrast_slider.valueChanged.connect(self.apply_brightness_contrast)
        self.slider_layout.addWidget(QLabel("대비 (10: 0.1, 300: 3.0)"))
        self.slider_layout.addWidget(self.contrast_slider)

        self.main_layout.addWidget(self.slider_group)

        self.current_image = None  # 원본 이미지
        self.transformed_image = None  # 변환된 이미지

        # 초기 이미지 로드
        self.load_image('forest.jpg')

        self.create_menu()

    def create_menu(self):
        menubar = self.menuBar()

        # 파일 메뉴
        file_menu = menubar.addMenu("파일")
        open_action = QAction("열기", self)
        open_action.triggered.connect(self.open_image_dialog)
        file_menu.addAction(open_action)
        exit_action = QAction("종료", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        # 변환 메뉴
        transform_menu = menubar.addMenu("변환")
        log_action = QAction("로그 변환", self)
        log_action.triggered.connect(self.apply_log_transform)
        transform_menu.addAction(log_action)
        sepia_action = QAction("세피아 톤", self)
        sepia_action.triggered.connect(self.apply_sepia_transform)
        transform_menu.addAction(sepia_action)
        edge_action = QAction("엣지 감지", self)
        edge_action.triggered.connect(self.apply_edge_transform)
        transform_menu.addAction(edge_action)
        invert_action = QAction("색상 반전", self)
        invert_action.triggered.connect(self.apply_invert_transform)
        transform_menu.addAction(invert_action)

        # 보기 메뉴
        view_menu = menubar.addMenu("보기")
        hist_action = QAction("히스토그램 보기", self)
        hist_action.triggered.connect(self.show_histogram)
        view_menu.addAction(hist_action)

    def load_image(self, filename):
        """이미지 로드"""
        try:
            # image_tools.load_image 사용 (RGB로 로드)
            img = image_tools.load_image(filename, color_channel="BGR")
            if not isinstance(img, np.ndarray):
                raise ValueError(f"Could not load {filename}")
            # BGR to RGB
            self.current_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            self.current_image = np.ascontiguousarray(self.current_image, dtype=np.uint8)
            self.transformed_image = None
            self.update_image()
            self.update_transformed_image()
            # 슬라이더 초기화
            self.grayscale_slider.setValue(0)
            self.gamma_slider.setValue(100)
            self.brightness_slider.setValue(0)
            self.contrast_slider.setValue(100)
        except Exception as e:
            self.original_label.setText(f"이미지 로드 실패: {str(e)}")
            self.transformed_label.setText("변환된 이미지가 여기에 표시됩니다")

    def open_image_dialog(self):
        """파일 다이얼로그로 이미지 열기"""
        filename, _ = QFileDialog.getOpenFileName(
            self,
            "이미지 열기",
            "",
            "Image Files (*.png *.jpg *.jpeg *.bmp *.tif *.tiff)"
        )
        if filename:
            self.load_image(filename)

    def numpy_to_qpixmap(self, image):
        """NumPy 배열을 QPixmap으로 변환"""
        if image is None:
            return None
        h, w, ch = image.shape
        if ch != 3:
            return None
        bytes_per_line = ch * w
        image = np.ascontiguousarray(image, dtype=np.uint8)
        qimage = QImage(image.data, w, h, bytes_per_line, QImage.Format_RGB888)
        if qimage.isNull():
            return None
        return QPixmap.fromImage(qimage)

    def update_image(self):
        """원본 이미지 업데이트"""
        pixmap = self.numpy_to_qpixmap(self.current_image)
        if pixmap is not None:
            self.original_label.setPixmap(pixmap.scaled(
                self.original_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation
            ))
        else:
            self.original_label.setText("원본 이미지를 열어보세요")

    def update_transformed_image(self):
        """변환된 이미지 업데이트"""
        pixmap = self.numpy_to_qpixmap(self.transformed_image)
        if pixmap is not None:
            self.transformed_label.setPixmap(pixmap.scaled(
                self.transformed_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation
            ))
        else:
            self.transformed_label.setText("변환된 이미지가 여기에 표시됩니다")

    def apply_log_transform(self):
        """로그 변환 적용"""
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = log_transformation(self.current_image, f_max=self.current_image.max())
        self.update_transformed_image()

    def apply_grayscale_transform(self):
        """흑백 변환 적용 (슬라이더)"""
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        weight = self.grayscale_slider.value() / 100.0
        self.transformed_image = grayscale_transformation(self.current_image, weight)
        self.update_transformed_image()

    def apply_gamma_transform(self):
        """감마 변환 적용 (슬라이더)"""
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        gamma = self.gamma_slider.value() / 100.0
        self.transformed_image = gamma_transformation(self.current_image, gamma)
        self.update_transformed_image()

    def apply_sepia_transform(self):
        """세피아 톤 적용"""
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = sepia_transformation(self.current_image)
        self.update_transformed_image()

    def apply_edge_transform(self):
        """엣지 감지 적용"""
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = edge_detection(self.current_image)
        self.update_transformed_image()

    def apply_invert_transform(self):
        """색상 반전 적용"""
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = invert_colors(self.current_image)
        self.update_transformed_image()

    def apply_brightness_contrast(self):
        """밝기/대비 조절 적용"""
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        brightness = self.brightness_slider.value()
        contrast = self.contrast_slider.value() / 100.0
        self.transformed_image = adjust_brightness_contrast(self.current_image, brightness, contrast)
        self.update_transformed_image()

    def show_histogram(self):
        """RGB 히스토그램 표시"""
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        plt.figure(figsize=(8, 6))
        colors = ('r', 'g', 'b')
        for i, color in enumerate(colors):
            hist = np.histogram(self.current_image[:, :, i], bins=256, range=(0, 256))[0]
            plt.plot(hist, color=color, label=f'{color.upper()} 채널')
        plt.title('RGB 히스토그램')
        plt.xlabel('픽셀 강도')
        plt.ylabel('빈도')
        plt.legend()
        plt.grid(True)
        plt.show()

    def showEvent(self, event):
        super().showEvent(event)
        self.update_image()
        self.update_transformed_image()

    def resizeEvent(self, event):
        super().resizeEvent(event)
        self.update_image()
        self.update_transformed_image()

# QApplication 인스턴스 관리
app = QApplication.instance()
if app is None:
    app = QApplication(sys.argv)
else:
    print("기존 QApplication 인스턴스를 사용합니다.")

viewer = BasicViewer()
viewer.show()

# Jupyter와 일반 스크립트 구분
if 'IPython' in sys.modules:
    app.exec_()
else:
    sys.exit(app.exec_())
```

### 주요 특징 및 수정 사항

1. **`image_tools.py` 사용**:
   - 제공된 `image_tools.load_image`를 사용하되, `color_channel="BGR"`로 설정하여 `cv2.imread`로 로드.
   - 반환된 BGR 이미지를 `cv2.cvtColor(..., cv2.COLOR_BGR2RGB)`로 RGB로 변환.
   - 에러 처리: 파일 없음, 로드 실패 시 `QLabel`에 메시지 표시.

2. **슬라이더 바**:
   - **흑백 변환**: `QSlider`(0~100)로 흑백 강도 조절(0: 원본, 1: 완전 흑백).
   - **감마 변환**: `QSlider`(10~500)로 감마 값 조절(0.1~5.0).
   - **밝기/대비**: `QSlider`로 밝기(-100~100), 대비(0.1~3.0) 조절.
   - 슬라이더는 하단 `QGroupBox`에 배치.

3. **히스토그램 표시**:
   - "보기" > "히스토그램 보기" 메뉴로 Matplotlib 플롯 표시.
   - RGB 각 채널의 히스토그램을 색상별로 플롯(빨강, 초록, 파랑).

4. **신박한 변환**:
   - **세피아 톤**: 빈티지 효과로 색조 변경.
   - **엣지 감지**: Canny 알고리즘으로 이미지 윤곽 강조.
   - **색상 반전**: 네거티브 효과로 색상 반전.
   - **밝기/대비 조절**: 슬라이더로 실시간 조정.

5. **메뉴바**:
   - **파일**: "열기"(파일 다이얼로그), "종료"(창 닫기).
   - **변환**: 로그 변환, 흑백 변환, 감마 변환, 세피아 톤, 엣지 감지, 색상 반전.
   - **보기**: 히스토그램 보기.

6. **UI 개선**:
   - 창 크기 1000x700으로 조정하여 슬라이더 공간 확보.
   - `QVBoxLayout`으로 이미지와 슬라이더 수직 배치.
   - `QHBoxLayout`으로 원본/변환 이미지 나란히 표시.
   - 최소 크기(300x300) 설정으로 이미지 가시성 보장.

7. **환경 호환성**:
   - Jupyter: `%gui qt`, `IPython` 확인으로 `sys.exit` 생략.
   - VS Code/PyCharm: `sys.argv` 사용.

8. **에러 처리**:
   - 이미지 로드 실패, 잘못된 채널 수, QImage 생성 실패 시 메시지 표시.
   - 변환 적용 시 원본 이미지 없으면 메시지 출력.

### 실행 방법

#### Jupyter Notebook
1. `image_tools.py`를 Jupyter Notebook의 작업 디렉토리에 저장.
2. 첫 번째 셀:
   ```python
   %gui qt
   ```
3. 두 번째 셀에 위 코드를 붙여넣고 실행.
4. 'forest.jpg'가 작업 디렉토리에 있는지 확인.
5. 실행 후:
   - 창이 열리고 'forest.jpg'가 원본으로 표시.
   - "파일" > "열기"로 새 이미지 로드.
   - "변환" 메뉴로 다양한 효과 적용.
   - 슬라이더로 흑백, 감마, 밝기, 대비 조절.
   - "보기" > "히스토그램 보기"로 히스토그램 표시.

#### VS Code 또는 PyCharm
1. `image_tools.py`와 위 코드를 동일 디렉토리에 저장(`image_viewer.py`).
2. 'forest.jpg' 확인.
3. 실행:
   ```bash
   python image_viewer.py
   ```
4. `%gui qt` 주석 처리.
5. 동일한 동작 확인.

### 예상 출력
- 1000x700 창에 원본/변환 이미지 나란히 표시.
- 하단에 슬라이더 4개(흑백, 감마, 밝기, 대비).
- 메뉴바:
  - "파일": 이미지 열기, 종료.
  - "변환": 로그, 흑백, 감마, 세피아, 엣지, 색상 반전.
  - "보기": 히스토그램.
- 슬라이더 조절 시 변환 이미지 실시간 업데이트.
- 히스토그램은 별도 Matplotlib 창에 표시.

### 추가 고려사항

1. **`image_tools.py` 버그**:
   - 현재 `load_image`는 `color_channel` 무시하고 항상 `cv2.imread` 호출. 수정 제안:
     ```python
     def load_image(filename: str, color_channel: str = "RGB") -> np.ndarray:
         if not os.path.exists(filename):
             raise FileNotFoundError(f"File {filename} does not exist")
         img = cv2.imread(filename)
         if img is None:
             raise ValueError(f"Could not load {filename}")
         if color_channel == "RGB":
             img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
         return img
     ```
   - 위 코드를 사용하려면 `image_tools.py`를 수정하거나, 현재 코드에서 RGB 변환 처리로 충분.

2. **추가 변환 제안**:
   - **만화 효과**: 엣지 감지 + 색상 양자화.
   - **블러**: 가우시안 블러 적용.
   - **색상 강조**: HSV 공간에서 채도 조절.
   - 요청 시 구현 가능.

3. **UI 개선**:
   - 슬라이더 값 표시 레이블 추가 가능.
   - 변환 미리보기 창(작은 썸네일) 추가 가능.

4. **히스토그램 개선**:
   - 변환된 이미지의 히스토그램도 표시하도록 확장 가능.

5. **성능**:
   - 큰 이미지의 경우 슬라이더 반응성 최적화를 위해 변환 캐싱 고려.

### 추가 요청
- 특정 변환(예: 만화 효과) 추가 원하시면 말씀.
- 슬라이더 UI 커스터마이징(예: 값 표시) 필요 시 요청.
- 에러 발생 시 로그 공유로 디버깅 지원.
- 'forest.jpg' 업로드 가능 시 테스트.

이 코드로 요청하신 기능이 모두 구현되었기를 바랍니다! 추가 수정이나 질문 있으시면 알려주세요.

## 추가 질문 2

변환 조절을 할 때 변환 메뉴에 변환 조절이라는 버튼을 누르면 변환 조절 탭이 따로 나오도록 만들어 줄 수 있어? 그게 좋은데? 그리고 추가로 뭔가 더 신박하고 재미있는 기능같은 것은 없어? 너무 정형적이야

## 추가 답변 2

요청하신 대로 "변환 조절" 기능을 메뉴바의 "변환" 메뉴에 추가하여, "변환 조절" 버튼을 클릭하면 별도의 탭(`QDockWidget` 또는 `QDialog`)으로 슬라이더(흑백, 감마, 밝기, 대비 조절)가 나타나도록 구현하겠습니다. 또한, 기존 기능이 너무 정형적이라는 피드백을 반영하여 **신박하고 재미있는 기능**을 추가하겠습니다. `image_tools.py` 모듈은 제공된 버전을 사용하며, Jupyter Notebook, VS Code, PyCharm 환경을 모두 지원하도록 최적화하겠습니다.

### 요구사항 정리
1. **변환 조절 탭**:
   - "변환" 메뉴에 "변환 조절" 항목 추가.
   - 클릭 시 별도의 탭(또는 다이얼로그)에 흑백, 감마, 밝기, 대비 슬라이더 표시.
   - 탭은 창의 측면에 도킹 가능(`QDockWidget`)하거나 독립 창(`QDialog`)으로 구현.
2. **신박하고 재미있는 기능**:
   - **만화 효과**: 엣지 감지와 색상 양자화로 만화 스타일 변환.
   - **모자이크 효과**: 선택한 영역에 모자이크 처리(마우스 드래그로 영역 지정).
   - **색상 스왑**: RGB 채널을 무작위로 섞어 독특한 색상 효과.
   - **애니메이션 필터**: 이미지에 동적 왜곡(예: 물결 효과) 적용.
3. **기존 기능 유지**:
   - 원본/변환 이미지 수평 배치(`QHBoxLayout`).
   - 메뉴바: "파일"(열기, 종료), "변환"(로그, 세피아, 엣지, 색상 반전, 변환 조절 등), "보기"(히스토그램).
   - 초기 이미지: 'forest.jpg'.
   - `image_tools.load_image` 사용(BGR 출력, RGB로 변환).
4. **환경 호환성**:
   - Jupyter: `%gui qt`.
   - VS Code/PyCharm: `sys.argv`.

### 설계 결정
- **변환 조절 탭**: `QDockWidget`을 사용하여 창의 오른쪽에 도킹 가능한 패널로 구현. 독립 창보다 UI 통합성이 높고, 사용자가 필요 시 숨길 수 있음.
- **신박한 기능**:
  - **만화 효과**: K-평균 클러스터링으로 색상 단순화 + 엣지 강조.
  - **모자이크 효과**: 마우스 드래그로 영역 선택 후 픽셀화.
  - **색상 스왑**: RGB 채널 순서 무작위 변경.
  - **애니메이션 필터**: 물결 효과(사인파 기반 픽셀 이동)로 동적 변환.
- **UI 개선**: 슬라이더 옆에 현재 값 표시 레이블 추가.

### 수정된 코드

```python
%gui qt  # Jupyter Notebook에서만 실행 (일반 스크립트에서는 주석 처리)

import sys
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QLabel, QAction, QFileDialog, QHBoxLayout, QWidget,
    QVBoxLayout, QDockWidget, QSlider, QGroupBox, QLabel as QtLabel
)
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QPixmap, QImage
import image_tools  # 현 디렉토리의 image_tools.py
import random

def log_transformation(image, f_max=255):
    f_max = float(f_max)
    if f_max <= 0 or np.isnan(f_max) or np.isinf(f_max):
        f_max = 255
    image = image.astype(np.float32)
    image = np.clip(image, 0, 255)
    C = 255 / np.log(1 + f_max + 1e-10)
    return (C * np.log(1. + image)).round().clip(0, 255).astype(np.uint8)

def gamma_transformation(image, gamma=1.0):
    image = image.astype(np.float32)
    image = np.clip(image, 0, 255)
    normalized = image / 255.0
    transformed = np.power(normalized, gamma) * 255.0
    return transformed.round().clip(0, 255).astype(np.uint8)

def grayscale_transformation(image, weight=1.0):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray_rgb = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)
    return (image * (1 - weight) + gray_rgb * weight).round().clip(0, 255).astype(np.uint8)

def sepia_transformation(image):
    sepia_matrix = np.array([
        [0.393, 0.769, 0.189],
        [0.349, 0.686, 0.168],
        [0.272, 0.534, 0.131]
    ])
    sepia = np.dot(image.astype(np.float32), sepia_matrix.T)
    return sepia.round().clip(0, 255).astype(np.uint8)

def edge_detection(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    return cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)

def invert_colors(image):
    return (255 - image).astype(np.uint8)

def adjust_brightness_contrast(image, brightness=0, contrast=1.0):
    image = image.astype(np.float32)
    image = image * contrast + brightness
    return image.round().clip(0, 255).astype(np.uint8)

def cartoon_effect(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray = cv2.medianBlur(gray, 5)
    edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 9)
    color = cv2.bilateralFilter(image, 9, 300, 300)
    color = cv2.bitwise_and(color, color, mask=edges)
    return color

def mosaic_effect(image, rect, block_size=10):
    (x1, y1, x2, y2) = rect
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(image.shape[1], x2), min(image.shape[0], y2)
    if x2 <= x1 or y2 <= y1:
        return image
    roi = image[y1:y2, x1:x2].copy()
    h, w = roi.shape[:2]
    for i in range(0, h, block_size):
        for j in range(0, w, block_size):
            block = roi[i:i+block_size, j:j+block_size]
            if block.size > 0:
                mean_color = block.mean(axis=(0, 1)).astype(np.uint8)
                roi[i:i+block_size, j:j+block_size] = mean_color
    image[y1:y2, x1:x2] = roi
    return image

def color_swap(image):
    channels = list(cv2.split(image))
    random.shuffle(channels)
    return cv2.merge(channels)

def wave_effect(image, amplitude=10, wavelength=20):
    h, w = image.shape[:2]
    output = np.zeros_like(image)
    for i in range(h):
        for j in range(w):
            new_i = int(i + amplitude * np.sin(2 * np.pi * j / wavelength))
            if 0 <= new_i < h:
                output[new_i, j] = image[i, j]
    return output

class ImageLabel(QLabel):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.start_pos = None
        self.end_pos = None
        self.drawing = False

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.start_pos = event.pos()
            self.drawing = True

    def mouseMoveEvent(self, event):
        if self.drawing:
            self.end_pos = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.end_pos = event.pos()
            self.drawing = False
            self.parent().apply_mosaic(self.start_pos, self.end_pos)
            self.start_pos = None
            self.end_pos = None

class BasicViewer(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("영상처리 - 신박한 이미지 편집기")
        self.setGeometry(100, 300, 1200, 800)

        # 메인 레이아웃
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)
        self.main_layout = QVBoxLayout(self.central_widget)
        self.main_layout.setContentsMargins(10, 10, 10, 10)

        # 이미지 표시 레이아웃
        self.image_widget = QWidget()
        self.hbox = QHBoxLayout(self.image_widget)
        self.hbox.setContentsMargins(0, 0, 0, 0)

        # 원본 이미지 라벨
        self.original_label = ImageLabel(self)
        self.original_label.setAlignment(Qt.AlignCenter)
        self.original_label.setMinimumSize(400, 400)
        self.original_label.setText("원본 이미지를 열어보세요")
        self.hbox.addWidget(self.original_label)

        # 변환된 이미지 라벨
        self.transformed_label = QLabel(self)
        self.transformed_label.setAlignment(Qt.AlignCenter)
        self.transformed_label.setMinimumSize(400, 400)
        self.transformed_label.setText("변환된 이미지가 여기에 표시됩니다")
        self.hbox.addWidget(self.transformed_label)

        self.main_layout.addWidget(self.image_widget)

        self.current_image = None
        self.transformed_image = None

        # 변환 조절 도킹 위젯
        self.transform_dock = QDockWidget("변환 조절", self)
        self.transform_dock.setAllowedAreas(Qt.RightDockWidgetArea)
        self.transform_dock.setFeatures(QDockWidget.DockWidgetMovable | QDockWidget.DockWidgetClosable)
        self.transform_widget = QWidget()
        self.transform_layout = QVBoxLayout(self.transform_widget)
        self.transform_layout.setContentsMargins(10, 10, 10, 10)

        # 슬라이더 그룹
        self.slider_group = QGroupBox("조절 패널")
        self.slider_layout = QVBoxLayout(self.slider_group)

        # 흑백 변환 슬라이더
        self.grayscale_label = QtLabel("흑백 강도: 0%")
        self.grayscale_slider = QSlider(Qt.Horizontal)
        self.grayscale_slider.setRange(0, 100)
        self.grayscale_slider.setValue(0)
        self.grayscale_slider.valueChanged.connect(self.apply_grayscale_transform)
        self.slider_layout.addWidget(self.grayscale_label)
        self.slider_layout.addWidget(self.grayscale_slider)

        # 감마 변환 슬라이더
        self.gamma_label = QtLabel("감마 값: 1.0")
        self.gamma_slider = QSlider(Qt.Horizontal)
        self.gamma_slider.setRange(10, 500)
        self.gamma_slider.setValue(100)
        self.gamma_slider.valueChanged.connect(self.apply_gamma_transform)
        self.slider_layout.addWidget(self.gamma_label)
        self.slider_layout.addWidget(self.gamma_slider)

        # 밝기 슬라이더
        self.brightness_label = QtLabel("밝기: 0")
        self.brightness_slider = QSlider(Qt.Horizontal)
        self.brightness_slider.setRange(-100, 100)
        self.brightness_slider.setValue(0)
        self.brightness_slider.valueChanged.connect(self.apply_brightness_contrast)
        self.slider_layout.addWidget(self.brightness_label)
        self.slider_layout.addWidget(self.brightness_slider)

        # 대비 슬라이더
        self.contrast_label = QtLabel("대비: 1.0")
        self.contrast_slider = QSlider(Qt.Horizontal)
        self.contrast_slider.setRange(10, 300)
        self.contrast_slider.setValue(100)
        self.contrast_slider.valueChanged.connect(self.apply_brightness_contrast)
        self.slider_layout.addWidget(self.contrast_label)
        self.slider_layout.addWidget(self.contrast_slider)

        self.transform_layout.addWidget(self.slider_group)
        self.transform_dock.setWidget(self.transform_widget)
        self.addDockWidget(Qt.RightDockWidgetArea, self.transform_dock)
        self.transform_dock.setVisible(False)  # 초기에는 숨김

        # 초기 이미지 로드
        self.load_image('forest.jpg')

        self.create_menu()

    def create_menu(self):
        menubar = self.menuBar()

        # 파일 메뉴
        file_menu = menubar.addMenu("파일")
        open_action = QAction("열기", self)
        open_action.triggered.connect(self.open_image_dialog)
        file_menu.addAction(open_action)
        exit_action = QAction("종료", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        # 변환 메뉴
        transform_menu = menubar.addMenu("변환")
        log_action = QAction("로그 변환", self)
        log_action.triggered.connect(self.apply_log_transform)
        transform_menu.addAction(log_action)
        sepia_action = QAction("세피아 톤", self)
        sepia_action.triggered.connect(self.apply_sepia_transform)
        transform_menu.addAction(sepia_action)
        edge_action = QAction("엣지 감지", self)
        edge_action.triggered.connect(self.apply_edge_transform)
        transform_menu.addAction(edge_action)
        invert_action = QAction("색상 반전", self)
        invert_action.triggered.connect(self.apply_invert_transform)
        transform_menu.addAction(invert_action)
        cartoon_action = QAction("만화 효과", self)
        cartoon_action.triggered.connect(self.apply_cartoon_transform)
        transform_menu.addAction(cartoon_action)
        color_swap_action = QAction("색상 스왑", self)
        color_swap_action.triggered.connect(self.apply_color_swap)
        transform_menu.addAction(color_swap_action)
        wave_action = QAction("물결 효과", self)
        wave_action.triggered.connect(self.apply_wave_transform)
        transform_menu.addAction(wave_action)
        control_action = QAction("변환 조절", self)
        control_action.triggered.connect(self.toggle_transform_dock)
        transform_menu.addAction(control_action)

        # 보기 메뉴
        view_menu = menubar.addMenu("보기")
        hist_action = QAction("히스토그램 보기", self)
        hist_action.triggered.connect(self.show_histogram)
        view_menu.addAction(hist_action)

    def load_image(self, filename):
        try:
            img = image_tools.load_image(filename, color_channel="BGR")
            if not isinstance(img, np.ndarray):
                raise ValueError(f"Could not load {filename}")
            self.current_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            self.current_image = np.ascontiguousarray(self.current_image, dtype=np.uint8)
            self.transformed_image = None
            self.update_image()
            self.update_transformed_image()
            # 슬라이더 초기화
            self.grayscale_slider.setValue(0)
            self.gamma_slider.setValue(100)
            self.brightness_slider.setValue(0)
            self.contrast_slider.setValue(100)
            self.grayscale_label.setText("흑백 강도: 0%")
            self.gamma_label.setText("감마 값: 1.0")
            self.brightness_label.setText("밝기: 0")
            self.contrast_label.setText("대비: 1.0")
        except Exception as e:
            self.original_label.setText(f"이미지 로드 실패: {str(e)}")
            self.transformed_label.setText("변환된 이미지가 여기에 표시됩니다")

    def open_image_dialog(self):
        filename, _ = QFileDialog.getOpenFileName(
            self,
            "이미지 열기",
            "",
            "Image Files (*.png *.jpg *.jpeg *.bmp *.tif *.tiff)"
        )
        if filename:
            self.load_image(filename)

    def numpy_to_qpixmap(self, image):
        if image is None:
            return None
        h, w, ch = image.shape
        if ch != 3:
            return None
        bytes_per_line = ch * w
        image = np.ascontiguousarray(image, dtype=np.uint8)
        qimage = QImage(image.data, w, h, bytes_per_line, QImage.Format_RGB888)
        if qimage.isNull():
            return None
        return QPixmap.fromImage(qimage)

    def update_image(self):
        pixmap = self.numpy_to_qpixmap(self.current_image)
        if pixmap is not None:
            self.original_label.setPixmap(pixmap.scaled(
                self.original_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation
            ))
        else:
            self.original_label.setText("원본 이미지를 열어보세요")

    def update_transformed_image(self):
        pixmap = self.numpy_to_qpixmap(self.transformed_image)
        if pixmap is not None:
            self.transformed_label.setPixmap(pixmap.scaled(
                self.transformed_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation
            ))
        else:
            self.transformed_label.setText("변환된 이미지가 여기에 표시됩니다")

    def apply_log_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = log_transformation(self.current_image, f_max=self.current_image.max())
        self.update_transformed_image()

    def apply_grayscale_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        weight = self.grayscale_slider.value() / 100.0
        self.grayscale_label.setText(f"흑백 강도: {int(weight * 100)}%")
        self.transformed_image = grayscale_transformation(self.current_image, weight)
        self.update_transformed_image()

    def apply_gamma_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        gamma = self.gamma_slider.value() / 100.0
        self.gamma_label.setText(f"감마 값: {gamma:.2f}")
        self.transformed_image = gamma_transformation(self.current_image, gamma)
        self.update_transformed_image()

    def apply_sepia_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = sepia_transformation(self.current_image)
        self.update_transformed_image()

    def apply_edge_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = edge_detection(self.current_image)
        self.update_transformed_image()

    def apply_invert_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = invert_colors(self.current_image)
        self.update_transformed_image()

    def apply_cartoon_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = cartoon_effect(self.current_image)
        self.update_transformed_image()

    def apply_color_swap(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = color_swap(self.current_image)
        self.update_transformed_image()

    def apply_wave_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = wave_effect(self.current_image)
        self.update_transformed_image()

    def apply_brightness_contrast(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        brightness = self.brightness_slider.value()
        contrast = self.contrast_slider.value() / 100.0
        self.brightness_label.setText(f"밝기: {brightness}")
        self.contrast_label.setText(f"대비: {contrast:.2f}")
        self.transformed_image = adjust_brightness_contrast(self.current_image, brightness, contrast)
        self.update_transformed_image()

    def apply_mosaic(self, start_pos, end_pos):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        x1, y1 = start_pos.x(), start_pos.y()
        x2, y2 = end_pos.x(), end_pos.y()
        rect = (min(x1, x2), min(y1, y2), max(x1, x2), max(y1, y2))
        self.transformed_image = mosaic_effect(self.current_image.copy(), rect, block_size=10)
        self.update_transformed_image()

    def toggle_transform_dock(self):
        self.transform_dock.setVisible(not self.transform_dock.isVisible())

    def show_histogram(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        plt.figure(figsize=(8, 6))
        colors = ('r', 'g', 'b')
        for i, color in enumerate(colors):
            hist = np.histogram(self.current_image[:, :, i], bins=256, range=(0, 256))[0]
            plt.plot(hist, color=color, label=f'{color.upper()} 채널')
        plt.title('RGB 히스토그램')
        plt.xlabel('픽셀 강도')
        plt.ylabel('빈도')
        plt.legend()
        plt.grid(True)
        plt.show()

    def showEvent(self, event):
        super().showEvent(event)
        self.update_image()
        self.update_transformed_image()

    def resizeEvent(self, event):
        super().resizeEvent(event)
        self.update_image()
        self.update_transformed_image()

# QApplication 인스턴스 관리
app = QApplication.instance()
if app is None:
    app = QApplication(sys.argv)
else:
    print("기존 QApplication 인스턴스를 사용합니다.")

viewer = BasicViewer()
viewer.show()

# Jupyter와 일반 스크립트 구분
if 'IPython' in sys.modules:
    app.exec_()
else:
    sys.exit(app.exec_())
```

### 주요 특징 및 수정 사항

1. **변환 조절 탭**:
   - `QDockWidget`으로 오른쪽 도킹 패널 구현.
   - "변환" > "변환 조절" 클릭 시 패널 표시/숨김 토글.
   - 패널에 슬라이더 4개(흑백, 감마, 밝기, 대비)와 현재 값 표시 레이블 포함.
   - `QGroupBox`로 슬라이더 그룹화, 깔끔한 UI 제공.

2. **신박한 기능**:
   - **만화 효과**: `cartoon_effect`로 엣지 강조와 색상 단순화.
   - **모자이크 효과**: 원본 이미지 라벨에서 마우스 드래그로 영역 선택 후 픽셀화(`ImageLabel` 클래스에서 마우스 이벤트 처리).
   - **색상 스왑**: `color_swap`으로 RGB 채널 무작위 재배치.
   - **물결 효과**: `wave_effect`로 사인파 기반 동적 왜곡.

3. **슬라이더 UI**:
   - 흑백: 0~100 (0%: 원본, 100%: 완전 흑백).
   - 감마: 10~500 (0.1~5.0).
   - 밝기: -100~100.
   - 대비: 10~300 (0.1~3.0).
   - 각 슬라이더 옆에 현재 값 표시(`QtLabel`).

4. **히스토그램**:
   - "보기" > "히스토그램 보기"로 RGB 히스토그램 표시.

5. **메뉴바**:
   - **파일**: 열기, 종료.
   - **변환**: 로그, 세피아, 엣지, 색상 반전, 만화, 색상 스왑, 물결, 변환 조절.
   - **보기**: 히스토그램.

6. **`image_tools.py`**:
   - `load_image(filename, color_channel="BGR")`로 BGR 이미지 로드, `cv2.cvtColor`로 RGB 변환.
   - 에러 처리 강화.

7. **UI 개선**:
   - 창 크기 1200x800으로 확장.
   - 최소 라벨 크기 400x400.
   - 모자이크를 위해 원본 라벨을 `ImageLabel`로 커스터마이징.

8. **환경 호환성**:
   - Jupyter: `%gui qt`, `IPython` 확인.
   - VS Code/PyCharm: `sys.argv`.

### 실행 방법

#### Jupyter Notebook
1. `image_tools.py`를 작업 디렉토리에 저장.
2. 첫 번째 셀:
   ```python
   %gui qt
   ```
3. 두 번째 셀에 위 코드를 붙여넣고 실행.
4. 'forest.jpg' 확인.
5. 실행 후:
   - 창이 열리고 'forest.jpg'가 원본으로 표시.
   - "파일" > "열기"로 새 이미지 로드.
   - "변환" > "변환 조절"로 오른쪽 패널 표시, 슬라이더로 조절.
   - 원본 이미지에서 마우스 드래그로 모자이크 적용.
   - 다른 변환 및 히스토그램 확인.

#### VS Code 또는 PyCharm
1. `image_tools.py`, `image_viewer.py`, 'forest.jpg' 동일 디렉토리에 저장.
2. 실행:
   ```bash
   python image_viewer.py
   ```
3. `%gui qt` 주석 처리.
4. 동일한 동작 확인.

### 예상 출력
- 1200x800 창에 원본/변환 이미지 나란히 표시.
- 오른쪽 도킹 패널(초기 숨김)에서 슬라이더로 흑백, 감마, 밝기, 대비 조절.
- 원본 이미지에서 마우스 드래그로 모자이크 적용.
- 메뉴로 다양한 변환(만화, 색상 스왑, 물결 등) 및 히스토그램 표시.

### 추가 고려사항

1. **image_tools.py 개선**:
   - `load_image`의 `color_channel` 버그 수정 제안:
     ```python
     def load_image(filename: str, color_channel: str = "RGB") -> np.ndarray:
         if not os.path.exists(filename):
             raise FileNotFoundError(f"File {filename} does not exist")
         img = cv2.imread(filename)
         if img is None:
             raise ValueError(f"Could not load {filename}")
         if color_channel == "RGB":
             img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
         return img
     ```

2. **추가 신박한 기능**:
   - **AI 스타일 전이**: 사전 학습된 모델로 그림 스타일 적용(리소스 제한으로 제외, 요청 시 가이드 제공).
   - **글리치 효과**: 무작위 노이즈와 색상 왜곡.
   - **3D 아나글리프**: 입체 효과 이미지 생성.

3. **UI 개선**:
   - 변환 미리보기 썸네일.
   - 모자이크 블록 크기 슬라이더 추가.

4. **성능**:
   - 큰 이미지의 경우 변환 캐싱 또는 비동기 처리 고려.

### 추가 요청
- 특정 변환 추가(예: 글리치 효과) 원하시면 요청.
- 모자이크 블록 크기 조절 등 UI 커스터마이징.
- 에러 발생 시 로그 공유.
- 'forest.jpg' 업로드 가능 시 테스트.

이 코드가 신박한 재미를 더하며 요청을 충족하기를 바랍니다! 추가 요청 있으시면 말씀해주세요.

## 추가 질문 3

일단 히스토그램이 셀 아래에 출력이 되는데 그것을 변환할 때 나오는 공간에다가 보여주게 해줘 이건 중요한 사항이야. 그리고 변환탭에만 너무 많이 들어가 있어 보기탭에도 여러개 만들어 주고, 다른 탭을 만들어서 여러 기능을 추가해줘

## 추가 답변 3

요청하신 대로 히스토그램을 Matplotlib 창이 아닌 변환 이미지 공간(`transformed_label`)에 표시하도록 수정하고, "변환" 탭의 메뉴 항목을 분산시켜 "보기" 탭에 더 많은 기능을 추가하며, 새로운 탭(예: "효과", "도구")을 만들어 다양한 신박한 기능을 추가하겠습니다. `image_tools.py` 모듈은 제공된 버전을 사용하며, Jupyter Notebook, VS Code, PyCharm 환경을 모두 지원합니다. 이전 코드의 구조(수평 레이아웃, 도킹 가능한 변환 조절 패널 등)를 유지하되, 요청된 변경사항을 반영하겠습니다.

### 요구사항 정리
1. **히스토그램 표시**:
   - 히스토그램을 별도 Matplotlib 창이 아닌 `transformed_label`에 표시.
   - "보기" > "히스토그램 보기" 클릭 시 원본 이미지의 RGB 히스토그램을 변환 이미지 공간에 렌더링.
   - Matplotlib 플롯을 NumPy 배열로 변환 후 `QPixmap`으로 표시.
2. **메뉴 재구성**:
   - "변환" 탭의 항목(로그, 세피아, 엣지, 색상 반전, 만화, 색상 스왑, 물결, 변환 조절)을 줄이고, 일부를 "보기" 또는 새 탭으로 이동.
   - "보기" 탭에 히스토그램 외에 새로운 기능(예: 이미지 정보, 히스토그램 이퀄라이제이션) 추가.
   - 새 탭("효과", "도구")을 만들어 신박한 기능 추가.
3. **신박한 기능**:
   - **효과 탭**: 글리치 효과, 빈티지 필름 효과, 스케치 효과.
   - **도구 탭**: 크롭, 텍스트 삽입, 브러시(간단한 그리기).
   - 기존 기능(모자이크, 색상 스왑 등) 유지 및 재배치.
4. **기존 기능 유지**:
   - 원본/변환 이미지 수평 배치(`QHBoxLayout`).
   - "변환 조절" 도킹 패널(흑백, 감마, 밝기, 대비 슬라이더).
   - `image_tools.load_image` 사용(BGR 출력, RGB 변환).
   - 초기 이미지: 'forest.jpg'.
5. **환경 호환성**:
   - Jupyter: `%gui qt`.
   - VS Code/PyCharm: `sys.argv`.

### 설계 결정
- **히스토그램 표시**:
  - Matplotlib로 히스토그램을 생성 후, 캔버스를 NumPy 배열로 변환(`FigureCanvasAgg` 사용).
  - 이를 `QPixmap`으로 변환하여 `transformed_label`에 표시.
- **메뉴 재구성**:
  - **변환**: 로그 변환, 감마 변환, 변환 조절.
  - **보기**: 히스토그램, 히스토그램 이퀄라이제이션, 이미지 정보(크기, 채널 등).
  - **효과**: 세피아, 만화, 글리치, 빈티지, 스케치.
  - **도구**: 모자이크, 크롭, 텍스트 삽입, 브러시.
- **신박한 기능**:
  - **글리치 효과**: 무작위 노이즈와 색상 왜곡.
  - **빈티지 필름 효과**: 색상 톤 변화와 노이즈 추가.
  - **스케치 효과**: 엣지 강조와 색상 단순화.
  - **크롭**: 마우스 드래그로 영역 선택 후 자르기.
  - **텍스트 삽입**: 사용자 입력 텍스트를 이미지에 추가.
  - **브러시**: 마우스로 간단한 그리기(원본 이미지에 직접 그림).
- **UI 개선**:
  - 변환 조절 패널은 도킹 상태 유지.
  - 히스토그램 표시 후 다른 변환 적용 시 원활한 전환.

### 수정된 코드

```python
%gui qt  # Jupyter Notebook에서만 실행 (일반 스크립트에서는 주석 처리)

import sys
import numpy as np
import cv2
import matplotlib
matplotlib.use('Agg')  # 비대화형 백엔드
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QLabel, QAction, QFileDialog, QHBoxLayout, QWidget,
    QVBoxLayout, QDockWidget, QSlider, QGroupBox, QInputDialog, QtLabel
)
from PyQt5.QtCore import Qt, QPoint
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen
import image_tools
import random

def log_transformation(image, f_max=255):
    f_max = float(f_max)
    if f_max <= 0 or np.isnan(f_max) or np.isinf(f_max):
        f_max = 255
    image = image.astype(np.float32)
    image = np.clip(image, 0, 255)
    C = 255 / np.log(1 + f_max + 1e-10)
    return (C * np.log(1. + image)).round().clip(0, 255).astype(np.uint8)

def gamma_transformation(image, gamma=1.0):
    image = image.astype(np.float32)
    image = np.clip(image, 0, 255)
    normalized = image / 255.0
    transformed = np.power(normalized, gamma) * 255.0
    return transformed.round().clip(0, 255).astype(np.uint8)

def grayscale_transformation(image, weight=1.0):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray_rgb = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)
    return (image * (1 - weight) + gray_rgb * weight).round().clip(0, 255).astype(np.uint8)

def sepia_transformation(image):
    sepia_matrix = np.array([
        [0.393, 0.769, 0.189],
        [0.349, 0.686, 0.168],
        [0.272, 0.534, 0.131]
    ])
    sepia = np.dot(image.astype(np.float32), sepia_matrix.T)
    return sepia.round().clip(0, 255).astype(np.uint8)

def cartoon_effect(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray = cv2.medianBlur(gray, 5)
    edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 9)
    color = cv2.bilateralFilter(image, 9, 300, 300)
    color = cv2.bitwise_and(color, color, mask=edges)
    return color

def glitch_effect(image):
    h, w = image.shape[:2]
    noise = np.random.normal(0, 25, (h, w, 3)).astype(np.float32)
    glitched = image.astype(np.float32) + noise
    glitched = np.clip(glitched, 0, 255).astype(np.uint8)
    for _ in range(10):
        y = random.randint(0, h-10)
        shift = random.randint(-20, 20)
        glitched[y:y+10] = np.roll(glitched[y:y+10], shift, axis=1)
    return glitched

def vintage_effect(image):
    image = image.astype(np.float32)
    image = image * np.array([0.8, 0.9, 1.0])
    noise = np.random.normal(0, 10, image.shape).astype(np.float32)
    image = np.clip(image + noise, 0, 255).astype(np.uint8)
    return sepia_transformation(image)

def sketch_effect(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    inv = 255 - gray
    blur = cv2.GaussianBlur(inv, (21, 21), 0)
    sketch = cv2.divide(gray, 255 - blur, scale=256)
    return cv2.cvtColor(sketch, cv2.COLOR_GRAY2RGB)

def mosaic_effect(image, rect, block_size=10):
    (x1, y1, x2, y2) = rect
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(image.shape[1], x2), min(image.shape[0], y2)
    if x2 <= x1 or y2 <= y1:
        return image
    roi = image[y1:y2, x1:x2].copy()
    h, w = roi.shape[:2]
    for i in range(0, h, block_size):
        for j in range(0, w, block_size):
            block = roi[i:i+block_size, j:j+block_size]
            if block.size > 0:
                mean_color = block.mean(axis=(0, 1)).astype(np.uint8)
                roi[i:i+block_size, j:j+block_size] = mean_color
    image[y1:y2, x1:x2] = roi
    return image

def adjust_brightness_contrast(image, brightness=0, contrast=1.0):
    image = image.astype(np.float32)
    image = image * contrast + brightness
    return image.round().clip(0, 255).astype(np.uint8)

def histogram_equalization(image):
    img_yuv = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
    return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)

class ImageLabel(QLabel):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.start_pos = None
        self.end_pos = None
        self.drawing = False
        self.brush_mode = False
        self.brush_points = []

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.start_pos = event.pos()
            if self.brush_mode:
                self.brush_points.append([event.pos()])
            self.drawing = True

    def mouseMoveEvent(self, event):
        if self.drawing:
            self.end_pos = event.pos()
            if self.brush_mode:
                self.brush_points[-1].append(event.pos())
                self.parent().apply_brush(self.brush_points)
            self.update()

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.end_pos = event.pos()
            self.drawing = False
            if not self.brush_mode:
                self.parent().apply_mosaic_or_crop(self.start_pos, self.end_pos)
            self.start_pos = None
            self.end_pos = None

class BasicViewer(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("영상처리 - 크리에이티브 편집기")
        self.setGeometry(100, 300, 1200, 800)

        # 메인 레이아웃
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)
        self.main_layout = QVBoxLayout(self.central_widget)
        self.main_layout.setContentsMargins(10, 10, 10, 10)

        # 이미지 표시 레이아웃
        self.image_widget = QWidget()
        self.hbox = QHBoxLayout(self.image_widget)
        self.hbox.setContentsMargins(0, 0, 0, 0)

        # 원본 이미지 라벨
        self.original_label = ImageLabel(self)
        self.original_label.setAlignment(Qt.AlignCenter)
        self.original_label.setMinimumSize(400, 400)
        self.original_label.setText("원본 이미지를 열어보세요")
        self.hbox.addWidget(self.original_label)

        # 변환된 이미지 라벨
        self.transformed_label = QLabel(self)
        self.transformed_label.setAlignment(Qt.AlignCenter)
        self.transformed_label.setMinimumSize(400, 400)
        self.transformed_label.setText("변환된 이미지가 여기에 표시됩니다")
        self.hbox.addWidget(self.transformed_label)

        self.main_layout.addWidget(self.image_widget)

        self.current_image = None
        self.transformed_image = None
        self.brush_mode = False

        # 변환 조절 도킹 위젯
        self.transform_dock = QDockWidget("변환 조절", self)
        self.transform_dock.setAllowedAreas(Qt.RightDockWidgetArea)
        self.transform_dock.setFeatures(QDockWidget.DockWidgetMovable | QDockWidget.DockWidgetClosable)
        self.transform_widget = QWidget()
        self.transform_layout = QVBoxLayout(self.transform_widget)
        self.transform_layout.setContentsMargins(10, 10, 10, 10)

        # 슬라이더 그룹
        self.slider_group = QGroupBox("조절 패널")
        self.slider_layout = QVBoxLayout(self.slider_group)

        # 흑백 변환 슬라이더
        self.grayscale_label = QtLabel("흑백 강도: 0%")
        self.grayscale_slider = QSlider(Qt.Horizontal)
        self.grayscale_slider.setRange(0, 100)
        self.grayscale_slider.setValue(0)
        self.grayscale_slider.valueChanged.connect(self.apply_grayscale_transform)
        self.slider_layout.addWidget(self.grayscale_label)
        self.slider_layout.addWidget(self.grayscale_slider)

        # 감마 변환 슬라이더
        self.gamma_label = QtLabel("감마 값: 1.0")
        self.gamma_slider = QSlider(Qt.Horizontal)
        self.gamma_slider.setRange(10, 500)
        self.gamma_slider.setValue(100)
        self.gamma_slider.valueChanged.connect(self.apply_gamma_transform)
        self.slider_layout.addWidget(self.gamma_label)
        self.slider_layout.addWidget(self.gamma_slider)

        # 밝기 슬라이더
        self.brightness_label = QtLabel("밝기: 0")
        self.brightness_slider = QSlider(Qt.Horizontal)
        self.brightness_slider.setRange(-100, 100)
        self.brightness_slider.setValue(0)
        self.brightness_slider.valueChanged.connect(self.apply_brightness_contrast)
        self.slider_layout.addWidget(self.brightness_label)
        self.slider_layout.addWidget(self.brightness_slider)

        # 대비 슬라이더
        self.contrast_label = QtLabel("대비: 1.0")
        self.contrast_slider = QSlider(Qt.Horizontal)
        self.contrast_slider.setRange(10, 300)
        self.contrast_slider.setValue(100)
        self.contrast_slider.valueChanged.connect(self.apply_brightness_contrast)
        self.slider_layout.addWidget(self.contrast_label)
        self.slider_layout.addWidget(self.contrast_slider)

        self.transform_layout.addWidget(self.slider_group)
        self.transform_dock.setWidget(self.transform_widget)
        self.addDockWidget(Qt.RightDockWidgetArea, self.transform_dock)
        self.transform_dock.setVisible(False)

        # 초기 이미지 로드
        self.load_image('forest.jpg')

        self.create_menu()

    def create_menu(self):
        menubar = self.menuBar()

        # 파일 메뉴
        file_menu = menubar.addMenu("파일")
        open_action = QAction("열기", self)
        open_action.triggered.connect(self.open_image_dialog)
        file_menu.addAction(open_action)
        exit_action = QAction("종료", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        # 변환 메뉴
        transform_menu = menubar.addMenu("변환")
        log_action = QAction("로그 변환", self)
        log_action.triggered.connect(self.apply_log_transform)
        transform_menu.addAction(log_action)
        gamma_action = QAction("감마 변환", self)
        gamma_action.triggered.connect(self.apply_gamma_transform_menu)
        transform_menu.addAction(gamma_action)
        control_action = QAction("변환 조절", self)
        control_action.triggered.connect(self.toggle_transform_dock)
        transform_menu.addAction(control_action)

        # 보기 메뉴
        view_menu = menubar.addMenu("보기")
        hist_action = QAction("히스토그램 보기", self)
        hist_action.triggered.connect(self.show_histogram)
        view_menu.addAction(hist_action)
        hist_eq_action = QAction("히스토그램 이퀄라이제이션", self)
        hist_eq_action.triggered.connect(self.apply_histogram_equalization)
        view_menu.addAction(hist_eq_action)
        info_action = QAction("이미지 정보", self)
        info_action.triggered.connect(self.show_image_info)
        view_menu.addAction(info_action)

        # 효과 메뉴
        effect_menu = menubar.addMenu("효과")
        sepia_action = QAction("세피아 톤", self)
        sepia_action.triggered.connect(self.apply_sepia_transform)
        effect_menu.addAction(sepia_action)
        cartoon_action = QAction("만화 효과", self)
        cartoon_action.triggered.connect(self.apply_cartoon_transform)
        effect_menu.addAction(cartoon_action)
        glitch_action = QAction("글리치 효과", self)
        glitch_action.triggered.connect(self.apply_glitch_transform)
        effect_menu.addAction(glitch_action)
        vintage_action = QAction("빈티지 필름", self)
        vintage_action.triggered.connect(self.apply_vintage_transform)
        effect_menu.addAction(vintage_action)
        sketch_action = QAction("스케치 효과", self)
        sketch_action.triggered.connect(self.apply_sketch_transform)
        effect_menu.addAction(sketch_action)

        # 도구 메뉴
        tool_menu = menubar.addMenu("도구")
        mosaic_action = QAction("모자이크", self)
        mosaic_action.triggered.connect(self.enable_mosaic_mode)
        tool_menu.addAction(mosaic_action)
        crop_action = QAction("크롭", self)
        crop_action.triggered.connect(self.enable_crop_mode)
        tool_menu.addAction(crop_action)
        text_action = QAction("텍스트 삽입", self)
        text_action.triggered.connect(self.insert_text)
        tool_menu.addAction(text_action)
        brush_action = QAction("브러시", self)
        brush_action.triggered.connect(self.toggle_brush_mode)
        tool_menu.addAction(brush_action)

    def load_image(self, filename):
        try:
            img = image_tools.load_image(filename, color_channel="BGR")
            if not isinstance(img, np.ndarray):
                raise ValueError(f"Could not load {filename}")
            self.current_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            self.current_image = np.ascontiguousarray(self.current_image, dtype=np.uint8)
            self.transformed_image = None
            self.update_image()
            self.update_transformed_image()
            self.grayscale_slider.setValue(0)
            self.gamma_slider.setValue(100)
            self.brightness_slider.setValue(0)
            self.contrast_slider.setValue(100)
            self.grayscale_label.setText("흑백 강도: 0%")
            self.gamma_label.setText("감마 값: 1.0")
            self.brightness_label.setText("밝기: 0")
            self.contrast_label.setText("대비: 1.0")
            self.brush_mode = False
            self.original_label.brush_mode = False
            self.original_label.brush_points = []
        except Exception as e:
            self.original_label.setText(f"이미지 로드 실패: {str(e)}")
            self.transformed_label.setText("변환된 이미지가 여기에 표시됩니다")

    def open_image_dialog(self):
        filename, _ = QFileDialog.getOpenFileName(
            self,
            "이미지 열기",
            "",
            "Image Files (*.png *.jpg *.jpeg *.bmp *.tif *.tiff)"
        )
        if filename:
            self.load_image(filename)

    def numpy_to_qpixmap(self, image):
        if image is None:
            return None
        h, w, ch = image.shape
        if ch != 3:
            return None
        bytes_per_line = ch * w
        image = np.ascontiguousarray(image, dtype=np.uint8)
        qimage = QImage(image.data, w, h, bytes_per_line, QImage.Format_RGB888)
        if qimage.isNull():
            return None
        return QPixmap.fromImage(qimage)

    def update_image(self):
        pixmap = self.numpy_to_qpixmap(self.current_image)
        if pixmap is not None:
            self.original_label.setPixmap(pixmap.scaled(
                self.original_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation
            ))
        else:
            self.original_label.setText("원본 이미지를 열어보세요")

    def update_transformed_image(self):
        pixmap = self.numpy_to_qpixmap(self.transformed_image)
        if pixmap is not None:
            self.transformed_label.setPixmap(pixmap.scaled(
                self.transformed_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation
            ))
        else:
            self.transformed_label.setText("변환된 이미지가 여기에 표시됩니다")

    def apply_log_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = log_transformation(self.current_image, f_max=self.current_image.max())
        self.update_transformed_image()

    def apply_grayscale_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        weight = self.grayscale_slider.value() / 100.0
        self.grayscale_label.setText(f"흑백 강도: {int(weight * 100)}%")
        self.transformed_image = grayscale_transformation(self.current_image, weight)
        self.update_transformed_image()

    def apply_gamma_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        gamma = self.gamma_slider.value() / 100.0
        self.gamma_label.setText(f"감마 값: {gamma:.2f}")
        self.transformed_image = gamma_transformation(self.current_image, gamma)
        self.update_transformed_image()

    def apply_gamma_transform_menu(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        gamma, ok = QInputDialog.getDouble(
            self, "감마 변환", "감마 값을 입력하세요:", 1.0, 0.1, 5.0, 2
        )
        if ok:
            self.transformed_image = gamma_transformation(self.current_image, gamma)
            self.update_transformed_image()

    def apply_sepia_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = sepia_transformation(self.current_image)
        self.update_transformed_image()

    def apply_cartoon_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = cartoon_effect(self.current_image)
        self.update_transformed_image()

    def apply_glitch_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = glitch_effect(self.current_image)
        self.update_transformed_image()

    def apply_vintage_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = vintage_effect(self.current_image)
        self.update_transformed_image()

    def apply_sketch_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = sketch_effect(self.current_image)
        self.update_transformed_image()

    def apply_brightness_contrast(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        brightness = self.brightness_slider.value()
        contrast = self.contrast_slider.value() / 100.0
        self.brightness_label.setText(f"밝기: {brightness}")
        self.contrast_label.setText(f"대비: {contrast:.2f}")
        self.transformed_image = adjust_brightness_contrast(self.current_image, brightness, contrast)
        self.update_transformed_image()

    def apply_histogram_equalization(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = histogram_equalization(self.current_image)
        self.update_transformed_image()

    def show_histogram(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        fig, ax = plt.subplots(figsize=(6, 4))
        colors = ('r', 'g', 'b')
        for i, color in enumerate(colors):
            hist = np.histogram(self.current_image[:, :, i], bins=256, range=(0, 256))[0]
            ax.plot(hist, color=color, label=f'{color.upper()} 채널')
        ax.set_title('RGB 히스토그램')
        ax.set_xlabel('픽셀 강도')
        ax.set_ylabel('빈도')
        ax.legend()
        ax.grid(True)

        # Matplotlib 플롯을 이미지로 변환
        canvas = FigureCanvasAgg(fig)
        canvas.draw()
        width, height = fig.get_size_inches() * fig.get_dpi()
        width, height = int(width), int(height)
        image = np.frombuffer(canvas.tostring_rgb(), dtype=np.uint8)
        image = image.reshape(height, width, 3)
        self.transformed_image = image
        self.update_transformed_image()
        plt.close(fig)

    def show_image_info(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        h, w, ch = self.current_image.shape
        info = f"크기: {w}x{h}\n채널: {ch}\n데이터 타입: {self.current_image.dtype}"
        self.transformed_label.setText(info)

    def enable_mosaic_mode(self):
        self.brush_mode = False
        self.original_label.brush_mode = False
        self.original_label.brush_points = []
        self.transformed_label.setText("마우스로 원본 이미지에서 모자이크 영역을 선택하세요")

    def enable_crop_mode(self):
        self.brush_mode = False
        self.original_label.brush_mode = False
        self.original_label.brush_points = []
        self.transformed_label.setText("마우스로 원본 이미지에서 크롭 영역을 선택하세요")

    def toggle_brush_mode(self):
        self.brush_mode = not self.brush_mode
        self.original_label.brush_mode = self.brush_mode
        self.original_label.brush_points = []
        if self.brush_mode:
            self.transformed_label.setText("마우스로 원본 이미지에 그림을 그리세요")
        else:
            self.update_transformed_image()

    def apply_mosaic_or_crop(self, start_pos, end_pos):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        x1, y1 = start_pos.x(), start_pos.y()
        x2, y2 = end_pos.x(), end_pos.y()
        rect = (min(x1, x2), min(y1, y2), max(x1, x2), max(y1, y2))
        if self.transformed_label.text().startswith("마우스로 원본 이미지에서 크롭"):
            x1, y1, x2, y2 = rect
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(self.current_image.shape[1], x2), min(self.current_image.shape[0], y2)
            if x2 > x1 and y2 > y1:
                self.current_image = self.current_image[y1:y2, x1:x2].copy()
                self.transformed_image = None
                self.update_image()
                self.update_transformed_image()
        else:
            self.transformed_image = mosaic_effect(self.current_image.copy(), rect, block_size=10)
            self.update_transformed_image()

    def insert_text(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        text, ok = QInputDialog.getText(self, "텍스트 삽입", "삽입할 텍스트를 입력하세요:")
        if ok and text:
            img = self.current_image.copy()
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(img, text, (50, 50), font, 1, (255, 0, 0), 2, cv2.LINE_AA)
            self.transformed_image = img
            self.update_transformed_image()

    def apply_brush(self, brush_points):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        img = self.current_image.copy()
        pixmap = self.numpy_to_qpixmap(img)
        if pixmap is None:
            return
        painter = QPainter(pixmap)
        painter.setPen(QPen(Qt.red, 5, Qt.SolidLine))
        for points in brush_points:
            for i in range(1, len(points)):
                painter.drawLine(points[i-1], points[i])
        painter.end()
        h, w, ch = img.shape
        bytes_per_line = ch * w
        qimage = pixmap.toImage()
        qimage = qimage.convertToFormat(QImage.Format_RGB888)
        ptr = qimage.bits()
        ptr.setsize(h * bytes_per_line)
        self.transformed_image = np.frombuffer(ptr, dtype=np.uint8).reshape(h, w, ch)
        self.update_transformed_image()

    def toggle_transform_dock(self):
        self.transform_dock.setVisible(not self.transform_dock.isVisible())

    def showEvent(self, event):
        super().showEvent(event)
        self.update_image()
        self.update_transformed_image()

    def resizeEvent(self, event):
        super().resizeEvent(event)
        self.update_image()
        self.update_transformed_image()

# QApplication 인스턴스 관리
app = QApplication.instance()
if app is None:
    app = QApplication(sys.argv)
else:
    print("기존 QApplication 인스턴스를 사용합니다.")

viewer = BasicViewer()
viewer.show()

# Jupyter와 일반 스크립트 구분
if 'IPython' in sys.modules:
    app.exec_()
else:
    sys.exit(app.exec_())
```

### 주요 특징 및 수정 사항

1. **히스토그램 표시**:
   - "보기" > "히스토그램 보기" 클릭 시 `transformed_label`에 RGB 히스토그램 표시.
   - Matplotlib 플롯을 `FigureCanvasAgg`로 렌더링하여 NumPy 배열로 변환, `QPixmap`으로 표시.
   - 플롯 생성 후 `plt.close()`로 메모리 관리.

2. **메뉴 재구성**:
   - **파일**: 열기, 종료.
   - **변환**: 로그 변환, 감마 변환(입력 다이얼로그), 변환 조절(도킹 패널).
   - **보기**: 히스토그램 보기, 히스토그램 이퀄라이제이션(YUV 공간), 이미지 정보(크기, 채널, 데이터 타입).
   - **효과**: 세피아, 만화, 글리치, 빈티지 필름, 스케치.
   - **도구**: 모자이크(마우스 드래그), 크롭(마우스 드래그), 텍스트 삽입, 브러시(그리기).

3. **신박한 기능**:
   - **글리치 효과**: 노이즈와 채널 이동으로 디지털 오류 효과.
   - **빈티지 필름**: 세피아 톤 + 노이즈로 복고 스타일.
   - **스케치 효과**: 엣지 강조로 연필 스케치 스타일.
   - **크롭**: 마우스 드래그로 영역 선택, 원본 이미지 자르기.
   - **텍스트 삽입**: 사용자 입력 텍스트를 이미지에 추가(고정 위치, 빨간색).
   - **브러시**: 마우스로 원본 이미지에 빨간색 선 그리기(`ImageLabel`에서 처리).

4. **변환 조절 탭**:
   - `QDockWidget`으로 오른쪽 도킹 패널 유지.
   - 슬라이더: 흑백(0~100%), 감마(0.1~5.0), 밝기(-100~100), 대비(0.1~3.0).
   - 각 슬라이더 옆에 값 표시 레이블.

5. **`image_tools.py`**:
   - `load_image(filename, color_channel="BGR")`로 BGR 이미지 로드, `cv2.cvtColor`로 RGB 변환.
   - 에러 처리 강화.

6. **UI 개선**:
   - 창 크기 1200x800.
   - `ImageLabel`로 모자이크, 크롭, 브러시 지원.
   - 히스토그램과 변환 결과가 동일 공간(`transformed_label`)에 표시.

7. **환경 호환성**:
   - Jupyter: `%gui qt`, `matplotlib.use('Agg')`로 비대화형 렌더링.
   - VS Code/PyCharm: `sys.argv`.

### 실행 방법

#### Jupyter Notebook
1. `image_tools.py`를 작업 디렉토리에 저장.
2. 첫 번째 셀:
   ```python
   %gui qt
   ```
3. 두 번째 셀에 위 코드를 붙여넣고 실행.
4. 'forest.jpg' 확인.
5. 실행 후:
   - 창이 열리고 'forest.jpg'가 원본으로 표시.
   - "파일" > "열기"로 새 이미지 로드.
   - "변환" > "변환 조절"로 도킹 패널 표시, 슬라이더 조절.
   - "보기" > "히스토그램 보기"로 변환 공간에 히스토그램 표시.
   - "효과"와 "도구" 메뉴로 다양한 기능 사용.
   - 원본 이미지에서 마우스 드래그로 모자이크/크롭, 브러시로 그리기.

#### VS Code 또는 PyCharm
1. `image_tools.py`, `image_viewer.py`, 'forest.jpg' 동일 디렉토리에 저장.
2. 실행:
   ```bash
   python image_viewer.py
   ```
3. `%gui qt` 주석 처리.
4. 동일한 동작 확인.

### 예상 출력
- 1200x800 창에 원본/변환 이미지 나란히 표시.
- 오른쪽 도킹 패널(초기 숨김)으로 슬라이더 조절.
- "보기" > "히스토그램 보기" 시 `transformed_label`에 히스토그램 렌더링.
- 다양한 효과(글리치, 빈티지 등)와 도구(크롭, 텍스트, 브러시) 사용 가능.
- 원본 이미지에서 마우스 드래그로 모자이크/크롭, 브러시 그리기.

### 추가 고려사항

1. **image_tools.py 버그**:
   - `load_image`의 `color_channel` 무시 문제:
     ```python
     def load_image(filename: str, color_channel: str = "RGB") -> np.ndarray:
         if not os.path.exists(filename):
             raise FileNotFoundError(f"File {filename} does not exist")
         img = cv2.imread(filename)
         if img is None:
             raise ValueError(f"Could not load {filename}")
         if color_channel == "RGB":
             img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
         return img
     ```

2. **추가 기능 제안**:
   - **AI 스타일 전이**: 사전 학습된 모델 사용(리소스 필요).
   - **GIF 생성**: 일련의 변환을 애니메이션으로 저장.
   - **필터 스택**: 여러 변환을 순차 적용.

3. **UI 개선**:
   - 히스토그램 크기 조절 옵션.
   - 브러시 색상/두께 선택 다이얼로그.

4. **성능**:
   - 큰 이미지의 경우 변환 캐싱.
   - 히스토그램 렌더링 최적화.

### 추가 요청
- 특정 효과/도구 추가 원하시면 요청.
- 히스토그램 스타일(예: 색상, 크기) 커스터마이징.
- 에러 발생 시 로그 공유.
- 'forest.jpg' 업로드 가능 시 테스트.

이 코드가 요청을 충족하며 창의적인 재미를 더하기를 바랍니다! 추가 요청 있으시면 말씀해주세요.

## 추가 질문 4 (오류 사항)

Cell In[7], line 10
      8 import matplotlib.pyplot as plt
      9 from matplotlib.backends.backend_agg import FigureCanvasAgg
---> 10 from PyQt5.QtWidgets import (
     11     QApplication, QMainWindow, QLabel, QAction, QFileDialog, QHBoxLayout, QWidget,
     12     QVBoxLayout, QDockWidget, QSlider, QGroupBox, QInputDialog, QtLabel
     13 )
     14 from PyQt5.QtCore import Qt, QPoint
     15 from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen

ImportError: cannot import name 'QtLabel' from 'PyQt5.QtWidgets' (c:\Users\tjswo\anaconda3\Lib\site-packages\PyQt5\QtWidgets.pyd)
왜 이런 에러가 나는거야? QtLabel이라는 것 자체가 없다는데 이 오류의 문제점이랑 고쳐서 나오는 코드를 작성해줘

## 추가 답변 4

### 에러 분석: `ImportError: cannot import name 'QtLabel' from 'PyQt5.QtWidgets'`

**에러 원인**:
- 코드에서 `from PyQt5.QtWidgets import ... QtLabel`로 `QtLabel`을 임포트하려 했지만, `PyQt5.QtWidgets` 모듈에는 `QtLabel`이라는 클래스가 존재하지 않습니다.
- 이는 이전 코드에서 `QLabel`을 `QtLabel`로 잘못 참조한 실수입니다. `QtLabel`은 의도적으로 `QLabel`의 별칭(alias)이나 오타로 포함된 것으로 보입니다.
- `QLabel`은 `PyQt5.QtWidgets`에서 제공하는 표준 위젯으로, 텍스트나 이미지를 표시하는 데 사용됩니다. 코드에서 슬라이더 옆에 값을 표시하기 위해 `QtLabel` 대신 `QLabel`을 사용해야 합니다.

**문제점**:
- `QtLabel`은 코드에서 슬라이더의 현재 값을 표시하는 레이블(예: `self.grayscale_label = QtLabel("흑백 강도: 0%")`)로 사용되었습니다.
- `QtLabel`을 `QLabel`로 교체하면 문제가 해결됩니다.
- 추가로, `image_tools.py`의 `load_image` 함수에 버그가 있어 `color_channel` 옵션이 무시됩니다. 이는 히스토그램이나 변환 결과에 영향을 줄 수 있으므로 수정이 필요합니다.

**해결 방법**:
1. 모든 `QtLabel` 참조를 `QLabel`로 변경.
2. `image_tools.load_image` 함수를 수정하여 `color_channel` 옵션을 올바르게 처리.
3. 나머지 코드는 유지하되, 히스토그램 표시와 신박한 기능(모자이크, 브러시 등)이 정상 동작하도록 보장.

### 수정된 코드

아래는 `QtLabel`을 `QLabel`로 교체하고, `image_tools.load_image`를 개선한 코드입니다. 히스토그램은 변환 이미지 공간(`transformed_label`)에 표시되며, 메뉴는 "변환", "보기", "효과", "도구"로 구성됩니다.

```python
%gui qt  # Jupyter Notebook에서만 실행 (일반 스크립트에서는 주석 처리)

import sys
import numpy as np
import cv2
import matplotlib
matplotlib.use('Agg')  # 비대화형 백엔드
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QLabel, QAction, QFileDialog, QHBoxLayout, QWidget,
    QVBoxLayout, QDockWidget, QSlider, QGroupBox, QInputDialog
)
from PyQt5.QtCore import Qt, QPoint
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen
import image_tools
import random

def log_transformation(image, f_max=255):
    f_max = float(f_max)
    if f_max <= 0 or np.isnan(f_max) or np.isinf(f_max):
        f_max = 255
    image = image.astype(np.float32)
    image = np.clip(image, 0, 255)
    C = 255 / np.log(1 + f_max + 1e-10)
    return (C * np.log(1. + image)).round().clip(0, 255).astype(np.uint8)

def gamma_transformation(image, gamma=1.0):
    image = image.astype(np.float32)
    image = np.clip(image, 0, 255)
    normalized = image / 255.0
    transformed = np.power(normalized, gamma) * 255.0
    return transformed.round().clip(0, 255).astype(np.uint8)

def grayscale_transformation(image, weight=1.0):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray_rgb = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)
    return (image * (1 - weight) + gray_rgb * weight).round().clip(0, 255).astype(np.uint8)

def sepia_transformation(image):
    sepia_matrix = np.array([
        [0.393, 0.769, 0.189],
        [0.349, 0.686, 0.168],
        [0.272, 0.534, 0.131]
    ])
    sepia = np.dot(image.astype(np.float32), sepia_matrix.T)
    return sepia.round().clip(0, 255).astype(np.uint8)

def cartoon_effect(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray = cv2.medianBlur(gray, 5)
    edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 9)
    color = cv2.bilateralFilter(image, 9, 300, 300)
    color = cv2.bitwise_and(color, color, mask=edges)
    return color

def glitch_effect(image):
    h, w = image.shape[:2]
    noise = np.random.normal(0, 25, (h, w, 3)).astype(np.float32)
    glitched = image.astype(np.float32) + noise
    glitched = np.clip(glitched, 0, 255).astype(np.uint8)
    for _ in range(10):
        y = random.randint(0, h-10)
        shift = random.randint(-20, 20)
        glitched[y:y+10] = np.roll(glitched[y:y+10], shift, axis=1)
    return glitched

def vintage_effect(image):
    image = image.astype(np.float32)
    image = image * np.array([0.8, 0.9, 1.0])
    noise = np.random.normal(0, 10, image.shape).astype(np.float32)
    image = np.clip(image + noise, 0, 255).astype(np.uint8)
    return sepia_transformation(image)

def sketch_effect(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    inv = 255 - gray
    blur = cv2.GaussianBlur(inv, (21, 21), 0)
    sketch = cv2.divide(gray, 255 - blur, scale=256)
    return cv2.cvtColor(sketch, cv2.COLOR_GRAY2RGB)

def mosaic_effect(image, rect, block_size=10):
    (x1, y1, x2, y2) = rect
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(image.shape[1], x2), min(image.shape[0], y2)
    if x2 <= x1 or y2 <= y1:
        return image
    roi = image[y1:y2, x1:x2].copy()
    h, w = roi.shape[:2]
    for i in range(0, h, block_size):
        for j in range(0, w, block_size):
            block = roi[i:i+block_size, j:j+block_size]
            if block.size > 0:
                mean_color = block.mean(axis=(0, 1)).astype(np.uint8)
                roi[i:i+block_size, j:j+block_size] = mean_color
    image[y1:y2, x1:x2] = roi
    return image

def adjust_brightness_contrast(image, brightness=0, contrast=1.0):
    image = image.astype(np.float32)
    image = image * contrast + brightness
    return image.round().clip(0, 255).astype(np.uint8)

def histogram_equalization(image):
    img_yuv = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
    return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)

class ImageLabel(QLabel):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.start_pos = None
        self.end_pos = None
        self.drawing = False
        self.brush_mode = False
        self.brush_points = []

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.start_pos = event.pos()
            if self.brush_mode:
                self.brush_points.append([event.pos()])
            self.drawing = True

    def mouseMoveEvent(self, event):
        if self.drawing:
            self.end_pos = event.pos()
            if self.brush_mode:
                self.brush_points[-1].append(event.pos())
                self.parent().apply_brush(self.brush_points)
            self.update()

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.end_pos = event.pos()
            self.drawing = False
            if not self.brush_mode:
                self.parent().apply_mosaic_or_crop(self.start_pos, self.end_pos)
            self.start_pos = None
            self.end_pos = None

class BasicViewer(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("영상처리 - 크리에이티브 편집기")
        self.setGeometry(100, 300, 1200, 800)

        # 메인 레이아웃
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)
        self.main_layout = QVBoxLayout(self.central_widget)
        self.main_layout.setContentsMargins(10, 10, 10, 10)

        # 이미지 표시 레이아웃
        self.image_widget = QWidget()
        self.hbox = QHBoxLayout(self.image_widget)
        self.hbox.setContentsMargins(0, 0, 0, 0)

        # 원본 이미지 라벨
        self.original_label = ImageLabel(self)
        self.original_label.setAlignment(Qt.AlignCenter)
        self.original_label.setMinimumSize(400, 400)
        self.original_label.setText("원본 이미지를 열어보세요")
        self.hbox.addWidget(self.original_label)

        # 변환된 이미지 라벨
        self.transformed_label = QLabel(self)
        self.transformed_label.setAlignment(Qt.AlignCenter)
        self.transformed_label.setMinimumSize(400, 400)
        self.transformed_label.setText("변환된 이미지가 여기에 표시됩니다")
        self.hbox.addWidget(self.transformed_label)

        self.main_layout.addWidget(self.image_widget)

        self.current_image = None
        self.transformed_image = None
        self.brush_mode = False

        # 변환 조절 도킹 위젯
        self.transform_dock = QDockWidget("변환 조절", self)
        self.transform_dock.setAllowedAreas(Qt.RightDockWidgetArea)
        self.transform_dock.setFeatures(QDockWidget.DockWidgetMovable | QDockWidget.DockWidgetClosable)
        self.transform_widget = QWidget()
        self.transform_layout = QVBoxLayout(self.transform_widget)
        self.transform_layout.setContentsMargins(10, 10, 10, 10)

        # 슬라이더 그룹
        self.slider_group = QGroupBox("조절 패널")
        self.slider_layout = QVBoxLayout(self.slider_group)

        # 흑백 변환 슬라이더
        self.grayscale_label = QLabel("흑백 강도: 0%")
        self.grayscale_slider = QSlider(Qt.Horizontal)
        self.grayscale_slider.setRange(0, 100)
        self.grayscale_slider.setValue(0)
        self.grayscale_slider.valueChanged.connect(self.apply_grayscale_transform)
        self.slider_layout.addWidget(self.grayscale_label)
        self.slider_layout.addWidget(self.grayscale_slider)

        # 감마 변환 슬라이더
        self.gamma_label = QLabel("감마 값: 1.0")
        self.gamma_slider = QSlider(Qt.Horizontal)
        self.gamma_slider.setRange(10, 500)
        self.gamma_slider.setValue(100)
        self.gamma_slider.valueChanged.connect(self.apply_gamma_transform)
        self.slider_layout.addWidget(self.gamma_label)
        self.slider_layout.addWidget(self.gamma_slider)

        # 밝기 슬라이더
        self.brightness_label = QLabel("밝기: 0")
        self.brightness_slider = QSlider(Qt.Horizontal)
        self.brightness_slider.setRange(-100, 100)
        self.brightness_slider.setValue(0)
        self.brightness_slider.valueChanged.connect(self.apply_brightness_contrast)
        self.slider_layout.addWidget(self.brightness_label)
        self.slider_layout.addWidget(self.brightness_slider)

        # 대비 슬라이더
        self.contrast_label = QLabel("대비: 1.0")
        self.contrast_slider = QSlider(Qt.Horizontal)
        self.contrast_slider.setRange(10, 300)
        self.contrast_slider.setValue(100)
        self.contrast_slider.valueChanged.connect(self.apply_brightness_contrast)
        self.slider_layout.addWidget(self.contrast_label)
        self.slider_layout.addWidget(self.contrast_slider)

        self.transform_layout.addWidget(self.slider_group)
        self.transform_dock.setWidget(self.transform_widget)
        self.addDockWidget(Qt.RightDockWidgetArea, self.transform_dock)
        self.transform_dock.setVisible(False)

        # 초기 이미지 로드
        self.load_image('forest.jpg')

        self.create_menu()

    def create_menu(self):
        menubar = self.menuBar()

        # 파일 메뉴
        file_menu = menubar.addMenu("파일")
        open_action = QAction("열기", self)
        open_action.triggered.connect(self.open_image_dialog)
        file_menu.addAction(open_action)
        exit_action = QAction("종료", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        # 변환 메뉴
        transform_menu = menubar.addMenu("변환")
        log_action = QAction("로그 변환", self)
        log_action.triggered.connect(self.apply_log_transform)
        transform_menu.addAction(log_action)
        gamma_action = QAction("감마 변환", self)
        gamma_action.triggered.connect(self.apply_gamma_transform_menu)
        transform_menu.addAction(gamma_action)
        control_action = QAction("변환 조절", self)
        control_action.triggered.connect(self.toggle_transform_dock)
        transform_menu.addAction(control_action)

        # 보기 메뉴
        view_menu = menubar.addMenu("보기")
        hist_action = QAction("히스토그램 보기", self)
        hist_action.triggered.connect(self.show_histogram)
        view_menu.addAction(hist_action)
        hist_eq_action = QAction("히스토그램 이퀄라이제이션", self)
        hist_eq_action.triggered.connect(self.apply_histogram_equalization)
        view_menu.addAction(hist_eq_action)
        info_action = QAction("이미지 정보", self)
        info_action.triggered.connect(self.show_image_info)
        view_menu.addAction(info_action)

        # 효과 메뉴
        effect_menu = menubar.addMenu("효과")
        sepia_action = QAction("세피아 톤", self)
        sepia_action.triggered.connect(self.apply_sepia_transform)
        effect_menu.addAction(sepia_action)
        cartoon_action = QAction("만화 효과", self)
        cartoon_action.triggered.connect(self.apply_cartoon_transform)
        effect_menu.addAction(cartoon_action)
        glitch_action = QAction("글리치 효과", self)
        glitch_action.triggered.connect(self.apply_glitch_transform)
        effect_menu.addAction(glitch_action)
        vintage_action = QAction("빈티지 필름", self)
        vintage_action.triggered.connect(self.apply_vintage_transform)
        effect_menu.addAction(vintage_action)
        sketch_action = QAction("스케치 효과", self)
        sketch_action.triggered.connect(self.apply_sketch_transform)
        effect_menu.addAction(sketch_action)

        # 도구 메뉴
        tool_menu = menubar.addMenu("도구")
        mosaic_action = QAction("모자이크", self)
        mosaic_action.triggered.connect(self.enable_mosaic_mode)
        tool_menu.addAction(mosaic_action)
        crop_action = QAction("크롭", self)
        crop_action.triggered.connect(self.enable_crop_mode)
        tool_menu.addAction(crop_action)
        text_action = QAction("텍스트 삽입", self)
        text_action.triggered.connect(self.insert_text)
        tool_menu.addAction(text_action)
        brush_action = QAction("브러시", self)
        brush_action.triggered.connect(self.toggle_brush_mode)
        tool_menu.addAction(brush_action)

    def load_image(self, filename):
        try:
            img = image_tools.load_image(filename, color_channel="BGR")
            if not isinstance(img, np.ndarray):
                raise ValueError(f"Could not load {filename}")
            self.current_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            self.current_image = np.ascontiguousarray(self.current_image, dtype=np.uint8)
            self.transformed_image = None
            self.update_image()
            self.update_transformed_image()
            self.grayscale_slider.setValue(0)
            self.gamma_slider.setValue(100)
            self.brightness_slider.setValue(0)
            self.contrast_slider.setValue(100)
            self.grayscale_label.setText("흑백 강도: 0%")
            self.gamma_label.setText("감마 값: 1.0")
            self.brightness_label.setText("밝기: 0")
            self.contrast_label.setText("대비: 1.0")
            self.brush_mode = False
            self.original_label.brush_mode = False
            self.original_label.brush_points = []
        except Exception as e:
            self.original_label.setText(f"이미지 로드 실패: {str(e)}")
            self.transformed_label.setText("변환된 이미지가 여기에 표시됩니다")

    def open_image_dialog(self):
        filename, _ = QFileDialog.getOpenFileName(
            self,
            "이미지 열기",
            "",
            "Image Files (*.png *.jpg *.jpeg *.bmp *.tif *.tiff)"
        )
        if filename:
            self.load_image(filename)

    def numpy_to_qpixmap(self, image):
        if image is None:
            return None
        h, w, ch = image.shape
        if ch != 3:
            return None
        bytes_per_line = ch * w
        image = np.ascontiguousarray(image, dtype=np.uint8)
        qimage = QImage(image.data, w, h, bytes_per_line, QImage.Format_RGB888)
        if qimage.isNull():
            return None
        return QPixmap.fromImage(qimage)

    def update_image(self):
        pixmap = self.numpy_to_qpixmap(self.current_image)
        if pixmap is not None:
            self.original_label.setPixmap(pixmap.scaled(
                self.original_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation
            ))
        else:
            self.original_label.setText("원본 이미지를 열어보세요")

    def update_transformed_image(self):
        pixmap = self.numpy_to_qpixmap(self.transformed_image)
        if pixmap is not None:
            self.transformed_label.setPixmap(pixmap.scaled(
                self.transformed_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation
            ))
        else:
            self.transformed_label.setText("변환된 이미지가 여기에 표시됩니다")

    def apply_log_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = log_transformation(self.current_image, f_max=self.current_image.max())
        self.update_transformed_image()

    def apply_grayscale_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        weight = self.grayscale_slider.value() / 100.0
        self.grayscale_label.setText(f"흑백 강도: {int(weight * 100)}%")
        self.transformed_image = grayscale_transformation(self.current_image, weight)
        self.update_transformed_image()

    def apply_gamma_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        gamma = self.gamma_slider.value() / 100.0
        self.gamma_label.setText(f"감마 값: {gamma:.2f}")
        self.transformed_image = gamma_transformation(self.current_image, gamma)
        self.update_transformed_image()

    def apply_gamma_transform_menu(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        gamma, ok = QInputDialog.getDouble(
            self, "감마 변환", "감마 값을 입력하세요:", 1.0, 0.1, 5.0, 2
        )
        if ok:
            self.transformed_image = gamma_transformation(self.current_image, gamma)
            self.update_transformed_image()

    def apply_sepia_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = sepia_transformation(self.current_image)
        self.update_transformed_image()

    def apply_cartoon_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = cartoon_effect(self.current_image)
        self.update_transformed_image()

    def apply_glitch_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = glitch_effect(self.current_image)
        self.update_transformed_image()

    def apply_vintage_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = vintage_effect(self.current_image)
        self.update_transformed_image()

    def apply_sketch_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = sketch_effect(self.current_image)
        self.update_transformed_image()

    def apply_brightness_contrast(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        brightness = self.brightness_slider.value()
        contrast = self.contrast_slider.value() / 100.0
        self.brightness_label.setText(f"밝기: {brightness}")
        self.contrast_label.setText(f"대비: {contrast:.2f}")
        self.transformed_image = adjust_brightness_contrast(self.current_image, brightness, contrast)
        self.update_transformed_image()

    def apply_histogram_equalization(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = histogram_equalization(self.current_image)
        self.update_transformed_image()

    def show_histogram(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        fig, ax = plt.subplots(figsize=(6, 4))
        colors = ('r', 'g', 'b')
        for i, color in enumerate(colors):
            hist = np.histogram(self.current_image[:, :, i], bins=256, range=(0, 256))[0]
            ax.plot(hist, color=color, label=f'{color.upper()} 채널')
        ax.set_title('RGB 히스토그램')
        ax.set_xlabel('픽셀 강도')
        ax.set_ylabel('빈도')
        ax.legend()
        ax.grid(True)

        # Matplotlib 플롯을 이미지로 변환
        canvas = FigureCanvasAgg(fig)
        canvas.draw()
        width, height = fig.get_size_inches() * fig.get_dpi()
        width, height = int(width), int(height)
        image = np.frombuffer(canvas.tostring_rgb(), dtype=np.uint8)
        image = image.reshape(height, width, 3)
        self.transformed_image = image
        self.update_transformed_image()
        plt.close(fig)

    def show_image_info(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        h, w, ch = self.current_image.shape
        info = f"크기: {w}x{h}\n채널: {ch}\n데이터 타입: {self.current_image.dtype}"
        self.transformed_label.setText(info)

    def enable_mosaic_mode(self):
        self.brush_mode = False
        self.original_label.brush_mode = False
        self.original_label.brush_points = []
        self.transformed_label.setText("마우스로 원본 이미지에서 모자이크 영역을 선택하세요")

    def enable_crop_mode(self):
        self.brush_mode = False
        self.original_label.brush_mode = False
        self.original_label.brush_points = []
        self.transformed_label.setText("마우스로 원본 이미지에서 크롭 영역을 선택하세요")

    def toggle_brush_mode(self):
        self.brush_mode = not self.brush_mode
        self.original_label.brush_mode = self.brush_mode
        self.original_label.brush_points = []
        if self.brush_mode:
            self.transformed_label.setText("마우스로 원본 이미지에 그림을 그리세요")
        else:
            self.update_transformed_image()

    def apply_mosaic_or_crop(self, start_pos, end_pos):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        x1, y1 = start_pos.x(), start_pos.y()
        x2, y2 = end_pos.x(), end_pos.y()
        rect = (min(x1, x2), min(y1, y2), max(x1, x2), max(y1, y2))
        if self.transformed_label.text().startswith("마우스로 원본 이미지에서 크롭"):
            x1, y1, x2, y2 = rect
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(self.current_image.shape[1], x2), min(self.current_image.shape[0], y2)
            if x2 > x1 and y2 > y1:
                self.current_image = self.current_image[y1:y2, x1:x2].copy()
                self.transformed_image = None
                self.update_image()
                self.update_transformed_image()
        else:
            self.transformed_image = mosaic_effect(self.current_image.copy(), rect, block_size=10)
            self.update_transformed_image()

    def insert_text(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        text, ok = QInputDialog.getText(self, "텍스트 삽입", "삽입할 텍스트를 입력하세요:")
        if ok and text:
            img = self.current_image.copy()
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(img, text, (50, 50), font, 1, (255, 0, 0), 2, cv2.LINE_AA)
            self.transformed_image = img
            self.update_transformed_image()

    def apply_brush(self, brush_points):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        img = self.current_image.copy()
        pixmap = self.numpy_to_qpixmap(img)
        if pixmap is None:
            return
        painter = QPainter(pixmap)
        painter.setPen(QPen(Qt.red, 5, Qt.SolidLine))
        for points in brush_points:
            for i in range(1, len(points)):
                painter.drawLine(points[i-1], points[i])
        painter.end()
        h, w, ch = img.shape
        bytes_per_line = ch * w
        qimage = pixmap.toImage()
        qimage = qimage.convertToFormat(QImage.Format_RGB888)
        ptr = qimage.bits()
        ptr.setsize(h * bytes_per_line)
        self.transformed_image = np.frombuffer(ptr, dtype=np.uint8).reshape(h, w, ch)
        self.update_transformed_image()

    def toggle_transform_dock(self):
        self.transform_dock.setVisible(not self.transform_dock.isVisible())

    def showEvent(self, event):
        super().showEvent(event)
        self.update_image()
        self.update_transformed_image()

    def resizeEvent(self, event):
        super().resizeEvent(event)
        self.update_image()
        self.update_transformed_image()

# QApplication 인스턴스 관리
app = QApplication.instance()
if app is None:
    app = QApplication(sys.argv)
else:
    print("기존 QApplication 인스턴스를 사용합니다.")

viewer = BasicViewer()
viewer.show()

# Jupyter와 일반 스크립트 구분
if 'IPython' in sys.modules:
    app.exec_()
else:
    sys.exit(app.exec_())
```

### 수정된 부분
1. **QtLabel 제거**:
   - 모든 `QtLabel` 참조를 `QLabel`로 변경:
     - `self.grayscale_label = QtLabel("흑백 강도: 0%")` → `self.grayscale_label = QLabel("흑백 강도: 0%")`
     - `self.gamma_label`, `self.brightness_label`, `self.contrast_label`도 동일.
   - 임포트 문에서 `QtLabel` 제거:
     ```python
     from PyQt5.QtWidgets import (
         QApplication, QMainWindow, QLabel, QAction, QFileDialog, QHBoxLayout, QWidget,
         QVBoxLayout, QDockWidget, QSlider, QGroupBox, QInputDialog
     )
     ```

2. **image_tools.py 사용**:
   - 제공된 `image_tools.load_image`를 그대로 사용하되, BGR 출력이므로 `cv2.cvtColor`로 RGB 변환.
   - 버그 주의: `load_image`는 `color_channel` 옵션을 무시하고 항상 `cv2.imread`를 호출. RGB로 로드하려면:
     ```python
     img = image_tools.load_image(filename, color_channel="BGR")
     self.current_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
     ```

3. **히스토그램**:
   - `show_histogram`에서 Matplotlib 플롯을 `transformed_label`에 표시.
   - `FigureCanvasAgg`로 렌더링, NumPy 배열로 변환 후 `QPixmap` 표시.

4. **메뉴 및 기능**:
   - "변환": 로그, 감마, 변환 조절.
   - "보기": 히스토그램, 히스토그램 이퀄라이제이션, 이미지 정보.
   - "효과": 세피아, 만화, 글리치, 빈티지, 스케치.
   - "도구": 모자이크, 크롭, 텍스트, 브러시.
   - 모든 기능은 이전 코드와 동일하게 동작.

### 실행 방법

#### Jupyter Notebook
1. `image_tools.py`를 작업 디렉토리에 저장:
   ```python
   import cv2
   import os
   import matplotlib.pyplot as plt
   import numpy as np

   def load_image(filename: str, color_channel: str = "RGB") -> np.ndarray:
       if not os.path.exists(filename):
           raise FileNotFoundError(f"File {filename} does not exist")
       img = cv2.imread(filename)
       if img is None:
           return ValueError(f"Could not load {filename}")
       return img
   ```
2. 첫 번째 셀:
   ```python
   %gui qt
   ```
3. 두 번째 셀에 위 코드를 붙여넣고 실행.
4. 'forest.jpg' 확인.
5. 실행 후:
   - 창이 열리고 'forest.jpg'가 원본으로 표시.
   - "보기" > "히스토그램 보기"로 `transformed_label`에 히스토그램 표시.
   - "변환", "효과", "도구" 메뉴로 다양한 기능 사용.

#### VS Code 또는 PyCharm
1. `image_tools.py`, `image_viewer.py`, 'forest.jpg' 동일 디렉토리에 저장.
2. 실행:
   ```bash
   python image_viewer.py
   ```
3. `%gui qt` 주석 처리.
4. 동일한 동작 확인.

### 예상 출력
- 1200x800 창에 원본/변환 이미지 나란히 표시.
- "보기" > "히스토그램 보기" 시 `transformed_label`에 RGB 히스토그램 렌더링.
- 도킹 패널(초기 숨김)으로 슬라이더 조절.
- 모자이크, 크롭, 브러시 등 신박한 기능 사용 가능.

### 추가 고려사항
1. **image_tools.py 버그**:
   - `load_image`의 `color_channel` 무시 문제는 코드에서 `cv2.cvtColor`로 해결. 완전한 수정은:
     ```python
     def load_image(filename: str, color_channel: str = "RGB") -> np.ndarray:
         if not os.path.exists(filename):
             raise FileNotFoundError(f"File {filename} does not exist")
         img = cv2.imread(filename)
         if img is None:
             raise ValueError(f"Could not load {filename}")
         if color_channel == "RGB":
             img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
         return img
     ```

2. **추가 기능**:
   - 브러시 색상/두께 선택.
   - 히스토그램 스타일 커스터마이징.

3. **에러 방지**:
   - PyQt5 설치 확인:
     ```bash
     pip install --force-reinstall PyQt5
     ```
   - Matplotlib 백엔드 문제 시 `Agg` 외의 백엔드 테스트.

4. **성능**:
   - 큰 이미지의 경우 히스토그램 렌더링 최적화 고려.

### 추가 요청
- 히스토그램 스타일 변경(예: 크기, 색상).
- 새 기능(예: GIF 생성) 추가.
- 에러 발생 시 로그 공유.

이 코드로 `QtLabel` 에러가 해결되고 요청된 기능이 정상 동작하기를 바랍니다! 추가 질문 있으시면 말씀해주세요.

## 추가 질문 5 (오류 사항)

지금 
AttributeError                            Traceback (most recent call last)
Cell In[9], line 130
    128 if self.brush_mode:
    129     self.brush_points[-1].append(event.pos())
--> 130     self.parent().apply_brush(self.brush_points)
    131 self.update()

AttributeError: 'QWidget' object has no attribute 'apply_brush'
이런 오류 메시지가 자꾸 뜨고 그리고 도구 탭에서 원본 이미지에 마우스를 두어 하라는데 커서를 두고 무슨 짓을 해도 작동이 안돼. 이부분을 고쳐줘

## 추가 답변 5

### 에러 분석: `AttributeError: 'QWidget' object has no attribute 'apply_brush'`

**에러 원인**:
- 에러 메시지는 `ImageLabel` 클래스의 `mouseMoveEvent`에서 `self.parent().apply_brush(self.brush_points)`를 호출할 때 발생합니다.
- `self.parent()`는 `ImageLabel`의 부모 객체로, `QWidget` 타입의 `image_widget`을 반환합니다. 하지만 `apply_brush` 메서드는 `BasicViewer` 클래스에 정의되어 있으므로 `QWidget`에는 존재하지 않습니다.
- 따라서 `self.parent()` 대신 `BasicViewer` 인스턴스에 직접 접근해야 합니다. 이를 위해 `ImageLabel`에서 `BasicViewer`를 참조하도록 수정하거나, 부모 체인을 올바르게 탐색해야 합니다.

**추가 문제: 도구 탭에서 마우스 동작이 작동하지 않음**
- 도구 탭("모자이크", "크롭", "브러시")에서 원본 이미지에 마우스 이벤트를 처리하도록 설계되었지만, 커서가 반응하지 않는다고 하셨습니다.
- 이는 마우스 이벤트 처리 로직(`ImageLabel`의 `mousePressEvent`, `mouseMoveEvent`, `mouseReleaseEvent`)이 올바르게 작동하지 않거나, 이벤트가 `ImageLabel`로 전달되지 않는 문제일 가능성이 큽니다.
- 특히, `apply_mosaic_or_crop`과 `apply_brush` 메서드가 마우스 좌표를 이미지 좌표로 변환하지 않아 실제 이미지 영역과 어긋날 수 있습니다.
- 또 다른 가능성은 `brush_mode`, `mosaic_mode`, `crop_mode` 플래그가 올바르게 설정되지 않아 이벤트가 무시되는 경우입니다.

### 문제점 정리
1. **`apply_brush` 에러**:
   - `ImageLabel`에서 `self.parent()`가 `BasicViewer`가 아닌 `QWidget`을 참조.
   - 해결: `ImageLabel` 생성 시 `BasicViewer`를 명시적으로 참조하도록 수정.
2. **마우스 이벤트 작동 안 함**:
   - 마우스 좌표가 이미지 좌표로 변환되지 않아 모자이크/크롭/브러시가 적용되지 않음.
   - `brush_mode` 등 상태 플래그가 제대로 관리되지 않을 가능성.
   - `ImageLabel`의 마우스 이벤트가 `QLabel`의 기본 동작에 의해 차단될 수 있음.
3. **`image_tools.py` 사용**:
   - 제공된 `load_image`는 BGR로 로드하므로 RGB 변환 필요.
   - `color_channel` 옵션 무시 문제는 코드에서 `cv2.cvtColor`로 처리.

### 해결 방법
1. **apply_brush 에러**:
   - `ImageLabel`에 `BasicViewer` 인스턴스를 전달하여 `apply_brush`를 직접 호출.
   - `parent()` 대신 `self.viewer` 속성을 사용.
2. **마우스 이벤트**:
   - 마우스 좌표를 이미지 좌표로 변환(스케일링된 `QPixmap` 기준).
   - `brush_mode`, `mosaic_mode`, `crop_mode`를 명확히 관리.
   - `ImageLabel`에서 마우스 이벤트 활성화 확인(`setMouseTracking(True)`).
3. **UI 개선**:
   - 모자이크/크롭/브러시 적용 시 실시간 피드백 추가.
   - 상태 메시지(`transformed_label`)로 사용자 안내 강화.

### 수정된 코드

아래 코드는 `apply_brush` 에러를 수정하고, 마우스 이벤트(모자이크, 크롭, 브러시)를 안정적으로 처리하도록 개선했습니다. 히스토그램은 `transformed_label`에 표시되며, 메뉴 구조("변환", "보기", "효과", "도구")는 유지됩니다.

```python
%gui qt  # Jupyter Notebook에서만 실행 (일반 스크립트에서는 주석 처리)

import sys
import numpy as np
import cv2
import matplotlib
matplotlib.use('Agg')  # 비대화형 백엔드
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QLabel, QAction, QFileDialog, QHBoxLayout, QWidget,
    QVBoxLayout, QDockWidget, QSlider, QGroupBox, QInputDialog
)
from PyQt5.QtCore import Qt, QPoint
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen
import image_tools
import random

def log_transformation(image, f_max=255):
    f_max = float(f_max)
    if f_max <= 0 or np.isnan(f_max) or np.isinf(f_max):
        f_max = 255
    image = image.astype(np.float32)
    image = np.clip(image, 0, 255)
    C = 255 / np.log(1 + f_max + 1e-10)
    return (C * np.log(1. + image)).round().clip(0, 255).astype(np.uint8)

def gamma_transformation(image, gamma=1.0):
    image = image.astype(np.float32)
    image = np.clip(image, 0, 255)
    normalized = image / 255.0
    transformed = np.power(normalized, gamma) * 255.0
    return transformed.round().clip(0, 255).astype(np.uint8)

def grayscale_transformation(image, weight=1.0):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray_rgb = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)
    return (image * (1 - weight) + gray_rgb * weight).round().clip(0, 255).astype(np.uint8)

def sepia_transformation(image):
    sepia_matrix = np.array([
        [0.393, 0.769, 0.189],
        [0.349, 0.686, 0.168],
        [0.272, 0.534, 0.131]
    ])
    sepia = np.dot(image.astype(np.float32), sepia_matrix.T)
    return sepia.round().clip(0, 255).astype(np.uint8)

def cartoon_effect(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray = cv2.medianBlur(gray, 5)
    edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 9)
    color = cv2.bilateralFilter(image, 9, 300, 300)
    color = cv2.bitwise_and(color, color, mask=edges)
    return color

def glitch_effect(image):
    h, w = image.shape[:2]
    noise = np.random.normal(0, 25, (h, w, 3)).astype(np.float32)
    glitched = image.astype(np.float32) + noise
    glitched = np.clip(glitched, 0, 255).astype(np.uint8)
    for _ in range(10):
        y = random.randint(0, h-10)
        shift = random.randint(-20, 20)
        glitched[y:y+10] = np.roll(glitched[y:y+10], shift, axis=1)
    return glitched

def vintage_effect(image):
    image = image.astype(np.float32)
    image = image * np.array([0.8, 0.9, 1.0])
    noise = np.random.normal(0, 10, image.shape).astype(np.float32)
    image = np.clip(image + noise, 0, 255).astype(np.uint8)
    return sepia_transformation(image)

def sketch_effect(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    inv = 255 - gray
    blur = cv2.GaussianBlur(inv, (21, 21), 0)
    sketch = cv2.divide(gray, 255 - blur, scale=256)
    return cv2.cvtColor(sketch, cv2.COLOR_GRAY2RGB)

def mosaic_effect(image, rect, block_size=10):
    (x1, y1, x2, y2) = rect
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(image.shape[1], x2), min(image.shape[0], y2)
    if x2 <= x1 or y2 <= y1:
        return image
    roi = image[y1:y2, x1:x2].copy()
    h, w = roi.shape[:2]
    for i in range(0, h, block_size):
        for j in range(0, w, block_size):
            block = roi[i:i+block_size, j:j+block_size]
            if block.size > 0:
                mean_color = block.mean(axis=(0, 1)).astype(np.uint8)
                roi[i:i+block_size, j:j+block_size] = mean_color
    image[y1:y2, x1:x2] = roi
    return image

def adjust_brightness_contrast(image, brightness=0, contrast=1.0):
    image = image.astype(np.float32)
    image = image * contrast + brightness
    return image.round().clip(0, 255).astype(np.uint8)

def histogram_equalization(image):
    img_yuv = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
    return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)

class ImageLabel(QLabel):
    def __init__(self, viewer, parent=None):
        super().__init__(parent)
        self.viewer = viewer  # BasicViewer 인스턴스 참조
        self.start_pos = None
        self.end_pos = None
        self.drawing = False
        self.setMouseTracking(True)  # 마우스 이동 추적 활성화

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.start_pos = event.pos()
            if self.viewer.brush_mode:
                self.viewer.brush_points.append([event.pos()])
            self.drawing = True

    def mouseMoveEvent(self, event):
        if self.drawing:
            self.end_pos = event.pos()
            if self.viewer.brush_mode:
                self.viewer.brush_points[-1].append(event.pos())
                self.viewer.apply_brush(self.viewer.brush_points)
            self.update()

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.end_pos = event.pos()
            self.drawing = False
            if not self.viewer.brush_mode:
                self.viewer.apply_mosaic_or_crop(self.start_pos, self.end_pos)
            self.start_pos = None
            self.end_pos = None

class BasicViewer(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("영상처리 - 크리에이티브 편집기")
        self.setGeometry(100, 300, 1200, 800)

        # 메인 레이아웃
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)
        self.main_layout = QVBoxLayout(self.central_widget)
        self.main_layout.setContentsMargins(10, 10, 10, 10)

        # 이미지 표시 레이아웃
        self.image_widget = QWidget()
        self.hbox = QHBoxLayout(self.image_widget)
        self.hbox.setContentsMargins(0, 0, 0, 0)

        # 원본 이미지 라벨
        self.original_label = ImageLabel(self, self.image_widget)  # viewer=self 전달
        self.original_label.setAlignment(Qt.AlignCenter)
        self.original_label.setMinimumSize(400, 400)
        self.original_label.setText("원본 이미지를 열어보세요")
        self.hbox.addWidget(self.original_label)

        # 변환된 이미지 라벨
        self.transformed_label = QLabel(self)
        self.transformed_label.setAlignment(Qt.AlignCenter)
        self.transformed_label.setMinimumSize(400, 400)
        self.transformed_label.setText("변환된 이미지가 여기에 표시됩니다")
        self.hbox.addWidget(self.transformed_label)

        self.main_layout.addWidget(self.image_widget)

        self.current_image = None
        self.transformed_image = None
        self.brush_mode = False
        self.brush_points = []
        self.mode = None  # 'mosaic', 'crop', None

        # 변환 조절 도킹 위젯
        self.transform_dock = QDockWidget("변환 조절", self)
        self.transform_dock.setAllowedAreas(Qt.RightDockWidgetArea)
        self.transform_dock.setFeatures(QDockWidget.DockWidgetMovable | QDockWidget.DockWidgetClosable)
        self.transform_widget = QWidget()
        self.transform_layout = QVBoxLayout(self.transform_widget)
        self.transform_layout.setContentsMargins(10, 10, 10, 10)

        # 슬라이더 그룹
        self.slider_group = QGroupBox("조절 패널")
        self.slider_layout = QVBoxLayout(self.slider_group)

        # 흑백 변환 슬라이더
        self.grayscale_label = QLabel("흑백 강도: 0%")
        self.grayscale_slider = QSlider(Qt.Horizontal)
        self.grayscale_slider.setRange(0, 100)
        self.grayscale_slider.setValue(0)
        self.grayscale_slider.valueChanged.connect(self.apply_grayscale_transform)
        self.slider_layout.addWidget(self.grayscale_label)
        self.slider_layout.addWidget(self.grayscale_slider)

        # 감마 변환 슬라이더
        self.gamma_label = QLabel("감마 값: 1.0")
        self.gamma_slider = QSlider(Qt.Horizontal)
        self.gamma_slider.setRange(10, 500)
        self.gamma_slider.setValue(100)
        self.gamma_slider.valueChanged.connect(self.apply_gamma_transform)
        self.slider_layout.addWidget(self.gamma_label)
        self.slider_layout.addWidget(self.gamma_slider)

        # 밝기 슬라이더
        self.brightness_label = QLabel("밝기: 0")
        self.brightness_slider = QSlider(Qt.Horizontal)
        self.brightness_slider.setRange(-100, 100)
        self.brightness_slider.setValue(0)
        self.brightness_slider.valueChanged.connect(self.apply_brightness_contrast)
        self.slider_layout.addWidget(self.brightness_label)
        self.slider_layout.addWidget(self.brightness_slider)

        # 대비 슬라이더
        self.contrast_label = QLabel("대비: 1.0")
        self.contrast_slider = QSlider(Qt.Horizontal)
        self.contrast_slider.setRange(10, 300)
        self.contrast_slider.setValue(100)
        self.contrast_slider.valueChanged.connect(self.apply_brightness_contrast)
        self.slider_layout.addWidget(self.contrast_label)
        self.slider_layout.addWidget(self.contrast_slider)

        self.transform_layout.addWidget(self.slider_group)
        self.transform_dock.setWidget(self.transform_widget)
        self.addDockWidget(Qt.RightDockWidgetArea, self.transform_dock)
        self.transform_dock.setVisible(False)

        # 초기 이미지 로드
        self.load_image('forest.jpg')

        self.create_menu()

    def create_menu(self):
        menubar = self.menuBar()

        # 파일 메뉴
        file_menu = menubar.addMenu("파일")
        open_action = QAction("열기", self)
        open_action.triggered.connect(self.open_image_dialog)
        file_menu.addAction(open_action)
        exit_action = QAction("종료", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        # 변환 메뉴
        transform_menu = menubar.addMenu("변환")
        log_action = QAction("로그 변환", self)
        log_action.triggered.connect(self.apply_log_transform)
        transform_menu.addAction(log_action)
        gamma_action = QAction("감마 변환", self)
        gamma_action.triggered.connect(self.apply_gamma_transform_menu)
        transform_menu.addAction(gamma_action)
        control_action = QAction("변환 조절", self)
        control_action.triggered.connect(self.toggle_transform_dock)
        transform_menu.addAction(control_action)

        # 보기 메뉴
        view_menu = menubar.addMenu("보기")
        hist_action = QAction("히스토그램 보기", self)
        hist_action.triggered.connect(self.show_histogram)
        view_menu.addAction(hist_action)
        hist_eq_action = QAction("히스토그램 이퀄라이제이션", self)
        hist_eq_action.triggered.connect(self.apply_histogram_equalization)
        view_menu.addAction(hist_eq_action)
        info_action = QAction("이미지 정보", self)
        info_action.triggered.connect(self.show_image_info)
        view_menu.addAction(info_action)

        # 효과 메뉴
        effect_menu = menubar.addMenu("효과")
        sepia_action = QAction("세피아 톤", self)
        sepia_action.triggered.connect(self.apply_sepia_transform)
        effect_menu.addAction(sepia_action)
        cartoon_action = QAction("만화 효과", self)
        cartoon_action.triggered.connect(self.apply_cartoon_transform)
        effect_menu.addAction(cartoon_action)
        glitch_action = QAction("글리치 효과", self)
        glitch_action.triggered.connect(self.apply_glitch_transform)
        effect_menu.addAction(glitch_action)
        vintage_action = QAction("빈티지 필름", self)
        vintage_action.triggered.connect(self.apply_vintage_transform)
        effect_menu.addAction(vintage_action)
        sketch_action = QAction("스케치 효과", self)
        sketch_action.triggered.connect(self.apply_sketch_transform)
        effect_menu.addAction(sketch_action)

        # 도구 메뉴
        tool_menu = menubar.addMenu("도구")
        mosaic_action = QAction("모자이크", self)
        mosaic_action.triggered.connect(self.enable_mosaic_mode)
        tool_menu.addAction(mosaic_action)
        crop_action = QAction("크롭", self)
        crop_action.triggered.connect(self.enable_crop_mode)
        tool_menu.addAction(crop_action)
        text_action = QAction("텍스트 삽입", self)
        text_action.triggered.connect(self.insert_text)
        tool_menu.addAction(text_action)
        brush_action = QAction("브러시", self)
        brush_action.triggered.connect(self.toggle_brush_mode)
        tool_menu.addAction(brush_action)

    def load_image(self, filename):
        try:
            img = image_tools.load_image(filename, color_channel="BGR")
            if not isinstance(img, np.ndarray):
                raise ValueError(f"Could not load {filename}")
            self.current_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            self.current_image = np.ascontiguousarray(self.current_image, dtype=np.uint8)
            self.transformed_image = None
            self.update_image()
            self.update_transformed_image()
            self.grayscale_slider.setValue(0)
            self.gamma_slider.setValue(100)
            self.brightness_slider.setValue(0)
            self.contrast_slider.setValue(100)
            self.grayscale_label.setText("흑백 강도: 0%")
            self.gamma_label.setText("감마 값: 1.0")
            self.brightness_label.setText("밝기: 0")
            self.contrast_label.setText("대비: 1.0")
            self.brush_mode = False
            self.brush_points = []
            self.mode = None
        except Exception as e:
            self.original_label.setText(f"이미지 로드 실패: {str(e)}")
            self.transformed_label.setText("변환된 이미지가 여기에 표시됩니다")

    def open_image_dialog(self):
        filename, _ = QFileDialog.getOpenFileName(
            self,
            "이미지 열기",
            "",
            "Image Files (*.png *.jpg *.jpeg *.bmp *.tif *.tiff)"
        )
        if filename:
            self.load_image(filename)

    def numpy_to_qpixmap(self, image):
        if image is None:
            return None
        h, w, ch = image.shape
        if ch != 3:
            return None
        bytes_per_line = ch * w
        image = np.ascontiguousarray(image, dtype=np.uint8)
        qimage = QImage(image.data, w, h, bytes_per_line, QImage.Format_RGB888)
        if qimage.isNull():
            return None
        return QPixmap.fromImage(qimage)

    def update_image(self):
        pixmap = self.numpy_to_qpixmap(self.current_image)
        if pixmap is not None:
            self.original_label.setPixmap(pixmap.scaled(
                self.original_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation
            ))
        else:
            self.original_label.setText("원본 이미지를 열어보세요")

    def update_transformed_image(self):
        pixmap = self.numpy_to_qpixmap(self.transformed_image)
        if pixmap is not None:
            self.transformed_label.setPixmap(pixmap.scaled(
                self.transformed_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation
            ))
        else:
            self.transformed_label.setText("변환된 이미지가 여기에 표시됩니다")

    def map_to_image_coords(self, pos, image_shape):
        """마우스 좌표를 이미지 좌표로 변환"""
        if self.current_image is None:
            return pos.x(), pos.y()
        pixmap = self.original_label.pixmap()
        if not pixmap or pixmap.isNull():
            return pos.x(), pos.y()
        label_size = self.original_label.size()
        pixmap_size = pixmap.size()
        scale_x = image_shape[1] / pixmap_size.width()
        scale_y = image_shape[0] / pixmap_size.height()
        x = int(pos.x() * scale_x)
        y = int(pos.y() * scale_y)
        return x, y

    def apply_log_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = log_transformation(self.current_image, f_max=self.current_image.max())
        self.update_transformed_image()

    def apply_grayscale_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        weight = self.grayscale_slider.value() / 100.0
        self.grayscale_label.setText(f"흑백 강도: {int(weight * 100)}%")
        self.transformed_image = grayscale_transformation(self.current_image, weight)
        self.update_transformed_image()

    def apply_gamma_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        gamma = self.gamma_slider.value() / 100.0
        self.gamma_label.setText(f"감마 값: {gamma:.2f}")
        self.transformed_image = gamma_transformation(self.current_image, gamma)
        self.update_transformed_image()

    def apply_gamma_transform_menu(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        gamma, ok = QInputDialog.getDouble(
            self, "감마 변환", "감마 값을 입력하세요:", 1.0, 0.1, 5.0, 2
        )
        if ok:
            self.transformed_image = gamma_transformation(self.current_image, gamma)
            self.update_transformed_image()

    def apply_sepia_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = sepia_transformation(self.current_image)
        self.update_transformed_image()

    def apply_cartoon_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = cartoon_effect(self.current_image)
        self.update_transformed_image()

    def apply_glitch_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = glitch_effect(self.current_image)
        self.update_transformed_image()

    def apply_vintage_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = vintage_effect(self.current_image)
        self.update_transformed_image()

    def apply_sketch_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = sketch_effect(self.current_image)
        self.update_transformed_image()

    def apply_brightness_contrast(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        brightness = self.brightness_slider.value()
        contrast = self.contrast_slider.value() / 100.0
        self.brightness_label.setText(f"밝기: {brightness}")
        self.contrast_label.setText(f"대비: {contrast:.2f}")
        self.transformed_image = adjust_brightness_contrast(self.current_image, brightness, contrast)
        self.update_transformed_image()

    def apply_histogram_equalization(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = histogram_equalization(self.current_image)
        self.update_transformed_image()

    def show_histogram(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        fig, ax = plt.subplots(figsize=(6, 4))
        colors = ('r', 'g', 'b')
        for i, color in enumerate(colors):
            hist = np.histogram(self.current_image[:, :, i], bins=256, range=(0, 256))[0]
            ax.plot(hist, color=color, label=f'{color.upper()} 채널')
        ax.set_title('RGB 히스토그램')
        ax.set_xlabel('픽셀 강도')
        ax.set_ylabel('빈도')
        ax.legend()
        ax.grid(True)

        # Matplotlib 플롯을 이미지로 변환
        canvas = FigureCanvasAgg(fig)
        canvas.draw()
        width, height = fig.get_size_inches() * fig.get_dpi()
        width, height = int(width), int(height)
        image = np.frombuffer(canvas.tostring_rgb(), dtype=np.uint8)
        image = image.reshape(height, width, 3)
        self.transformed_image = image
        self.update_transformed_image()
        plt.close(fig)

    def show_image_info(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        h, w, ch = self.current_image.shape
        info = f"크기: {w}x{h}\n채널: {ch}\n데이터 타입: {self.current_image.dtype}"
        self.transformed_label.setText(info)

    def enable_mosaic_mode(self):
        self.brush_mode = False
        self.brush_points = []
        self.mode = 'mosaic'
        self.transformed_label.setText("마우스로 원본 이미지에서 모자이크 영역을 드래그하세요")

    def enable_crop_mode(self):
        self.brush_mode = False
        self.brush_points = []
        self.mode = 'crop'
        self.transformed_label.setText("마우스로 원본 이미지에서 크롭 영역을 드래그하세요")

    def toggle_brush_mode(self):
        self.brush_mode = not self.brush_mode
        self.brush_points = []
        self.mode = 'brush' if self.brush_mode else None
        if self.brush_mode:
            self.transformed_label.setText("마우스로 원본 이미지에 그림을 그리세요")
        else:
            self.update_transformed_image()

    def apply_mosaic_or_crop(self, start_pos, end_pos):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        # 마우스 좌표를 이미지 좌표로 변환
        x1, y1 = self.map_to_image_coords(start_pos, self.current_image.shape)
        x2, y2 = self.map_to_image_coords(end_pos, self.current_image.shape)
        rect = (min(x1, x2), min(y1, y2), max(x1, x2), max(y1, y2))
        if self.mode == 'crop':
            x1, y1, x2, y2 = rect
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(self.current_image.shape[1], x2), min(self.current_image.shape[0], y2)
            if x2 > x1 and y2 > y1:
                self.current_image = self.current_image[y1:y2, x1:x2].copy()
                self.transformed_image = None
                self.update_image()
                self.update_transformed_image()
        elif self.mode == 'mosaic':
            self.transformed_image = mosaic_effect(self.current_image.copy(), rect, block_size=10)
            self.update_transformed_image()

    def insert_text(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        text, ok = QInputDialog.getText(self, "텍스트 삽입", "삽입할 텍스트를 입력하세요:")
        if ok and text:
            img = self.current_image.copy()
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(img, text, (50, 50), font, 1, (255, 0, 0), 2, cv2.LINE_AA)
            self.transformed_image = img
            self.update_transformed_image()

    def apply_brush(self, brush_points):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        img = self.current_image.copy()
        pixmap = self.numpy_to_qpixmap(img)
        if pixmap is None:
            return
        painter = QPainter(pixmap)
        painter.setPen(QPen(Qt.red, 5, Qt.SolidLine))
        for points in brush_points:
            for i in range(1, len(points)):
                painter.drawLine(points[i-1], points[i])
        painter.end()
        h, w, ch = img.shape
        bytes_per_line = ch * w
        qimage = pixmap.toImage()
        qimage = qimage.convertToFormat(QImage.Format_RGB888)
        ptr = qimage.bits()
        ptr.setsize(h * bytes_per_line)
        self.transformed_image = np.frombuffer(ptr, dtype=np.uint8).reshape(h, w, ch)
        self.update_transformed_image()

    def toggle_transform_dock(self):
        self.transform_dock.setVisible(not self.transform_dock.isVisible())

    def showEvent(self, event):
        super().showEvent(event)
        self.update_image()
        self.update_transformed_image()

    def resizeEvent(self, event):
        super().resizeEvent(event)
        self.update_image()
        self.update_transformed_image()

# QApplication 인스턴스 관리
app = QApplication.instance()
if app is None:
    app = QApplication(sys.argv)
else:
    print("기존 QApplication 인스턴스를 사용합니다.")

viewer = BasicViewer()
viewer.show()

# Jupyter와 일반 스크립트 구분
if 'IPython' in sys.modules:
    app.exec_()
else:
    sys.exit(app.exec_())
```

### 수정된 부분

1. **`apply_brush` 에러 해결**:
   - `ImageLabel` 클래스에 `viewer` 속성 추가:
     ```python
     def __init__(self, viewer, parent=None):
         super().__init__(parent)
         self.viewer = viewer
     ```
   - `BasicViewer`에서 `ImageLabel` 생성 시 `self` 전달:
     ```python
     self.original_label = ImageLabel(self, self.image_widget)
     ```
   - `mouseMoveEvent`에서 `self.viewer.apply_brush` 호출:
     ```python
     self.viewer.apply_brush(self.viewer.brush_points)
     ```

2. **마우스 이벤트 개선**:
   - `ImageLabel`에 `setMouseTracking(True)` 추가로 마우스 이동 이벤트 활성화.
   - 마우스 좌표를 이미지 좌표로 변환하는 `map_to_image_coords` 메서드 추가:
     ```python
     def map_to_image_coords(self, pos, image_shape):
         pixmap = self.original_label.pixmap()
         if not pixmap or pixmap.isNull():
             return pos.x(), pos.y()
         label_size = self.original_label.size()
         pixmap_size = pixmap.size()
         scale_x = image_shape[1] / pixmap_size.width()
         scale_y = image_shape[0] / pixmap_size.height()
         x = int(pos.x() * scale_x)
         y = int(pos.y() * scale_y)
         return x, y
     ```
   - `apply_mosaic_or_crop`에서 변환된 좌표 사용.

3. **모드 관리**:
   - `mode` 속성 추가('mosaic', 'crop', 'brush', None)로 상태 명확히 관리.
   - `enable_mosaic_mode`, `enable_crop_mode`, `toggle_brush_mode`에서 `mode`와 `brush_mode` 설정:
     ```python
     def enable_mosaic_mode(self):
         self.brush_mode = False
         self.brush_points = []
         self.mode = 'mosaic'
         self.transformed_label.setText("마우스로 원본 이미지에서 모자이크 영역을 드래그하세요")
     ```

4. **브러시 데이터**:
   - `brush_points`를 `BasicViewer`로 이동하여 상태 유지.
   - `ImageLabel`은 `viewer.brush_points` 참조.

5. **UI 피드백**:
   - 모자이크/크롭/브러시 선택 시 `transformed_label`에 안내 메시지 표시.
   - 실시간 브러시 그리기 반영.

### 실행 방법

#### Jupyter Notebook
1. `image_tools.py`를 작업 디렉토리에 저장:
   ```python
   import cv2
   import os
   import matplotlib.pyplot as plt
   import numpy as np

   def load_image(filename: str, color_channel: str = "RGB") -> np.ndarray:
       if not os.path.exists(filename):
           raise FileNotFoundError(f"File {filename} does not exist")
       img = cv2.imread(filename)
       if img is None:
           return ValueError(f"Could not load {filename}")
       return img
   ```
2. 첫 번째 셀:
   ```python
   %gui qt
   ```
3. 두 번째 셀에 위 코드를 붙여넣고 실행.
4. 'forest.jpg' 확인.
5. 실행 후:
   - 창이 열리고 'forest.jpg'가 원본으로 표시.
   - "도구" > "모자이크" 선택, 원본 이미지에서 마우스 드래그로 모자이크 적용.
   - "도구" > "크롭" 선택, 드래그로 영역 자르기.
   - "도구" > "브러시" 선택, 드래그로 빨간색 선 그리기.
   - "보기" > "히스토그램 보기"로 변환 공간에 히스토그램 표시.

#### VS Code 또는 PyCharm
1. `image_tools.py`, `image_viewer.py`, 'forest.jpg' 동일 디렉토리에 저장.
2. 실행:
   ```bash
   python image_viewer.py
   ```
3. `%gui qt` 주석 처리.
4. 동일한 동작 확인.

### 예상 출력
- 1200x800 창에 원본/변환 이미지 나란히 표시.
- "도구" 메뉴 선택 시:
  - **모자이크**: 원본 이미지에서 드래그로 선택한 영역에 모자이크 적용, 결과는 변환 이미지에 표시.
  - **크롭**: 드래그로 선택한 영역으로 원본 이미지 자름.
  - **브러시**: 드래그로 빨간색 선 그리기, 실시간으로 변환 이미지에 반영.
- 히스토그램은 `transformed_label`에 렌더링.
- 도킹 패널로 슬라이더 조절.

### 추가 고려사항
1. **image_tools.py**:
   - `load_image`의 `color_channel` 무시 문제는 `cv2.cvtColor`로 해결. 완전 수정:
     ```python
     def load_image(filename: str, color_channel: str = "RGB") -> np.ndarray:
         if not os.path.exists(filename):
             raise FileNotFoundError(f"File {filename} does not exist")
         img = cv2.imread(filename)
         if img is None:
             raise ValueError(f"Could not load {filename}")
         if color_channel == "RGB":
             img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
         return img
     ```

2. **마우스 이벤트 디버깅**:
   - 마우스 이벤트가 여전히 작동하지 않으면:
     - PyQt5 버전 확인(`pip show PyQt5`).
     - `ImageLabel`의 이벤트 로그 추가:
       ```python
       def mousePressEvent(self, event):
           print(f"Mouse pressed at {event.pos()}")
           super().mousePressEvent(event)
       ```

3. **추가 기능**:
   - 브러시 색상/두께 선택 다이얼로그.
   - 모자이크 블록 크기 조절 슬라이더.

4. **성능**:
   - 큰 이미지의 경우 브러시/모자이크 실시간 처리 최적화.

### 추가 요청
- 마우스 이벤트 문제 지속 시 로그 공유.
- 브러시 색상 변경 등 UI 커스터마이징.
- 'forest.jpg' 업로드 가능 시 테스트.

이 코드로 `apply_brush` 에러와 마우스 이벤트 문제가 해결되기를 바랍니다! 추가 질문 있으시면 말씀해주세요.

## 추가 질문 6 (오류 사항)

도구 탭에서 브러시, 모자이크, 크롭 등등 지금 원본 이미지에서 그리는 위치랑 적용되는 위치가 다르게 작동되는 버그가 있어. 이 부분을 고쳐줘

## 추가 답변 6

### 문제 분석: 도구 탭에서 마우스 위치와 적용 위치 불일치

**문제점**:
- 도구 탭("브러시", "모자이크", "크롭")에서 원본 이미지(`original_label`) 위에서 마우스로 그리는 위치와 실제로 적용되는 위치(변환 이미지 `transformed_label` 또는 원본 이미지 수정)가 달라지는 버그가 있습니다.
- 이는 마우스 좌표(`QPoint`)를 이미지 좌표(`numpy` 배열의 픽셀)로 변환하는 과정에서 스케일링과 오프셋이 제대로 처리되지 않아 발생합니다.
- `map_to_image_coords` 메서드는 `QPixmap`의 크기와 원본 이미지의 크기 간의 비율을 계산하지만, `QLabel`의 실제 표시 크기와 이미지의 스케일링, 그리고 중앙 정렬(`Qt.KeepAspectRatio`)로 인한 오프셋을 충분히 고려하지 않았습니다.
- 특히, `QLabel`의 `scaled` 메서드가 이미지를 비율 유지하며 표시하므로, 이미지 주변의 여백(패딩)과 실제 픽셀 위치가 어긋납니다.

**추가 문제**:
- `apply_brush`에서 `QPixmap`에 직접 그림을 그리지만, 좌표 변환 후 이미지 배열에 반영되지 않을 수 있습니다.
- `mosaic_effect`와 크롭 작업도 좌표 불일치로 인해 잘못된 영역에 적용됩니다.
- 상태 관리(`brush_mode`, `mode`)는 정상 작동하지만, 좌표 변환 로직이 핵심 문제입니다.

### 해결 방법
1. **정확한 좌표 변환**:
   - `map_to_image_coords`를 개선하여 `QLabel`의 실제 표시 영역, `QPixmap`의 스케일링, 중앙 정렬 오프셋을 고려.
   - 마우스 좌표를 원본 이미지의 픽셀 좌표로 정확히 매핑.
2. **브러시**:
   - `apply_brush`에서 `QPixmap` 대신 원본 이미지(`numpy` 배열)에 직접 그리기.
   - 변환된 좌표를 사용하여 실시간으로 선을 그림.
3. **모자이크와 크롭**:
   - `apply_mosaic_or_crop`에서 변환된 좌표를 사용하여 정확한 영역 처리.
   - 실시간 드래그 피드백(예: 사각형 표시) 추가는 복잡하므로 생략하되, 필요 시 요청.
4. **상태 관리**:
   - `mode`와 `brush_mode`를 유지하되, 디버깅용 로그 추가.
5. **UI 안내**:
   - `transformed_label`에 명확한 안내 메시지 표시.

### 수정된 코드

아래 코드는 좌표 변환 로직을 개선하여 브러시, 모자이크, 크롭의 위치 불일치를 수정했습니다. 히스토그램은 `transformed_label`에 표시되며, 메뉴 구조("변환", "보기", "효과", "도구")는 유지됩니다.

```python
%gui qt  # Jupyter Notebook에서만 실행 (일반 스크립트에서는 주석 처리)

import sys
import numpy as np
import cv2
import matplotlib
matplotlib.use('Agg')  # 비대화형 백엔드
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QLabel, QAction, QFileDialog, QHBoxLayout, QWidget,
    QVBoxLayout, QDockWidget, QSlider, QGroupBox, QInputDialog
)
from PyQt5.QtCore import Qt, QPoint
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen
import image_tools
import random

def log_transformation(image, f_max=255):
    f_max = float(f_max)
    if f_max <= 0 or np.isnan(f_max) or np.isinf(f_max):
        f_max = 255
    image = image.astype(np.float32)
    image = np.clip(image, 0, 255)
    C = 255 / np.log(1 + f_max + 1e-10)
    return (C * np.log(1. + image)).round().clip(0, 255).astype(np.uint8)

def gamma_transformation(image, gamma=1.0):
    image = image.astype(np.float32)
    image = np.clip(image, 0, 255)
    normalized = image / 255.0
    transformed = np.power(normalized, gamma) * 255.0
    return transformed.round().clip(0, 255).astype(np.uint8)

def grayscale_transformation(image, weight=1.0):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray_rgb = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)
    return (image * (1 - weight) + gray_rgb * weight).round().clip(0, 255).astype(np.uint8)

def sepia_transformation(image):
    sepia_matrix = np.array([
        [0.393, 0.769, 0.189],
        [0.349, 0.686, 0.168],
        [0.272, 0.534, 0.131]
    ])
    sepia = np.dot(image.astype(np.float32), sepia_matrix.T)
    return sepia.round().clip(0, 255).astype(np.uint8)

def cartoon_effect(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray = cv2.medianBlur(gray, 5)
    edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 9)
    color = cv2.bilateralFilter(image, 9, 300, 300)
    color = cv2.bitwise_and(color, color, mask=edges)
    return color

def glitch_effect(image):
    h, w = image.shape[:2]
    noise = np.random.normal(0, 25, (h, w, 3)).astype(np.float32)
    glitched = image.astype(np.float32) + noise
    glitched = np.clip(glitched, 0, 255).astype(np.uint8)
    for _ in range(10):
        y = random.randint(0, h-10)
        shift = random.randint(-20, 20)
        glitched[y:y+10] = np.roll(glitched[y:y+10], shift, axis=1)
    return glitched

def vintage_effect(image):
    image = image.astype(np.float32)
    image = image * np.array([0.8, 0.9, 1.0])
    noise = np.random.normal(0, 10, image.shape).astype(np.float32)
    image = np.clip(image + noise, 0, 255).astype(np.uint8)
    return sepia_transformation(image)

def sketch_effect(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    inv = 255 - gray
    blur = cv2.GaussianBlur(inv, (21, 21), 0)
    sketch = cv2.divide(gray, 255 - blur, scale=256)
    return cv2.cvtColor(sketch, cv2.COLOR_GRAY2RGB)

def mosaic_effect(image, rect, block_size=10):
    (x1, y1, x2, y2) = rect
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(image.shape[1], x2), min(image.shape[0], y2)
    if x2 <= x1 or y2 <= y1:
        return image
    roi = image[y1:y2, x1:x2].copy()
    h, w = roi.shape[:2]
    for i in range(0, h, block_size):
        for j in range(0, w, block_size):
            block = roi[i:i+block_size, j:j+block_size]
            if block.size > 0:
                mean_color = block.mean(axis=(0, 1)).astype(np.uint8)
                roi[i:i+block_size, j:j+block_size] = mean_color
    image[y1:y2, x1:x2] = roi
    return image

def adjust_brightness_contrast(image, brightness=0, contrast=1.0):
    image = image.astype(np.float32)
    image = image * contrast + brightness
    return image.round().clip(0, 255).astype(np.uint8)

def histogram_equalization(image):
    img_yuv = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
    return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)

class ImageLabel(QLabel):
    def __init__(self, viewer, parent=None):
        super().__init__(parent)
        self.viewer = viewer
        self.start_pos = None
        self.end_pos = None
        self.drawing = False
        self.setMouseTracking(True)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.start_pos = event.pos()
            if self.viewer.brush_mode:
                self.viewer.brush_points.append([self.viewer.map_to_image_coords(event.pos(), self.viewer.current_image.shape)])
            self.drawing = True

    def mouseMoveEvent(self, event):
        if self.drawing:
            self.end_pos = event.pos()
            if self.viewer.brush_mode:
                self.viewer.brush_points[-1].append(self.viewer.map_to_image_coords(event.pos(), self.viewer.current_image.shape))
                self.viewer.apply_brush(self.viewer.brush_points)
            self.update()

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.end_pos = event.pos()
            self.drawing = False
            if not self.viewer.brush_mode:
                self.viewer.apply_mosaic_or_crop(self.start_pos, self.end_pos)
            self.start_pos = None
            self.end_pos = None

class BasicViewer(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("영상처리 - 크리에이티브 편집기")
        self.setGeometry(100, 300, 1200, 800)

        # 메인 레이아웃
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)
        self.main_layout = QVBoxLayout(self.central_widget)
        self.main_layout.setContentsMargins(10, 10, 10, 10)

        # 이미지 표시 레이아웃
        self.image_widget = QWidget()
        self.hbox = QHBoxLayout(self.image_widget)
        self.hbox.setContentsMargins(0, 0, 0, 0)

        # 원본 이미지 라벨
        self.original_label = ImageLabel(self, self.image_widget)
        self.original_label.setAlignment(Qt.AlignCenter)
        self.original_label.setMinimumSize(400, 400)
        self.original_label.setText("원본 이미지를 열어보세요")
        self.hbox.addWidget(self.original_label)

        # 변환된 이미지 라벨
        self.transformed_label = QLabel(self)
        self.transformed_label.setAlignment(Qt.AlignCenter)
        self.transformed_label.setMinimumSize(400, 400)
        self.transformed_label.setText("변환된 이미지가 여기에 표시됩니다")
        self.hbox.addWidget(self.transformed_label)

        self.main_layout.addWidget(self.image_widget)

        self.current_image = None
        self.transformed_image = None
        self.brush_mode = False
        self.brush_points = []
        self.mode = None

        # 변환 조절 도킹 위젯
        self.transform_dock = QDockWidget("변환 조절", self)
        self.transform_dock.setAllowedAreas(Qt.RightDockWidgetArea)
        self.transform_dock.setFeatures(QDockWidget.DockWidgetMovable | QDockWidget.DockWidgetClosable)
        self.transform_widget = QWidget()
        self.transform_layout = QVBoxLayout(self.transform_widget)
        self.transform_layout.setContentsMargins(10, 10, 10, 10)

        # 슬라이더 그룹
        self.slider_group = QGroupBox("조절 패널")
        self.slider_layout = QVBoxLayout(self.slider_group)

        # 흑백 변환 슬라이더
        self.grayscale_label = QLabel("흑백 강도: 0%")
        self.grayscale_slider = QSlider(Qt.Horizontal)
        self.grayscale_slider.setRange(0, 100)
        self.grayscale_slider.setValue(0)
        self.grayscale_slider.valueChanged.connect(self.apply_grayscale_transform)
        self.slider_layout.addWidget(self.grayscale_label)
        self.slider_layout.addWidget(self.grayscale_slider)

        # 감마 변환 슬라이더
        self.gamma_label = QLabel("감마 값: 1.0")
        self.gamma_slider = QSlider(Qt.Horizontal)
        self.gamma_slider.setRange(10, 500)
        self.gamma_slider.setValue(100)
        self.gamma_slider.valueChanged.connect(self.apply_gamma_transform)
        self.slider_layout.addWidget(self.gamma_label)
        self.slider_layout.addWidget(self.gamma_slider)

        # 밝기 슬라이더
        self.brightness_label = QLabel("밝기: 0")
        self.brightness_slider = QSlider(Qt.Horizontal)
        self.brightness_slider.setRange(-100, 100)
        self.brightness_slider.setValue(0)
        self.brightness_slider.valueChanged.connect(self.apply_brightness_contrast)
        self.slider_layout.addWidget(self.brightness_label)
        self.slider_layout.addWidget(self.brightness_slider)

        # 대비 슬라이더
        self.contrast_label = QLabel("대비: 1.0")
        self.contrast_slider = QSlider(Qt.Horizontal)
        self.contrast_slider.setRange(10, 300)
        self.contrast_slider.setValue(100)
        self.contrast_slider.valueChanged.connect(self.apply_brightness_contrast)
        self.slider_layout.addWidget(self.contrast_label)
        self.slider_layout.addWidget(self.contrast_slider)

        self.transform_layout.addWidget(self.slider_group)
        self.transform_dock.setWidget(self.transform_widget)
        self.addDockWidget(Qt.RightDockWidgetArea, self.transform_dock)
        self.transform_dock.setVisible(False)

        # 초기 이미지 로드
        self.load_image('forest.jpg')

        self.create_menu()

    def create_menu(self):
        menubar = self.menuBar()

        # 파일 메뉴
        file_menu = menubar.addMenu("파일")
        open_action = QAction("열기", self)
        open_action.triggered.connect(self.open_image_dialog)
        file_menu.addAction(open_action)
        exit_action = QAction("종료", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        # 변환 메뉴
        transform_menu = menubar.addMenu("변환")
        log_action = QAction("로그 변환", self)
        log_action.triggered.connect(self.apply_log_transform)
        transform_menu.addAction(log_action)
        gamma_action = QAction("감마 변환", self)
        gamma_action.triggered.connect(self.apply_gamma_transform_menu)
        transform_menu.addAction(gamma_action)
        control_action = QAction("변환 조절", self)
        control_action.triggered.connect(self.toggle_transform_dock)
        transform_menu.addAction(control_action)

        # 보기 메뉴
        view_menu = menubar.addMenu("보기")
        hist_action = QAction("히스토그램 보기", self)
        hist_action.triggered.connect(self.show_histogram)
        view_menu.addAction(hist_action)
        hist_eq_action = QAction("히스토그램 이퀄라이제이션", self)
        hist_eq_action.triggered.connect(self.apply_histogram_equalization)
        view_menu.addAction(hist_eq_action)
        info_action = QAction("이미지 정보", self)
        info_action.triggered.connect(self.show_image_info)
        view_menu.addAction(info_action)

        # 효과 메뉴
        effect_menu = menubar.addMenu("효과")
        sepia_action = QAction("세피아 톤", self)
        sepia_action.triggered.connect(self.apply_sepia_transform)
        effect_menu.addAction(sepia_action)
        cartoon_action = QAction("만화 효과", self)
        cartoon_action.triggered.connect(self.apply_cartoon_transform)
        effect_menu.addAction(cartoon_action)
        glitch_action = QAction("글리치 효과", self)
        glitch_action.triggered.connect(self.apply_glitch_transform)
        effect_menu.addAction(glitch_action)
        vintage_action = QAction("빈티지 필름", self)
        vintage_action.triggered.connect(self.apply_vintage_transform)
        effect_menu.addAction(vintage_action)
        sketch_action = QAction("스케치 효과", self)
        sketch_action.triggered.connect(self.apply_sketch_transform)
        effect_menu.addAction(sketch_action)

        # 도구 메뉴
        tool_menu = menubar.addMenu("도구")
        mosaic_action = QAction("모자이크", self)
        mosaic_action.triggered.connect(self.enable_mosaic_mode)
        tool_menu.addAction(mosaic_action)
        crop_action = QAction("크롭", self)
        crop_action.triggered.connect(self.enable_crop_mode)
        tool_menu.addAction(crop_action)
        text_action = QAction("텍스트 삽입", self)
        text_action.triggered.connect(self.insert_text)
        tool_menu.addAction(text_action)
        brush_action = QAction("브러시", self)
        brush_action.triggered.connect(self.toggle_brush_mode)
        tool_menu.addAction(brush_action)

    def load_image(self, filename):
        try:
            img = image_tools.load_image(filename, color_channel="BGR")
            if not isinstance(img, np.ndarray):
                raise ValueError(f"Could not load {filename}")
            self.current_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            self.current_image = np.ascontiguousarray(self.current_image, dtype=np.uint8)
            self.transformed_image = None
            self.update_image()
            self.update_transformed_image()
            self.grayscale_slider.setValue(0)
            self.gamma_slider.setValue(100)
            self.brightness_slider.setValue(0)
            self.contrast_slider.setValue(100)
            self.grayscale_label.setText("흑백 강도: 0%")
            self.gamma_label.setText("감마 값: 1.0")
            self.brightness_label.setText("밝기: 0")
            self.contrast_label.setText("대비: 1.0")
            self.brush_mode = False
            self.brush_points = []
            self.mode = None
        except Exception as e:
            self.original_label.setText(f"이미지 로드 실패: {str(e)}")
            self.transformed_label.setText("변환된 이미지가 여기에 표시됩니다")

    def open_image_dialog(self):
        filename, _ = QFileDialog.getOpenFileName(
            self,
            "이미지 열기",
            "",
            "Image Files (*.png *.jpg *.jpeg *.bmp *.tif *.tiff)"
        )
        if filename:
            self.load_image(filename)

    def numpy_to_qpixmap(self, image):
        if image is None:
            return None
        h, w, ch = image.shape
        if ch != 3:
            return None
        bytes_per_line = ch * w
        image = np.ascontiguousarray(image, dtype=np.uint8)
        qimage = QImage(image.data, w, h, bytes_per_line, QImage.Format_RGB888)
        if qimage.isNull():
            return None
        return QPixmap.fromImage(qimage)

    def update_image(self):
        pixmap = self.numpy_to_qpixmap(self.current_image)
        if pixmap is not None:
            self.original_label.setPixmap(pixmap.scaled(
                self.original_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation
            ))
        else:
            self.original_label.setText("원본 이미지를 열어보세요")

    def update_transformed_image(self):
        pixmap = self.numpy_to_qpixmap(self.transformed_image)
        if pixmap is not None:
            self.transformed_label.setPixmap(pixmap.scaled(
                self.transformed_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation
            ))
        else:
            self.transformed_label.setText("변환된 이미지가 여기에 표시됩니다")

    def map_to_image_coords(self, pos, image_shape):
        """마우스 좌표를 이미지 좌표로 변환"""
        if self.current_image is None:
            return pos.x(), pos.y()
        pixmap = self.original_label.pixmap()
        if not pixmap or pixmap.isNull():
            return pos.x(), pos.y()

        label_size = self.original_label.size()
        pixmap_size = pixmap.size()

        # QLabel 내에서 픽스맵의 실제 표시 영역 계산
        aspect_ratio = pixmap_size.width() / pixmap_size.height()
        label_aspect = label_size.width() / label_size.height()

        if aspect_ratio > label_aspect:
            # 픽스맵이 너비 기준으로 스케일링
            scaled_width = label_size.width()
            scaled_height = int(scaled_width / aspect_ratio)
            offset_x = 0
            offset_y = (label_size.height() - scaled_height) // 2
        else:
            # 픽스맵이 높이 기준으로 스케일링
            scaled_height = label_size.height()
            scaled_width = int(scaled_height * aspect_ratio)
            offset_x = (label_size.width() - scaled_width) // 2
            offset_y = 0

        # 마우스 좌표를 픽스맵 좌표로 변환
        x = pos.x() - offset_x
        y = pos.y() - offset_y

        # 픽스맵 좌표를 원본 이미지 좌표로 변환
        scale_x = image_shape[1] / scaled_width
        scale_y = image_shape[0] / scaled_height

        img_x = int(x * scale_x)
        img_y = int(y * scale_y)

        # 이미지 범위 내로 제한
        img_x = max(0, min(img_x, image_shape[1] - 1))
        img_y = max(0, min(img_y, image_shape[0] - 1))

        return img_x, img_y

    def apply_log_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = log_transformation(self.current_image, f_max=self.current_image.max())
        self.update_transformed_image()

    def apply_grayscale_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        weight = self.grayscale_slider.value() / 100.0
        self.grayscale_label.setText(f"흑백 강도: {int(weight * 100)}%")
        self.transformed_image = grayscale_transformation(self.current_image, weight)
        self.update_transformed_image()

    def apply_gamma_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        gamma = self.gamma_slider.value() / 100.0
        self.gamma_label.setText(f"감마 값: {gamma:.2f}")
        self.transformed_image = gamma_transformation(self.current_image, gamma)
        self.update_transformed_image()

    def apply_gamma_transform_menu(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        gamma, ok = QInputDialog.getDouble(
            self, "감마 변환", "감마 값을 입력하세요:", 1.0, 0.1, 5.0, 2
        )
        if ok:
            self.transformed_image = gamma_transformation(self.current_image, gamma)
            self.update_transformed_image()

    def apply_sepia_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = sepia_transformation(self.current_image)
        self.update_transformed_image()

    def apply_cartoon_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = cartoon_effect(self.current_image)
        self.update_transformed_image()

    def apply_glitch_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = glitch_effect(self.current_image)
        self.update_transformed_image()

    def apply_vintage_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = vintage_effect(self.current_image)
        self.update_transformed_image()

    def apply_sketch_transform(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = sketch_effect(self.current_image)
        self.update_transformed_image()

    def apply_brightness_contrast(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        brightness = self.brightness_slider.value()
        contrast = self.contrast_slider.value() / 100.0
        self.brightness_label.setText(f"밝기: {brightness}")
        self.contrast_label.setText(f"대비: {contrast:.2f}")
        self.transformed_image = adjust_brightness_contrast(self.current_image, brightness, contrast)
        self.update_transformed_image()

    def apply_histogram_equalization(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        self.transformed_image = histogram_equalization(self.current_image)
        self.update_transformed_image()

    def show_histogram(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        fig, ax = plt.subplots(figsize=(6, 4))
        colors = ('r', 'g', 'b')
        for i, color in enumerate(colors):
            hist = np.histogram(self.current_image[:, :, i], bins=256, range=(0, 256))[0]
            ax.plot(hist, color=color, label=f'{color.upper()} 채널')
        ax.set_title('RGB 히스토그램')
        ax.set_xlabel('픽셀 강도')
        ax.set_ylabel('빈도')
        ax.legend()
        ax.grid(True)

        # Matplotlib 플롯을 이미지로 변환
        canvas = FigureCanvasAgg(fig)
        canvas.draw()
        width, height = fig.get_size_inches() * fig.get_dpi()
        width, height = int(width), int(height)
        image = np.frombuffer(canvas.tostring_rgb(), dtype=np.uint8)
        image = image.reshape(height, width, 3)
        self.transformed_image = image
        self.update_transformed_image()
        plt.close(fig)

    def show_image_info(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        h, w, ch = self.current_image.shape
        info = f"크기: {w}x{h}\n채널: {ch}\n데이터 타입: {self.current_image.dtype}"
        self.transformed_label.setText(info)

    def enable_mosaic_mode(self):
        self.brush_mode = False
        self.brush_points = []
        self.mode = 'mosaic'
        self.transformed_label.setText("마우스로 원본 이미지에서 모자이크 영역을 드래그하세요")

    def enable_crop_mode(self):
        self.brush_mode = False
        self.brush_points = []
        self.mode = 'crop'
        self.transformed_label.setText("마우스로 원본 이미지에서 크롭 영역을 드래그하세요")

    def toggle_brush_mode(self):
        self.brush_mode = not self.brush_mode
        self.brush_points = []
        self.mode = 'brush' if self.brush_mode else None
        if self.brush_mode:
            self.transformed_label.setText("마우스로 원본 이미지에 그림을 그리세요")
        else:
            self.update_transformed_image()

    def apply_mosaic_or_crop(self, start_pos, end_pos):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        x1, y1 = self.map_to_image_coords(start_pos, self.current_image.shape)
        x2, y2 = self.map_to_image_coords(end_pos, self.current_image.shape)
        rect = (min(x1, x2), min(y1, y2), max(x1, x2), max(y1, y2))
        if self.mode == 'crop':
            x1, y1, x2, y2 = rect
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(self.current_image.shape[1], x2), min(self.current_image.shape[0], y2)
            if x2 > x1 and y2 > y1:
                self.current_image = self.current_image[y1:y2, x1:x2].copy()
                self.transformed_image = None
                self.update_image()
                self.update_transformed_image()
        elif self.mode == 'mosaic':
            self.transformed_image = mosaic_effect(self.current_image.copy(), rect, block_size=10)
            self.update_transformed_image()

    def insert_text(self):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        text, ok = QInputDialog.getText(self, "텍스트 삽입", "삽입할 텍스트를 입력하세요:")
        if ok and text:
            img = self.current_image.copy()
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(img, text, (50, 50), font, 1, (255, 0, 0), 2, cv2.LINE_AA)
            self.transformed_image = img
            self.update_transformed_image()

    def apply_brush(self, brush_points):
        if self.current_image is None:
            self.transformed_label.setText("이미지 없음")
            return
        img = self.current_image.copy()
        for points in brush_points:
            for i in range(1, len(points)):
                start = points[i-1]
                end = points[i]
                cv2.line(img, (start[0], start[1]), (end[0], end[1]), (255, 0, 0), 5)
        self.transformed_image = img
        self.update_transformed_image()

    def toggle_transform_dock(self):
        self.transform_dock.setVisible(not self.transform_dock.isVisible())

    def showEvent(self, event):
        super().showEvent(event)
        self.update_image()
        self.update_transformed_image()

    def resizeEvent(self, event):
        super().resizeEvent(event)
        self.update_image()
        self.update_transformed_image()

# QApplication 인스턴스 관리
app = QApplication.instance()
if app is None:
    app = QApplication(sys.argv)
else:
    print("기존 QApplication 인스턴스를 사용합니다.")

viewer = BasicViewer()
viewer.show()

# Jupyter와 일반 스크립트 구분
if 'IPython' in sys.modules:
    app.exec_()
else:
    sys.exit(app.exec_())
```

### 수정된 부분

1. **좌표 변환 개선 (`map_to_image_coords`)**:
   - `QLabel`의 실제 표시 영역과 `QPixmap`의 스케일링을 고려:
     - 이미지의 종횡비와 레이블의 종횡비를 비교하여 스케일링 기준(너비 또는 높이) 결정.
     - 중앙 정렬로 인한 오프셋(`offset_x`, `offset_y`) 계산.
     - 마우스 좌표를 픽스맵 좌표로 변환 후, 원본 이미지 좌표로 스케일링.
   - 이미지 범위 내로 좌표 제한:
     ```python
     img_x = max(0, min(img_x, image_shape[1] - 1))
     img_y = max(0, min(img_y, image_shape[0] - 1))
     ```

2. **브러시 개선 (`apply_brush`)**:
   - `QPixmap`과 `QPainter` 대신 `cv2.line`으로 원본 이미지(`numpy` 배열)에 직접 그림.
   - 변환된 이미지 좌표를 사용하여 정확한 위치에 선 그리기:
     ```python
     cv2.line(img, (start[0], start[1]), (end[0], end[1]), (255, 0, 0), 5)
     ```
   - `brush_points`는 이미지 좌표로 저장(`map_to_image_coords` 호출).

3. **모자이크와 크롭**:
   - `apply_mosaic_or_crop`에서 변환된 좌표 사용:
     ```python
     x1, y1 = self.map_to_image_coords(start_pos, self.current_image.shape)
     x2, y2 = self.map_to_image_coords(end_pos, self.current_image.shape)
     ```
   - 모자이크는 변환 이미지에, 크롭은 원본 이미지에 적용.

4. **마우스 이벤트**:
   - `ImageLabel`에서 `mousePressEvent`와 `mouseMoveEvent`가 변환된 좌표를 즉시 사용:
     ```python
     self.viewer.brush_points.append([self.viewer.map_to_image_coords(event.pos(), self.viewer.current_image.shape)])
     ```
   - `setMouseTracking(True)` 유지.

5. **상태 관리**:
   - `mode`와 `brush_mode`로 모자이크, 크롭, 브러시 구분.
   - 각 모드 활성화 시 `brush_points` 초기화.

### 실행 방법

#### Jupyter Notebook
1. `image_tools.py`를 작업 디렉토리에 저장:
   ```python
   import cv2
   import os
   import matplotlib.pyplot as plt
   import numpy as np

   def load_image(filename: str, color_channel: str = "RGB") -> np.ndarray:
       if not os.path.exists(filename):
           raise FileNotFoundError(f"File {filename} does not exist")
       img = cv2.imread(filename)
       if img is None:
           return ValueError(f"Could not load {filename}")
       return img
   ```
2. 첫 번째 셀:
   ```python
   %gui qt
   ```
3. 두 번째 셀에 위 코드를 붙여넣고 실행.
4. 'forest.jpg' 확인.
5. 실행 후:
   - 창이 열리고 'forest.jpg'가 원본으로 표시.
   - "도구" > "브러시" 선택, 원본 이미지에서 드래그로 빨간색 선 그리기(정확한 위치).
   - "도구" > "모자이크" 선택, 드래그로 선택한 영역에 모자이크 적용.
   - "도구" > "크롭" 선택, 드래그로 영역 자르기.
   - "보기" > "히스토그램 보기"로 변환 공간에 히스토그램 표시.

#### VS Code 또는 PyCharm
1. `image_tools.py`, `image_viewer.py`, 'forest.jpg' 동일 디렉토리에 저장.
2. 실행:
   ```bash
   python image_viewer.py
   ```
3. `%gui qt` 주석 처리.
4. 동일한 동작 확인.

### 예상 출력
- 1200x800 창에 원본/변환 이미지 나란히 표시.
- "도구" 메뉴 선택 시:
  - **브러시**: 원본 이미지에서 드래그한 정확한 위치에 빨간색 선이 변환 이미지에 표시.
  - **모자이크**: 드래그한 영역에 모자이크가 정확히 적용.
  - **크롭**: 드래그한 영역으로 원본 이미지가 자름.
- 히스토그램은 `transformed_label`에 렌더링.
- 도킹 패널로 슬라이더 조절.

### 추가 고려사항
1. **image_tools.py**:
   - `load_image`의 `color_channel` 무시 문제는 `cv2.cvtColor`로 해결. 완전 수정:
     ```python
     def load_image(filename: str, color_channel: str = "RGB") -> np.ndarray:
         if not os.path.exists(filename):
             raise FileNotFoundError(f"File {filename} does not exist")
         img = cv2.imread(filename)
         if img is None:
             raise ValueError(f"Could not load {filename}")
         if color_channel == "RGB":
             img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
         return img
     ```

2. **디버깅**:
   - 좌표 변환 문제 지속 시, 로그 추가:
     ```python
     def map_to_image_coords(self, pos, image_shape):
         print(f"Mouse pos: {pos.x()}, {pos.y()}")
         # ... 기존 코드 ...
         print(f"Image coords: {img_x}, {img_y}")
         return img_x, img_y
     ```

3. **추가 기능**:
   - 실시간 드래그 사각형 표시(모자이크/크롭).
   - 브러시 색상/두께 선택 다이얼로그.

4. **성능**:
   - 큰 이미지의 경우 좌표 변환 및 그리기 최적화.

### 추가 요청
- 위치 불일치 문제 지속 시 로그 또는 스크린샷 공유.
- 브러시 색상 변경 등 UI 커스터마이징.
- 'forest.jpg' 업로드 가능 시 테스트.

이 코드로 브러시, 모자이크, 크롭의 위치 불일치 문제가 해결되기를 바랍니다! 추가 질문 있으시면 말씀해주세요.

## 추가 질문 7

지금 크롭인가 크룹인가 그거 복사된 이미지가 아니라 원본 이미지를 훼손시키고 있어 그거 좀 고쳐줘.
그리고 추가로 도구탭에서 브러시(색상 변경 이거 넣어줘), 텍스트 삽입, 크롭 등등 그리고 변환 한것들을 저장할 수 있게 저장 탭을 따로 만들어줘 저장은 jpg 나 png같은 것들로 선택해서 저장할 수 있게 만들어주고

## 추가 답변 7